In [1]:
import typing
import pandas as pd
import re
import functools
import numpy as np
import math


In [2]:
class Token:
    def __init__(self, type: str, value: str):
        self.type = type
        self.value = value

    def __repr__(self):
        return f"Token({self.type}, '{self.value}')"

In [3]:
class ParseTreeNode:
    def __init__(self, type: str, value: typing.Optional[str] = None, children: typing.Optional[typing.List["ParseTreeNode"]] = None):
        self.type = type
        self.value = value
        self.children = children or []
        self.start_index = None
        self.end_index = None

    def __repr__(self):
        return f"ParseTreeNode({self.type}, value={self.value}, children={self.children}, start_index={self.start_index}, end_index={self.end_index})"
    
    def reify(self, function_factory):
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "+" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 + arg1
            else:
                # we need to use late binding
                f = function_factory.get("Add")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        # TODO: this doesn't have the associative property, so it's necessary to flatten the tree if we want to do away with the requirement for parentheses
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "-" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 - arg1
            else:
                # we need to use late binding
                f = function_factory.get("Sub")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if(self.type == "expression") and len(self.children) == 1:
            return self.children[0].reify(function_factory)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "*" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 * arg1
            else:
                # we need to use late binding
                f = function_factory.get("Mul")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        # TODO: this doesn't have the associative property, so it's necessary to flatten the tree if this is actually the plan
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "/" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 / arg1
            else:
                # we need to use late binding
                f = function_factory.get("Div")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "%" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 % arg1
            else:
                # we need to use late binding
                f = function_factory.get("Mod")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "**" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 ** arg1
            else:
                # we need to use late binding
                f = function_factory.get("Pow")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "<" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 < arg1
            else:
                # we need to use late binding
                f = function_factory.get("Lt")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "<=" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 <= arg1
            else:
                # we need to use late binding
                f = function_factory.get("Le")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == ">" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 > arg1
            else:
                # we need to use late binding
                f = function_factory.get("Gt")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == ">=" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 >= arg1
            else:
                # we need to use late binding
                f = function_factory.get("Ge")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "==" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 == arg1
            else:
                # we need to use late binding
                f = function_factory.get("Eq")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "!=" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 != arg1
            else:
                # we need to use late binding
                f = function_factory.get("Ne")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
                
        if self.type == "term" and len(self.children) == 1:
            return self.children[0].reify(function_factory)
        if self.type == "factor" and len(self.children) == 1:
            return self.children[0].reify(function_factory)
        if self.type == "factor" and len(self.children) == 3 and self.children[0].value == "(" and self.children[2].value == ")":
            return self.children[1].reify(function_factory)
        if self.type == "number" and (self.children is None or len(self.children) == 0):
            try:
                return int(self.value)
            except:
                return float(self.value)
        if self.type == "string" and self.children is None or len(self.children) == 0:
            return self.value
            
        if self.type == "factor" and len(self.children) == 4 and self.children[0].type == "identifier" and self.children[1].value == "(" and self.children[2].type == "arguments" and self.children[3].value == ")":

            identifier_node = self.children[0]
            name = identifier_node.value

            try:
                definition = function_factory.get(name)
            except ValueError:
                raise ValueError(f"Function '{name}' not found in the factory.")

            params = {}

            arguments_node = self.children[2]
            param_index = 0
            param_names = list(definition.parameters.keys())

            # Handle named arguments first
            named_params_processed = set()  # Keep track of named params

            def flatten_arguments(arguments_node):
                rv = []
                if arguments_node.children[0].type == "argument":
                    rv.append(arguments_node.children[0])
                    if(len(arguments_node.children) == 3 and arguments_node.children[1].value == ","):
                        rv.extend(flatten_arguments(arguments_node.children[2]))
                return rv
                
            flattened_arguments = flatten_arguments(arguments_node)

            #print(flattened_arguments)
                
            for argument_node in flattened_arguments:
                if len(argument_node.children) == 3 and argument_node.children[1].type == "operator" and argument_node.children[1].value == "=":
                    param_name = argument_node.children[0].value
                    param_value_node = argument_node.children[2]

                    if param_name in named_params_processed: # Skip already processed named parameters
                        continue

                    try:
                        param_def = definition.parameters[param_name]
                    except KeyError:
                        raise ValueError(f"Parameter '{param_name}' not found for indicator '{name}'.")

                    param_value = param_value_node.reify(function_factory)  # Evaluate the value node
                    params[param_name] = param_value
                    named_params_processed.add(param_name) # Add to the set of processed named parameters

            # Next, handle positional arguments (skip named ones)
            for argument_node in flattened_arguments:
                if len(argument_node.children) == 1:  # Positional argument
                    try:
                        param_name = param_names[param_index]
                        if param_name in named_params_processed: # Skip if already named
                            param_index += 1
                            continue

                        param_def = definition.parameters[param_name]
                    except IndexError:
                        raise ValueError(f"Incorrect number of positional parameters for '{name}'.")

                    param_value_node = argument_node.children[0]
                    param_value = param_value_node.reify(function_factory)
                    params[param_name] = param_value
                    param_index += 1



            required_params = set(definition.parameters.keys())
            provided_params = set(params.keys())
            if required_params != provided_params:
                missing = required_params - provided_params
                raise ValueError(f"Missing required parameters for {name}: {missing}")

            return definition.create_function(**params)

        raise ValueError(f"Cannot reify node of type: {self.type} with {len(self.children)} children: {self}")

In [4]:
class GrammarRule:
    def __init__(self, left: str, right: typing.List[str]):
        self.left = left
        self.right = right

    def __repr__(self):
        return f"{self.left} -> {' '.join(self.right)}"

In [5]:
class Grammar:
    def __init__(self, grammar_string: str):
        """Initializes a Grammar object by parsing the grammar string."""
        self.rules = []
        for line in grammar_string.strip().splitlines():
            if line.strip():  # Skip empty lines
                parts = line.split("->")
                if len(parts) != 2:
                    raise ValueError(f"Invalid grammar rule: {line}")
                left = parts[0].strip()
                right = [part.strip() for part in parts[1].split()]
                self.rules.append(GrammarRule(left, right))  # Store rules as attributes

    def build_parse_tree(self, tokens: typing.List["Token"], start_symbol: str = "expression") -> typing.Optional["ParseTreeNode"]:
        """Builds a parse tree from a list of tokens using the grammar rules."""

        def _parse(index: int, nonterminal: str, current_depth=0) -> typing.Optional["ParseTreeNode"]:
            applicable_rules = [rule for rule in self.rules if rule.left == nonterminal]

            if index >= len(tokens):  # End of tokens
                if any(not rule.right for rule in applicable_rules): # Check for a matching epsilon rule
                    return ParseTreeNode(nonterminal, children=[])
                return None # No matching epsilon rule

            if not applicable_rules:
                return None

            for rule in applicable_rules:
                rule_matched = True
                children = []
                current_index = index

                for symbol in rule.right:
                    if current_index >= len(tokens):
                        rule_matched = False
                        break

                    if current_index < len(tokens):
                        token = tokens[current_index]

                        if (symbol == token.type) or (symbol == f'"{token.value}"') or \
                           (symbol == "identifier" and token.type == "identifier") or \
                           (symbol == "number" and token.type == "number") or \
                           (symbol == "string" and token.type == "string") or \
                           (symbol == "operator" and token.type == "operator"):
                            child = ParseTreeNode(token.type, value=token.value)
                            child.start_index = current_index
                            child.end_index = current_index
                            children.append(child)
                            current_index += 1  # Increment for terminal

                        elif any(gr.left == symbol for gr in self.rules):
                            child_node = _parse(current_index, symbol, current_depth + 1)
                            if child_node:
                                children.append(child_node)
                                current_index = child_node.end_index + 1
                            else:
                                rule_matched = False
                                break

                        else:
                            rule_matched = False
                            break

                if rule_matched:
                    node = ParseTreeNode(nonterminal, children=children)
                    node.start_index = children[0].start_index if children else index # Handle epsilon rules where children is empty
                    node.end_index = children[-1].end_index if children else index -1 # Handle epsilon rules where children is empty

                    return node

            return None

        return _parse(0, start_symbol)  # Allow specifying the start symbol
        
    def parse(self, input_string: str, start_symbol: str = "expression"):
        """Parses an input string into a parse tree."""
        tokens = self.tokenize(input_string)  # Tokenize the input string
        return self.build_parse_tree(tokens, start_symbol)

    def tokenize(self, expression: str) -> typing.List[Token]:
        """
        Tokenizes a string expression, splitting on spaces and identifying operators.
        """
    
        # Pattern to match tokens. Note: if we wanted to be really fancy, we would specify the token types in the grammar.
        pattern = r"(\*\*|\*|/|//|%|\+|-|==|!=|<=|>=|<|>|=|!|&&|\|\||&|\||\^|~|<<|>>|\(|\)|\[|\]|\{|\}|,|:|\.|->|@|=|;|\+=|-=|\*=|/=|//=|%=|&=|\|=|\^=|\<<=|>>=)|'([^']+)'|\"([^\"]+)\"|(\d+\.?\d*)|([a-zA-Z_]\w*)"
    
        tokens = []
        for match in re.finditer(pattern, expression):
            operator_match = match.group(1)
            single_quote_match = match.group(2)
            double_quote_match = match.group(3)
            number_match = match.group(4)
            identifier_match = match.group(5)
    
            if operator_match:
                tokens.append(Token("operator", operator_match))
            elif single_quote_match:
                tokens.append(Token("string", single_quote_match))
            elif double_quote_match:
                tokens.append(Token("string", double_quote_match))
            elif number_match:
                tokens.append(Token("number", number_match))
            elif identifier_match:
                tokens.append(Token("identifier", identifier_match))
            else:
                raise ValueError(f"invalid token in {expression}")
    
        return tokens

In [6]:
class FunctionInstance:
    def __init__(self, name: str, parameters: typing.Dict[str, typing.Any], definition):
        self.name = name
        self.parameters = parameters
        self.definition = definition

    def evaluate_parameters(self, data):
        rv = {}

        for k in self.parameters:
            v = self.parameters[k]
            if isinstance(v, FunctionInstance):
                rv[k] = v.calculate(data)
            else:
                rv[k] = v
        return rv

    def calculate(self, data: pd.DataFrame): 
        """
        Screens the data using the screener's definition and parameters.

        Args:
            data: The Pandas DataFrame containing the data.

        Returns:
            A Pandas Dataframe
        """
        return self.definition.calculate(data, self.evaluate_parameters(data)) 

    def __repr__(self):
        params_str = ", ".join(f"{name}={value}" for name, value in self.parameters.items())
        return f"{self.definition.name}({params_str})"

In [7]:
class ParameterType:
    """
    A class for specifying parameters for screeners and indicators.
    """

    def __init__(self,
#                 name: str,
                 data_type: typing.Literal["integer", "real", "boolean", "string"],
                 min_val: typing.Union[int, float, None] = None,
                 max_val: typing.Union[int, float, None] = None,
                 default: typing.Any = None,
                 timeframe_defaults: typing.Dict[typing.Literal["tick", "1s", "5s", "15s", "1m", "2m", "5m", "15m", "1d", "1w", "1M"], typing.Any] = None,
                 increment: typing.Union[int, float, None] = None,
                 allowed_strings: typing.List[str] | None = None):
#        if not isinstance(name, str):
#            raise TypeError("name must be a string")
        if data_type not in ("integer", "real", "boolean", "string", "any"):
            raise ValueError("data_type must be 'integer', 'real', 'boolean', 'string', or 'any'")

        if min_val is not None:
            if data_type == "integer" and not isinstance(min_val, int):
                raise TypeError("min_val must be an integer for integer data_type")
            elif data_type in ("real", "integer") and not isinstance(min_val, (int, float)):
                raise TypeError("min_val must be a number for real or integer data_type")

        if max_val is not None:
            if data_type == "integer" and not isinstance(max_val, int):
                raise TypeError("max_val must be an integer for integer data_type")
            elif data_type in ("real", "integer") and not isinstance(max_val, (int, float)):
                raise TypeError("max_val must be a number for real or integer data_type")

        if timeframe_defaults is not None:
            if not isinstance(timeframe_defaults, dict):
                raise TypeError("timeframe_defaults must be a dictionary")
            for timeframe in timeframe_defaults:
                if timeframe not in ("tick", "1s", "5s", "15s", "1m", "2m", "5m", "15m", "1d", "1w", "1M"):
                    raise ValueError(f"Invalid timeframe: {timeframe}")

        if data_type == "integer" and increment is None:
            increment = 1
        elif data_type == "real" and increment is None:
            increment = 0.01

        if data_type == "string" and allowed_strings is not None and not isinstance(allowed_strings, list):
          raise TypeError("allowed_strings must be a list of strings")

        if data_type != "string" and allowed_strings is not None:
          raise ValueError("allowed_strings can only be specified for string data type")

#        self.name = name
        self.data_type = data_type
        self.min_val = min_val
        self.max_val = max_val
        self.default = default
        self.timeframe_defaults = timeframe_defaults or {}
        self.increment = increment
        self.allowed_strings = allowed_strings

    def get_default(self) -> typing.Any:
        return self.default

    def get_possible_values(self) -> typing.Iterable[typing.Any]:
        if self.data_type == "integer":
            if self.min_val is not None and self.max_val is not None:
                return range(self.min_val, self.max_val + 1)
        elif self.data_type == "real":
            if self.min_val is not None and self.max_val is not None:
                current = self.min_val
                while current <= self.max_val:
                    yield current
                    current += 0.01
        elif self.data_type == "boolean":
            return [True, False]
        elif self.data_type == "string":
            if self.allowed_strings is not None:  # Check if allowed_strings is defined
                return self.allowed_strings  # If defined, return those values
            else:
                return []  # Return an empty list if allowed_strings is None (unrestricted)
        return []

    def __repr__(self):
#        return f"ParameterType(name='{self.name}', data_type='{self.data_type}', min_val={self.min_val}, max_val={self.max_val}, default={self.default}, allowed_strings={self.allowed_strings})"
        return f"ParameterType(data_type='{self.data_type}', min_val={self.min_val}, max_val={self.max_val}, default={self.default}, allowed_strings={self.allowed_strings})"

In [8]:
class FunctionDefinition:
    def __init__(self, name: str, parameters: typing.Dict[str, "ParameterType"], calculation_function, factory=None): 
        if not isinstance(name, str):
            raise TypeError("name must be a string")

        if not isinstance(parameters, dict):
            raise TypeError("parameters must be a dictionary")

        if not all(isinstance(param, ParameterType) for param in parameters.values()):
            raise TypeError("All values in parameters must be ParameterType objects")

        if len(set(parameters.keys())) != len(parameters.keys()): # Check for duplicate keys
            raise ValueError("Parameter names must be unique.")

        if not callable(calculation_function):
            raise TypeError("calculation_function must be callable")

        self.name = name
        self.parameters = parameters
        self.calculation_function = calculation_function
        self.factory = factory

    def create_function(self, **kwargs: typing.Any) -> "FunctionInstance":
        params = {}
        for name, param_def in self.parameters.items():
            value = kwargs.get(name)

            if value is None:
                value = param_def.get_default()

            if param_def.data_type == "integer" and not isinstance(value, int):
                raise TypeError(f"Value for parameter '{name}' must be an integer")
            elif param_def.data_type == "real" and not isinstance(value, (int, float)):
                raise TypeError(f"Value for parameter '{name}' must be a number")
            elif param_def.data_type == "boolean" and not isinstance(value, bool):
                raise TypeError(f"Value for parameter '{name}' must be a boolean")
            elif param_def.data_type == "string" and not isinstance(value, str):
                raise TypeError(f"Value for parameter '{name}' must be a string")
            elif param_def.data_type in ("integer", "real"):
                if param_def.min_val is not None and value < param_def.min_val:  # Check min_val
                    raise ValueError(f"Value for parameter '{name}' must be greater than or equal to {param_def.min_val}")
                if param_def.max_val is not None and value > param_def.max_val:  # Check max_val
                    raise ValueError(f"Value for parameter '{name}' must be less than or equal to {param_def.max_val}")

            if param_def.data_type == "string" and param_def.allowed_strings is not None and value not in param_def.allowed_strings:
                raise ValueError(f"Value {value} is not in allowed strings for parameter {name}")

            params[name] = value

        return FunctionInstance(self.name, params, self)

    def calculate(self, data: pd.DataFrame, params: typing.Dict[str, typing.Any]) -> pd.DataFrame:
        """
        Calculates the function using the provided data and parameters.
        """
        kwargs = params.copy() 
        return self.calculation_function(data, **kwargs)

    def __repr__(self):
        return f"FunctionDefinition(name='{self.name}', parameters={self.parameters}, calculation_function={self.calculation_function.__name__ if hasattr(self.calculation_function, '__name__') else str(self.calculation_function)}, factory={self.factory})"

In [9]:
# FIXME: the grammar and reification modules do not correctly handle order of operations for operations
# that do not have the associative property. An easy workaround is to require parentheses.
# The parser does not handle epsilon correctly either.
# It's possible to alter this to use standard order of operations by permitting them in the grammar,
# flattening contiguous subexpression branches with equal-priority operators before reification,
# and performing the operations iteratively in the desired direction.

# REMINDER for LLM tools: This grammar MUST be an epsilon-free grammar.
# That means that rules may not be terminated explicitly.
# Higher rules have higher priority.
# More than one lookahead token is permitted.
default_grammar_specification = """
expression -> term "+" term
expression -> term "-" term
expression -> term "*" term
expression -> term "/" term
expression -> term "%" term
expression -> term "**" term
expression -> term "<" term
expression -> term "<=" term
expression -> term ">" term
expression -> term ">=" term
expression -> term "==" term
expression -> term "!=" term
expression -> term "&&" term
expression -> term "||" term
expression -> term "^^" term
expression -> term
term -> factor
term -> factor "[" expression "]"
factor -> "(" expression ")"
factor -> number
factor -> string
factor -> "-" factor
factor -> "!" factor
factor -> "+" factor
factor -> identifier "(" arguments ")"
factor -> identifier
factor -> optimization
optimization -> "@" identifier "(" expression "," optimization_arguments ")"
optimization -> "@" identifier "(" expression ")"
optimization_arguments -> optimization_argument "," optimization_arguments
optimization_arguments -> optimization_argument
optimization_argument -> argument
optimization_argument -> optimization_parameter
optimization_parameter -> "@" identifier "=" expression
arguments -> argument "," arguments
arguments -> argument
argument -> identifier "=" expression
argument -> expression
"""

class FunctionFactory:
    """
    A class to manage a suite of function definitions.
    """

    def __init__(self, grammar_specification=default_grammar_specification, should_register_basic_operations=True, should_register_basic_indicators=True, should_register_basic_screeners=True, should_register_basic_portfolio_calculators=True):
        self.function_definitions: typing.Dict[str, Definition] = {}
        self.grammar = Grammar(default_grammar_specification)
        if should_register_basic_operations:
            self.register_basic_operations()
        if should_register_basic_indicators:
            self.register_basic_indicators()
        if should_register_basic_screeners:
            self.register_basic_screeners()
        if should_register_basic_portfolio_calculators:
            self.register_basic_portfolio_calculators()

    def register(self, function_definition):
        """
        Registers a new screener definition.

        Args:
            function_definition: The Definition to register.

        Raises:
            ValueError: If a screener with the same name is already registered.
        """
#        if function_definition.name in self.function_definitions:
#            raise ValueError(f"A screener with the name '{function_definition.name}' is already registered.")
        self.function_definitions[function_definition.name] = function_definition
        function_definition.ffactory = self
        
    def register_basic_operations(self):
        add_a0_param = ParameterType("any")
        add_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Add", {"a0": add_a0_param, "a1": add_a1_param}, calculate_add))

        sub_a0_param = ParameterType("any")
        sub_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Sub", {"a0": sub_a0_param, "a1": sub_a1_param}, calculate_sub))

        mul_a0_param = ParameterType("any")
        mul_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Mul", {"a0": mul_a0_param, "a1": mul_a1_param}, calculate_mul))

        div_a0_param = ParameterType("any")
        div_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Div", {"a0": div_a0_param, "a1": div_a1_param}, calculate_div))

        mod_a0_param = ParameterType("any")
        mod_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Mod", {"a0": mod_a0_param, "a1": mod_a1_param}, calculate_mod))

        pow_a0_param = ParameterType("any")
        pow_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Pow", {"a0": pow_a0_param, "a1": pow_a1_param}, calculate_pow))

        lt_a0_param = ParameterType("any")
        lt_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Lt", {"a0": lt_a0_param, "a1": lt_a1_param}, calculate_lt))

        le_a0_param = ParameterType("any")
        le_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Le", {"a0": le_a0_param, "a1": le_a1_param}, calculate_le))

        gt_a0_param = ParameterType("any")
        gt_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Gt", {"a0": gt_a0_param, "a1": gt_a1_param}, calculate_gt))

        ge_a0_param = ParameterType("any")
        ge_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Ge", {"a0": ge_a0_param, "a1": ge_a1_param}, calculate_ge))

        eq_a0_param = ParameterType("any")
        eq_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Eq", {"a0": eq_a0_param, "a1": eq_a1_param}, calculate_eq))

        ne_a0_param = ParameterType("any")
        ne_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Ne", {"a0": ge_a0_param, "a1": ne_a1_param}, calculate_ne))

    def register_basic_screeners(factory):
        # Top N Screener FunctionDefinition and Registration
        top_n_field_param = ParameterType("any", default="return")  # Example allowed strings
        top_n_n_param = ParameterType("integer", min_val=1, default=5)
        factory.register(FunctionDefinition("TopN", {"field": top_n_field_param, "top_n": top_n_n_param}, top_n_screener_function))
        
        # Percentile Screener Definition and Registration
        percentile_field_param = ParameterType("any", default="return")
        percentile_percentile_param = ParameterType("real", min_val=0.0, max_val=1.0, default=.1)
        factory.register(FunctionDefinition("Percentile", {"field": percentile_field_param, "percentile": percentile_percentile_param}, percentile_screener_function))

    def register_basic_portfolio_calculators(factory):
        calculate_normalize = functools.partial(apply_daily_function, f=do_normalize)
        normalize_weights = ParameterType("any", default="weights")
        factory.register(FunctionDefinition("Normalize", {"weights": normalize_weights}, calculate_normalize))
        
        calculate_abs_threshold = functools.partial(apply_daily_function, f=do_abs_threshold)
        abs_threshold_weights = ParameterType("any", default="weights")
        abs_threshold_threshold = ParameterType("real", default=1e-6)
        factory.register(FunctionDefinition("AbsThreshold", {"weights": abs_threshold_weights, "threshold": abs_threshold_threshold}, calculate_abs_threshold))
        
        calculate_abs_min_cutoff = functools.partial(apply_daily_function, f=do_abs_min_cutoff)
        abs_min_cutoff_weights = ParameterType("any", default="weights")
        abs_min_cutoff_cutoff = ParameterType("real", default=1e-6)
        factory.register(FunctionDefinition("AbsMinCutoff", {"weights": abs_min_cutoff_weights, "minimum": abs_min_cutoff_cutoff}, calculate_abs_min_cutoff))
        
        calculate_abs_max_cutoff = functools.partial(apply_daily_function, f=do_abs_max_cutoff)
        abs_max_cutoff_weights = ParameterType("any", default="weights")
        abs_max_cutoff_cutoff = ParameterType("real", default=1e-6)
        factory.register(FunctionDefinition("AbsMaxCutoff", {"weights": abs_max_cutoff_weights, "maximum": abs_max_cutoff_cutoff}, calculate_abs_max_cutoff)) 
        
    def register_basic_indicators(factory): 
        """Registers basic technical indicator functions with this factory instance."""

        # SMA
        sma_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        calculate_sma = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_sma) # Referencing global do_calculate_sma
        factory.register(FunctionDefinition("SMA", {"length": sma_length_param}, calculate_sma)) # Use factory.register


        # RSI
        rsi_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_rsi = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_rsi) # Referencing global do_calculate_rsi
        factory.register(FunctionDefinition("RSI", {"length": rsi_length_param}, calculate_rsi)) # Use factory.register


        # MACD
        fast_length_param = ParameterType("integer", min_val=1, max_val=100, default=12)
        slow_length_param = ParameterType("integer", min_val=1, max_val=200, default=26)
        signal_length_param = ParameterType("integer", min_val=5, max_val=50, default=9) # Corrected min_val
        calculate_macd = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_macd) # Referencing global do_calculate_macd
        factory.register(FunctionDefinition("MACD", {"fast_length": fast_length_param, "slow_length": slow_length_param, "signal_length": signal_length_param}, calculate_macd)) # Use factory.register


        # Bollinger Bands
        bb_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        std_dev_param = ParameterType("real", min_val=0.1, max_val=5.0, default=2.0, increment=0.1)
        calculate_bollinger_bands = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_bollinger_bands) # Referencing global do_calculate_bollinger_bands
        factory.register(FunctionDefinition("BB", {"length": bb_length_param, "std_dev": std_dev_param}, calculate_bollinger_bands)) # Use factory.register


        # Rolling VWAP
        rvwap_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        calculate_rvwap = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_rvwap) # Referencing global do_calculate_rvwap
        factory.register(FunctionDefinition("RVWAP", {"length": rvwap_length_param}, calculate_rvwap)) # Use factory.register


        # Average True Range
        atr_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_atr = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_atr) # Referencing global do_calculate_atr
        factory.register(FunctionDefinition("ATR", {"length": atr_length_param}, calculate_atr)) # Use factory.register


        # ADX
        adx_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_adx = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_adx) # Referencing global do_calculate_adx
        factory.register(FunctionDefinition("ADX", {"length": adx_length_param}, calculate_adx)) # Use factory.register


        # Commodity Channel Index
        cci_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_cci = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_cci) # Referencing global do_calculate_cci
        factory.register(FunctionDefinition("CCI", {"length": cci_length_param}, calculate_cci)) # Use factory.register


        # Chaikin Money Flow
        cmf_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_cmf = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_cmf) # Referencing global do_calculate_cmf
        factory.register(FunctionDefinition("CMF", {"length": cmf_length_param}, calculate_cmf)) # Use factory.register


        # Aroon
        aroon_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_aroon = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_aroon) # Referencing global do_calculate_aroon
        factory.register(FunctionDefinition("Aroon", {"length": aroon_length_param}, calculate_aroon)) # Use factory.register


        # MFI
        mfi_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_mfi = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_mfi) # Referencing global do_calculate_mfi
        factory.register(FunctionDefinition("MFI", {"length": mfi_length_param}, calculate_mfi)) # Use factory.register


        # Percent Rank
        pct_rank_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_pct_rank = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_pct_rank) # Referencing global do_calculate_pct_rank
        factory.register(FunctionDefinition("PCT", {"length": pct_rank_length_param}, calculate_pct_rank)) # Use factory.register


        # Price Range Percentage
        prp_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_prp = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_prp) # Referencing global do_calculate_prp
        factory.register(FunctionDefinition("PRP", {"length": prp_length_param}, calculate_prp)) # Use factory.register


        # Log Return
        lret_length_param = ParameterType("integer", min_val=1, max_val=200, default=1)
        calculate_lret = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_lret) # Referencing global do_calculate_lret
        factory.register(FunctionDefinition("LRET", {"length": lret_length_param}, calculate_lret)) # Use factory.register


        # Shift
        shift_n_param = ParameterType("integer", min_val=1, max_val=200, default=1)
        shift_series_param = ParameterType("any", default="close")
        calculate_shift = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_shift) # Referencing global do_calculate_shift
        factory.register(FunctionDefinition("Shift", {"series": shift_series_param, "n": shift_n_param}, calculate_shift)) # Use factory.register    

    
    def get(self, name: str):
        """
        Retrieves a function definition by name.

        Args:
            name: The name of the screener.

        Returns:
            The FunctionDefinition object.

        Raises:
            ValueError: If no screener with the given name is registered.
        """
        if name not in self.function_definitions:
            raise ValueError(f"No function found with the name '{name}'.")
        return self.function_definitions[name]

    def parse(self, expression):
        parse_tree = self.grammar.parse(expression)
        reified_expression = parse_tree.reify(self)
        return reified_expression

    def __repr__(self):
        return f"FunctionFactory(functions={self.function_definitions})"



In [10]:
# Functions to carry out basic mathematical operations

#expression -> term "+" term
def calculate_add(df, a0, a1):
    return a0 + a1

#expression -> term "-" term
def calculate_sub(df, a0, a1):
    return a0 - a1

#expression -> term "*" term
def calculate_mul(df, a0, a1):
    return a0 * a1

#expression -> term "/" term
def calculate_div(df, a0, a1):
    return a0 / a1

#expression -> term "%" term
def calculate_mod(df, a0, a1):
    return a0 % a1

#expression -> term "**" term
def calculate_pow(df, a0, a1):
    return a0 ** a1

#expression -> term "<" term
def calculate_lt(df, a0, a1):
    return a0 < a1

#expression -> term "<=" term
def calculate_le(df, a0, a1):
    return a0 <= a1

#expression -> term ">" term
def calculate_gt(df, a0, a1):
    return a0 > a1

#expression -> term ">=" term
def calculate_ge(df, a0, a1):
    return a0 >= a1

#expression -> term "==" term
def calculate_eq(df, a0, a1):
    return a0 == a1

#expression -> term "!=" term
def calculate_ne(df, a0, a1):
    return a0 != a1

#expression -> term "&&" term
#expression -> term "||" term
#expression -> term "^^" term

In [11]:
# Functions for basic portfolio calculations

def do_normalize(context, weights):
    """Normalizes a Series of weights (including negative) to sum to 1 (absolute values)."""
    if isinstance(weights, pd.DataFrame):
        weights = weights.iloc[:, 0]
    elif not isinstance(weights, pd.Series):
        weights = pd.Series(weights)

    wt = weights.astype("float")
    absolute_weights = wt.abs()
    signs = wt.div(absolute_weights).fillna(0)
    total_absolute_weight = absolute_weights.sum()

    if total_absolute_weight == 0:
        return pd.DataFrame({"weight": [0.0] * len(weights)}, index=weights.index)

    normalized_absolute_weights = absolute_weights / total_absolute_weight
    normalized_weights = normalized_absolute_weights * signs

    return pd.DataFrame({"weight": normalized_weights}, index=weights.index)


def do_abs_threshold(context, weights, threshold):
    if isinstance(weights, pd.DataFrame):
        weights = weights.iloc[:, 0]
    elif not isinstance(weights, pd.Series):
        weights = pd.Series(weights)

    wt = weights.astype("float")
    absolute_weights = wt.abs()

    # Apply threshold
    wt[absolute_weights < threshold] = 0

    return pd.DataFrame({"weight": wt}, index=weights.index)

def do_abs_min_cutoff(context, weights, minimum):
    """Applies a minimum weight to a list of weights."""
    if isinstance(weights, pd.DataFrame):
        weights = weights.iloc[:, 0]
    elif not isinstance(weights, pd.Series):
        weights = pd.Series(weights)

    wt = weights.astype("float")
    absolute_weights = wt.abs()

    # Apply minimum
    mask = (absolute_weights > 0) & (absolute_weights < minimum)
    wt[mask] = np.sign(wt[mask]) * minimum

    return pd.DataFrame({"weight": wt}, index=weights.index)

def do_abs_max_cutoff(context, weights, maximum):
    if isinstance(weights, pd.DataFrame):
        weights = weights.iloc[:, 0]
    elif not isinstance(weights, pd.Series):
        weights = pd.Series(weights)

    wt = weights.astype("float")
    absolute_weights = wt.abs()

    # Apply maximum
    mask = absolute_weights > maximum
    wt[mask] = np.sign(wt[mask]) * maximum

    return pd.DataFrame({"weight": wt}, index=weights.index)

In [12]:
# Functions to carry out screener operations
def top_n_screener_function(context, field, top_n):
    """
    Produces a boolean mask (pd.Series) for the top 5 readings per day.

    Args:
        df: Pandas DataFrame with columns for date and reading.
        date_col: Name of the column containing the date. Should be datetime or convertible.
        reading_col: Name of the column containing the reading.

    Returns:
        A pandas Series (boolean mask) with True for rows corresponding to the 
        top 5 readings for each day, and False otherwise. Returns
        an empty Series if the input DataFrame is empty.
    """
    print(field)
    if(isinstance(field, str)):
        foo = context.groupby("date")[field].rank(ascending=False, method='first')
    else:
        foo = field.groupby(by=context["date"]).rank(ascending=False, method="first")
    print(foo)
    mask = foo <= top_n 
    return mask.iloc[:, 0]


def percentile_screener_function(context, field, percentile):
    
    if(isinstance(field, str)):
        foo = context.groupby(by="date")[field].rank(ascending=False, method='first', pct=True)
    else:
        foo = field.groupby(by=context["date"]).rank(ascending=False, method="first", pct=True)
    mask = foo >= percentile
    
    return mask.iloc[:, 0]


In [13]:
# Utility function for applying generic indicator functions to dataframes with multiple symbols
def calculate_indicator_by(df, field, indicator_function, *args, **kwargs):
    """
    Calculates an indicator by a specified field within a Pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        field (str): The field to group by (e.g., 'symbol', 'date').
        indicator_function (callable): The indicator function to apply.
        *args: Positional arguments to pass to the indicator function.
        **kwargs: Keyword arguments to pass to the indicator function.

    Returns:
        pd.DataFrame: The DataFrame with the calculated indicator(s) added.
    """
    if field in df.columns:
        result_dfs = []
        for group_value, group_df in df.groupby(field):
            result_dfs.append(indicator_function(group_df.copy(), *args, **kwargs).assign(**{field: group_value}))
        rv = pd.concat(result_dfs)
        return rv.drop(field, axis=1)
    else:
        return indicator_function(df.copy(), *args, **kwargs)


# Note: I am not completely happy with how this works; ideally, this would be merged with calculate_indicator_by
def apply_daily_function(context, weights, f, *args, **kwargs):
    if(isinstance(weights, str)):
        foo = context.groupby("date")[weights].apply(lambda a: f(context, a, *args, **kwargs))
    else:
        foo = weights.groupby(by=context["date"]).apply(lambda a: f(context, a, *args, **kwargs))
    # Swap levels and drop the 'date' level
    foo.index = foo.index.droplevel("date")    
    return foo.iloc[:, 0]

In [14]:
# Functions to calculate basic technical indicators

# SMA
def do_calculate_sma(df: pd.DataFrame, length: int) -> pd.DataFrame:
    print(f"calculate_sma(df, {length})")
    sma_values = df['close'].rolling(window=length).mean().values
    return pd.DataFrame({f"SMA({length})": sma_values}, index=df.index)

# RSI
def do_calculate_rsi(df: pd.DataFrame, length: int) -> pd.DataFrame:
    length = int(length)
    delta = df['close'].diff()
    gains = delta.clip(lower=0)
    losses = -delta.clip(upper=0)
    avg_gains = gains.rolling(window=length).mean()
    avg_losses = losses.rolling(window=length).mean()
    rs = avg_gains / avg_losses.replace(0, float('inf'))
    rsi = 100 - (100 / (1 + rs))
    rsi_values = rsi.values
    return pd.DataFrame({f"RSI({length})": rsi_values}, index=df.index)

# MACD
def do_calculate_macd(df: pd.DataFrame, fast_length: int, slow_length: int, signal_length: int) -> pd.DataFrame:
    ema_fast = df['close'].ewm(span=fast_length, adjust=False).mean()
    ema_slow = df['close'].ewm(span=slow_length, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_length, adjust=False).mean()
    histogram = macd - signal
    return pd.DataFrame({f'MACD({fast_length},{slow_length},{signal_length})["macd"]': macd.values, f'MACD({fast_length},{slow_length},{signal_length})["signal"]': signal.values, f'MACD({fast_length},{slow_length},{signal_length})["histogram"]': histogram.values}, index=df.index)

# Bollinger Bands
def do_calculate_bollinger_bands(df: pd.DataFrame, length: int, std_dev: float) -> pd.DataFrame:
    rolling_mean = df['close'].rolling(window=length).mean()
    rolling_std = df['close'].rolling(window=length).std()
    upper_band = rolling_mean + (rolling_std * std_dev)
    lower_band = rolling_mean - (rolling_std * std_dev)
    middle_values = rolling_mean.values
    upper_values = upper_band.values
    lower_values = lower_band.values
    bb_df = pd.DataFrame({f'BB({length},{std_dev})["middle"]': middle_values, f'BB({length},{std_dev})["upper"]': upper_values, f'BB({length},{std_dev})["lower"]': lower_values}, index=df.index)
    return bb_df

# Rolling VWAP
def do_calculate_rvwap(df: pd.DataFrame, length: int) -> pd.DataFrame:
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    rolling_volume = df['volume'].rolling(length).sum()
    typical_price_x_volume = df["volume"] * typical_price
    rolling_typical_price_x_volume = typical_price_x_volume.rolling(length).sum()
    vwap = rolling_typical_price_x_volume / rolling_volume
    return pd.DataFrame({f"RVWAP({length})": vwap.values}, index=df.index)

# Average True Range
def do_calculate_atr(df, length):
    """Calculates Average True Range (ATR)."""
    tr1 = df["high"] - df["low"]
    tr2 = abs(df["high"] - df["close"].shift(1))
    tr3 = abs(df["low"] - df["close"].shift(1))
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(window=length).mean()
    return pd.DataFrame({f"ATR({length})": atr}, index=df.index)

# ADX
def do_calculate_adx(df, length):
    """Calculates Average Directional Index (ADX) and Directional Movement Indicators."""
    high = df["high"]
    low = df["low"]
    close = df["close"]

    upmove = high - high.shift(1)
    downmove = low.shift(1) - low
    plus_dm = pd.Series(np.where((upmove > downmove) & (upmove > 0), upmove, 0))
    minus_dm = pd.Series(np.where((downmove > upmove) & (downmove > 0), downmove, 0))

    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(df["low"] - df["close"].shift(1))
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # Calculate +DI and -DI
    plus_di = 100 * (plus_dm.ewm(alpha=1 / length).mean() / true_range.ewm(alpha=1 / length).mean())
    minus_di = 100 * (minus_dm.ewm(alpha=1 / length).mean() / true_range.ewm(alpha=1 / length).mean())

    # Calculate DX
    dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di).replace(0, np.inf)

    # Calculate ADX
    adx = dx.ewm(alpha=1 / length).mean().fillna(0)

    return pd.DataFrame({f"ADX({length}):adx": adx, f"ADX({length}):pdi": plus_di, f"ADX({length})mdi": minus_di}, index=df.index)

# Commodity Channel Index
def do_calculate_cci(df, length):
    """Calculates Commodity Channel Index (CCI)."""
    typical_price = (df["high"] + df["low"] + df["close"]) / 3
    ma_typical_price = typical_price.rolling(window=length).mean()
    mean_deviation = pd.Series(abs(typical_price - ma_typical_price)).rolling(window=length).mean()
    cci = (typical_price - ma_typical_price) / (0.015 * mean_deviation)
    return pd.DataFrame({f"CCI({length})": cci})

# Chaikin Money Flow
def do_calculate_cmf(df, length):
    """Calculates Chaikin Money Flow (CMF)."""
    money_flow = ((df["close"] - df["low"]) - (df["high"] - df["close"])) / (df["high"] - df["low"]) * df["volume"]
    money_flow_volume = money_flow.rolling(window=length).sum()
    volume_sum = df["volume"].rolling(window=length).sum()
    cmf = money_flow_volume / volume_sum
    return pd.DataFrame({f"CMF({length})": cmf}, index=df.index)

# Aroon
def do_calculate_aroon(df, length):
    """Calculates Aroon Up and Aroon Down (incremental optimization)."""
    high = df["high"].values
    low = df["low"].values
    aroon_up = np.zeros(len(df))
    aroon_down = np.zeros(len(df))

    if len(df) < length:
        return pd.DataFrame({f"AROON({length}):up": aroon_up, f"AROON({length}):down": aroon_down}, index=df.index)

    highest_index = 0
    lowest_index = 0

    for i in range(length, len(df)):
        window_start = i - length

        # Update highest index
        if highest_index < window_start:  # If the previous highest is outside the window
            highest_index = window_start
            for j in range(window_start + 1, i):
                if high[j] > high[highest_index]:
                    highest_index = j
        elif high[i - 1] >= high[highest_index]:
            highest_index = i - 1

        # Update lowest index
        if lowest_index < window_start:  # If the previous lowest is outside the window
            lowest_index = window_start
            for j in range(window_start + 1, i):
                if low[j] < low[lowest_index]:
                    lowest_index = j
        elif low[i - 1] <= low[lowest_index]:
            lowest_index = i - 1

        aroon_up[i] = (length - (i - 1 - highest_index)) * 100.0 / length
        aroon_down[i] = (length - (i - 1 - lowest_index)) * 100.0 / length

    return pd.DataFrame({f"AROON({length}):up": aroon_up, f"AROON({length}):down": aroon_down}, index=df.index)

# MFI
def do_calculate_mfi(df, length):
    """Calculates Money Flow Index (MFI)."""
    typical_price = (df["high"] + df["low"] + df["close"]) / 3
    money_flow = typical_price * df["volume"]

    positive_money_flow = money_flow[df["close"] > df["close"].shift(1)]
    negative_money_flow = money_flow[df["close"] <= df["close"].shift(1)]

    positive_money_flow = positive_money_flow.rolling(window=length).sum()
    negative_money_flow = abs(negative_money_flow.rolling(window=length).sum())

    money_ratio = positive_money_flow / negative_money_flow
    mfi = 100 - (100 / (1 + money_ratio))
    return pd.DataFrame({f"MFI({length})": mfi}, index=df.index)

# Percent Rank
def do_calculate_pct_rank(df, length):
    """Calculates percentile rank using pandas only."""
    pct_rank = df['close'].rolling(window=length).apply(lambda x: (x < x[-1]).sum() / (len(x)-1) if len(x) > 1 else 0, raw=True)
    return pd.DataFrame({f"PCT({length})": pct_rank}, index=df.index)

# Price Range Percentage
def do_calculate_prp(df, length):
    """Calculates the Price Range Percentage."""
    high_max = df["high"].rolling(window=length).max()
    low_min = df["low"].rolling(window=length).min()
    range_width = high_max - low_min
    price_percentage = (df["close"] - low_min) / range_width * 100
    return pd.DataFrame({f"PRP({length})": price_percentage}, index=df.index)

# Log Return
def do_calculate_lret(df, length):
    """Calculates Log Return"""
    v = (df["close"] / df["close"].shift(length)).apply(lambda a: np.log(a) if a != np.nan else np.nan)
    return pd.DataFrame({f"LRET({length})": v}, index=df.index)

# Shift
def do_calculate_shift(df, series="close", n=1):
    name = "!unknown"
    if isinstance(series, str):
        name = series
        series = df[series]
    elif isinstance(series, pd.DataFrame):
        if(series.shape[1] != 1):
            raise ValueError(f"Shift() currently will operate on a single column only")
        name = series.columns[0]
        series = series.iloc[:, 0]
    elif isinstance(series, pd.Series):
        name = series.name
    return pd.DataFrame({f"Shift({name},{n})":series.shift(n)}, index=series.index)


In [15]:
# Ref: https://thepatternsite.com/8NewPriceLines.html
# This function detects the 'N new price lines (rising)' pattern in a given DataFrame.
# The pattern consists of N consecutive candle lines, each with a higher high than the previous one.
# The function returns a pandas Series of booleans indicating the presence of the pattern.
# True values indicate the start of an 'N new price lines' pattern.

def do_detect_n_new_price_lines_rising(df: pandas.core.frame.DataFrame, n: int = 8) -> pandas.Series:
    """
    Detects the 'N new price lines (rising)' pattern.

    Args:
        df: DataFrame with OHLC data and a 'date' column.
        n: The number of consecutive candles required to form the pattern.

    Returns:
        pandas.Series: Boolean Series indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the daily highs.
    highs = df["high"]

    # Check for N consecutive candles with higher highs.
    pattern_detected = highs.rolling(window=n).apply(lambda x: all(x[i] < x[i+1] for i in range(len(x)-1)), raw=True)

    # Shift the result to align with the start of the pattern.
    pattern_detected = pattern_detected.shift(-(n - 1))

    # Fill NaN values with False.
    pattern_detected = pattern_detected.fillna(False)

    # Convert to boolean.
    pattern_detected = pattern_detected.astype(bool)

    return pattern_detected


# Ref: https://thepatternsite.com/8NewPriceLines.html
# This function detects the 'N new price lines (falling)' pattern in a DataFrame.
# The pattern consists of N consecutive candle lines, each with a lower low than the previous one.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_n_new_price_lines_falling(df: pd.DataFrame, n: int = 8) -> pd.Series:
    """
    Detects the 'N new price lines (falling)' candlestick pattern.

    Args:
        df: DataFrame with 'low' column.
        n: The number of consecutive lines required to form the pattern.

    Returns:
        Boolean Series indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    lows = df["low"]
    is_falling = lows.rolling(window=n).apply(lambda x: all(x[i] < x[i+1] for i in range(n-1)), raw=True)
    return is_falling.astype(bool)


# Ref: https://thepatternsite.com/AbandonBaby.html
# This function detects the "Abandoned Baby, Bearish" candlestick pattern.
# The pattern consists of three candles:
# 1. A white candle in an upward trend.
# 2. A doji candle whose lower shadow is above the previous candle's high.
# 3. A black candle whose upper shadow is below the doji's low.

def do_detect_bearish_abandoned_baby(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Abandoned Baby candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        Boolean Series indicating Bearish Abandoned Baby patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body and shadows
    df['body'] = df['close'] - df['open']
    df['upper_shadow'] = df['high'] - df.apply(lambda row: max(row['open'], row['close']), axis=1)
    df['lower_shadow'] = df.apply(lambda row: min(row['open'], row['close']), axis=1) - df['low']

    # Check for the pattern
    pattern = (
        (df['body'].shift(2) > 0) &  # White candle two days prior
        (abs(df['body'].shift(1)) < 0.01 * (df['high'].shift(1) - df['low'].shift(1))) &  # Doji with small body size on previous day
        (df['lower_shadow'].shift(1) > df['high'].shift(2)) &  # Doji's lower shadow above previous white candle's high
        (df['body'] < 0) &  # Black candle today
        (df['upper_shadow'] < df['low'].shift(1))   # Upper shadow below doji's low
    )

    return pattern


# Ref: https://thepatternsite.com/AbandonBabyBull.html
# This function detects the Bullish Abandoned Baby candlestick pattern.
# The pattern consists of three candles:
# 1. A black candle.
# 2. A doji candle that gaps below the shadows of the adjacent candles.
# 3. A white candle whose lower shadow remains above the top of the doji.
# The function returns a pandas Series of booleans, where True indicates the presence of the pattern.

def do_detect_bullish_abandoned_baby(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Abandoned Baby candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns "open", "high", "low", "close", "volume", and "date".

    Returns:
        pandas.Series: Boolean Series indicating Bullish Abandoned Baby patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadows
    df['body'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.loc[:, ['open', 'close']].max(axis=1)
    df['lower_shadow'] = df.loc[:, ['open', 'close']].min(axis=1) - df['low']

    # Identify the pattern
    is_bullish_abandoned_baby = (
        (df['close'].shift(2) < df['open'].shift(2)) &  # First candle is black
        (abs(df['close'].shift(1) - df['open'].shift(1)) < 0.0001) & #Second candle is a Doji
        (df['open'].shift(1) < df['close'].shift(2)) & (df['open'].shift(1) > df['close'].shift(0)) & # Gap condition
        (df['close'] > df['open']) &  # Third candle is white
        (df['low'] > df['close'].shift(1))
    )

    return is_bullish_abandoned_baby


# Ref: https://thepatternsite.com/AboveStomach.html
# The "Above the Stomach" pattern consists of two candles in a downward trend.
# The first candle is black (close < open), and the second is white (close > open).
# The body of the second candle must be at or above the midpoint of the first candle's body.


def do_detect_above_the_stomach(df: pandas.DataFrame) -> pandas.Series:
    """
    Detects the "Above the Stomach" candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the midpoint of the first candle's body.
    midpoint = (df["open"].shift(1) + df["close"].shift(1)) / 2

    # Identify the pattern.
    mask = (df["close"].shift(1) < df["open"].shift(1)) & \
           (df["close"] > df["open"]) & \
           (df["open"] >= midpoint) & \
           (df["close"] >= midpoint)

    return mask


# Ref: https://thepatternsite.com/AdvanceBlock.html
# The Advance Block pattern consists of three white candles in an upward trend.
# Each candle opens within the body of the previous candle, and the shadows grow taller on the last two candles.
# This function detects the Advance Block pattern in a given DataFrame.

def do_detect_advance_block(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Advance Block candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the Advance Block pattern.  Returns an empty Series if df is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    #Calculate body size and shadow heights for each candle
    df['body'] = df['close'] - df['open']
    df['upper_shadow'] = df['high'] - df['close']
    df['lower_shadow'] = df['open'] - df['low']


    #Check for three consecutive white candles
    is_white = df['body'] > 0
    three_white_candles = is_white & is_white.shift(1) & is_white.shift(2)

    #Check open condition
    opens_in_previous_body = (df['open'] >= df['low'].shift(1)) & (df['open'] <= df['high'].shift(1))
    opens_in_previous_body = opens_in_previous_body & opens_in_previous_body.shift(1)

    #Check shadow condition
    increasing_upper_shadows = df['upper_shadow'] >= df['upper_shadow'].shift(1)
    increasing_upper_shadows = increasing_upper_shadows & increasing_upper_shadows.shift(1)

    #Combine conditions
    advance_block = three_white_candles & opens_in_previous_body & increasing_upper_shadows

    return advance_block

# Ref: https://thepatternsite.com/AbandonBaby.html
# This function detects the Bearish Abandoned Baby candlestick pattern.
# The pattern consists of three candles:
# 1. A white candle in an upward price trend.
# 2. A doji candle whose lower shadow is above the previous candle's high.
# 3. A black candle whose upper shadow is below the doji's low.

def do_detect_bearish_abandoned_baby(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Abandoned Baby candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series with True where the pattern is detected, False otherwise.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadows
    df['body'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df[['high', 'close']].max(axis=1) - df[['high', 'close']].min(axis=1)
    df['lower_shadow'] = df[['low', 'open']].min(axis=1) - df[['low', 'open']].max(axis=1)

    # Identify pattern components
    is_white = df['close'] > df['open']
    is_doji = df['body'] < 0.001 * df['high'] #Consider this to be a doji 
    is_black = df['close'] < df['open']

    #Check conditions for Bearish Abandoned Baby
    pattern_mask = (
      is_white.shift(2) & 
      is_doji.shift(1) &
      is_black &
      (df['low'].shift(1) > df['high'].shift(2)) &
      (df['high'] < df['low'].shift(1))
    )

    return pattern_mask


# Ref: https://thepatternsite.com/BeltHoldBear.html
# The Bearish Belt Hold is a candlestick pattern that appears in an uptrend.
# It's characterized by a long black candle where the open is near the high and the close is near the low.
# This function detects the pattern based on the provided criteria.

def do_detect_bearish_belt_hold(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Belt Hold candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size and total range.
    body_size = df['open'] - df['close']
    total_range = df['high'] - df['low']

    #Conditions for Bearish Belt Hold:
    # 1. Body size is positive (black candle).
    # 2. Open is near the high (within some tolerance). Let's use 0.1% for the tolerance.
    # 3. Close is near the low (within some tolerance). Let's use 0.1% for the tolerance.
    # 4. The open must be higher than the close.
    
    is_bearish_belt_hold = (body_size > 0) & (df['open'] >= df['high'] * 0.999) & (df['close'] <= df['low'] * 1.001)

    return is_bearish_belt_hold


# Ref: https://thepatternsite.com/BearBreakaway.html
# The following function implements a simplified detection algorithm for the Bearish Breakaway candlestick pattern.
# The algorithm identifies the pattern based on the relative heights and positions of five consecutive candles.
# It does not incorporate all the nuances described in the original documentation.  Further refinements could include
# considering the body sizes more precisely, adding checks for shadow overlaps, and potentially using more sophisticated
# gap detection criteria.


def do_detect_bearish_breakaway(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Breakaway candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Breakaway patterns (True for detected patterns, False otherwise).
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    def is_bearish_breakaway(candle_group):
        # Check for 5 candles
        if len(candle_group) != 5:
            return False

        # Extract candle data
        open_prices = candle_group["open"].values
        high_prices = candle_group["high"].values
        low_prices = candle_group["low"].values
        close_prices = candle_group["close"].values

        # Check conditions:
        # 1. Tall white candle
        if not (high_prices[0] > open_prices[0] and close_prices[0] > open_prices[0] and close_prices[0] > high_prices[0] * 0.5):
            return False
        # 2. Another white candle with gap
        if not (close_prices[1] > open_prices[1] and open_prices[1] > close_prices[0]):
            return False
        # 3. Higher close
        if not (close_prices[2] > close_prices[1]):
            return False
        # 4. Another white candle with higher close
        if not (close_prices[3] > close_prices[2]):
            return False
        # 5. Tall black candle closing within gap
        if not (close_prices[4] < open_prices[4] and close_prices[4] < open_prices[1] and close_prices[4] > low_prices[4] * 0.5):
            return False


        return True

    # Apply function to rolling window of 5 candles
    result = df.rolling(window=5, min_periods=5).apply(is_bearish_breakaway, raw=True)
    # shift the results back by four days
    return result.shift(-4).fillna(False).astype(bool)


# Ref: https://thepatternsite.com/DojiStarBear.html
# The Bearish Doji Star is a two-candle pattern occurring in an uptrend.
# The first candle is a long white candle.  The second candle gaps higher and forms a doji,
# meaning its open and close prices are very near each other. The shadows on the doji are relatively short.
# This pattern is considered a continuation pattern, not a reversal, as price tends to continue rising after its appearance.
# The function below implements this detection logic.


def do_detect_bearish_doji_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Doji Star candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size of the current candle
    df['body_size'] = abs(df['close'] - df['open'])

    # Calculate the total range of the candle
    df['candle_range'] = df['high'] - df['low']

    # Calculate the ratio of the body to the total range
    df['body_range_ratio'] = df['body_size'] / df['candle_range']

    # Calculate the gap between the current and previous candles
    df['gap'] = df['open'] - df['close'].shift(1)

    # Identify Doji candles
    df['is_doji'] = (df['body_range_ratio'] < 0.1)  # Adjust threshold as needed

    # Identify Bullish Doji Star Pattern
    result = (df['gap'] > 0) & (df['is_doji']) & (df['close'].shift(1) > df['open'].shift(1)) & (df['candle_range'].shift(1) > 0)


    return result

# Ref: https://thepatternsite.com/BearEngulfing.html
# This function detects the Bearish Engulfing candlestick pattern.
# It requires a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The function returns a boolean Series indicating the presence of the pattern for each row.


def do_detect_bearish_engulfing(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Engulfing candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Engulfing patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size of each candle.
    df['body_size'] = abs(df['close'] - df['open'])

    # Check for the engulfing condition.
    mask = (df['close'].shift(1) > df['open'].shift(1)) & \
           (df['open'] > df['close'].shift(1)) & \
           (df['close'] < df['open'].shift(1)) & \
           (df['body_size'] < df['body_size'].shift(-1))

    return mask

# Ref: https://thepatternsite.com/HaramiBear.html
# This function detects the Bearish Harami candlestick pattern.
# It takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The function returns a pandas Series of booleans, where True indicates the presence of a Bearish Harami pattern.


def do_detect_bearish_harami(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Bearish Harami candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Harami patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors.
    df['body_size'] = abs(df['close'] - df['open'])
    df['body_color'] = (df['close'] > df['open']).astype(int)

    # Shift data to compare with the preceding candle.
    shifted_body_size = df['body_size'].shift(1)
    shifted_body_color = df['body_color'].shift(1)

    # Define conditions for Bearish Harami.
    condition1 = (shifted_body_color == 1)  # Previous candle is white.
    condition2 = (df['body_color'] == 0)  # Current candle is black.
    condition3 = (df['open'] >= df['open'].shift(1)) & (df['close'] <= df['close'].shift(1)) & (df['open'] <= df['close'].shift(1)) & (df['close'] >= df['open'].shift(1))  # Current candle body is inside previous candle body.

    # Combine conditions and create the result series.
    bearish_harami = condition1 & condition2 & condition3

    return bearish_harami


# Ref: https://thepatternsite.com/HaramiCrossBear.html
# This function detects the Bearish Harami Cross candlestick pattern.
# A Bearish Harami Cross consists of two candles:
# 1. A long white (bullish) candle.
# 2. A subsequent doji candle whose high and low fall within the range of the first candle.

def do_detect_bearish_harami_cross(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Harami Cross candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and ranges.
    df['body_size'] = abs(df['close'] - df['open'])
    df['candle_range'] = df['high'] - df['low']

    # Identify doji candles (approximately equal open and close).
    df['is_doji'] = abs(df['close'] - df['open']) < 0.001 * df['candle_range']


    #Check for Bearish Harami Cross pattern
    pattern = (df['close'].shift(1) > df['open'].shift(1)) & \
              (df['is_doji']) & \
              (df['high'] <= df['high'].shift(1)) & \
              (df['low'] >= df['low'].shift(1))

    return pattern

# Ref: https://thepatternsite.com/KickingBear.html
# The bearish kicking candlestick pattern consists of two marubozu candles with a gap separating them.
# The first candle is a white marubozu (a white candle with no shadows), and the second candle is a black marubozu (a black candle with no shadows).
# The gap between the two candles is a crucial element of the pattern.
# The pattern is considered a bearish reversal signal.


def do_detect_bearish_kicking(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Kicking candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Kicking patterns (True if pattern detected).
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadows.  This is done efficiently for performance.
    df['body_size'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df[['high', 'close']].max(axis=1) - df[['high', 'close']].min(axis=1)
    df['lower_shadow'] = df[['low', 'open']].max(axis=1) - df[['low', 'open']].min(axis=1)

    # Identify white and black marubozu candles
    df['is_white_marubozu'] = (df['close'] > df['open']) & (df['upper_shadow'] == 0) & (df['lower_shadow'] == 0)
    df['is_black_marubozu'] = (df['close'] < df['open']) & (df['upper_shadow'] == 0) & (df['lower_shadow'] == 0)
    
    # Detect the pattern using a rolling window of size 2.  .shift() efficiently handles this for speed.
    is_bearish_kicking = (df['is_white_marubozu'].shift(1) & df['is_black_marubozu'] & (df['open'] - df['close'].shift(1) > 0))

    return is_bearish_kicking


# Ref: https://thepatternsite.com/MeetingLinesBear.html
# The Bearish Meeting Lines pattern consists of two tall candles: a white candle followed by a black candle.
# The closing prices of the two candles are near each other.
# This function detects this pattern.  The pattern's predictive power is not considered here.

def do_detect_bearish_meeting_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Meeting Lines candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body size
    df['body'] = abs(df['close'] - df['open'])

    # Identify tall candles (needs parameterization)
    tall_candle_threshold = 0.0  #FIXME: This needs to be determined empirically.
    df['is_tall'] = df['body'] > tall_candle_threshold

    # Check for two consecutive tall candles of opposite colors.
    df['bearish_meeting_lines'] = (
        (df['close'] > df['open']) & df['is_tall'] &
        (df['close'].shift(-1) < df['open'].shift(-1)) & df['is_tall'].shift(-1) &
        (abs(df['close'] - df['close'].shift(-1)) < 0.0) #FIXME: Needs a better definition of "near". 
                                                         #Perhaps a percentage of the average range of the two candles.
    )
    return df['bearish_meeting_lines']


# Ref: https://thepatternsite.com/SeparateLinesBear.html
# This function detects the Bearish Separating Lines candlestick pattern.
# It checks for a tall white candle followed by a tall black candle with similar opening prices,
# within a downward price trend.

def do_detect_bearish_separating_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Separating Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Separating Lines patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_white'] = df['close'] > df['open']

    # Identify candidate patterns (two-candle sequence)
    mask = (df['is_white'] == True) & (df['is_white'].shift(-1) == False)
    candidates = df[mask]

    # Check for similar opening prices and tall candles (relative to context)
    pattern_detected = (abs(candidates['open'] - candidates['open'].shift(-1)) / candidates['body_size'].shift(-1)) < 0.2  & (candidates['body_size'] > candidates['body_size'].rolling(window=20).mean()) & (candidates['body_size'].shift(-1) > candidates['body_size'].rolling(window=20).mean().shift(-1))

    # Extend the boolean mask to the original DataFrame using the index
    result = pd.Series(False, index=df.index)
    result[candidates.index] = pattern_detected

    return result


# Ref: https://thepatternsite.com/SidebySideWhiteLinesBear.html
# This function detects the 'Bearish Side by Side White Lines' candlestick pattern.
# The pattern consists of three candles: a black candle followed by two white candles
# with similar bodies and opening prices, and closing prices below the black candle's body.

def do_detect_bearish_side_by_side_white_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Side by Side White Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Side by Side White Lines patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_black'] = df['close'] < df['open']

    # Identify potential patterns
    pattern_mask = (df['is_black'].shift(2) &
                    ~df['is_black'].shift(1) &
                    ~df['is_black'])

    # Check conditions for the second and third candles.
    # The second and third candles should have approximately the same size and opening price.  This is a subjective condition.  Consider alternative ways to define this.
    pattern_mask &= (abs(df['body_size'].shift(1) - df['body_size']) / df['body_size'].shift(1) < 0.5)  #Example using relative difference of 50%
    pattern_mask &= (abs(df['open'].shift(1) - df['open']) / df['open'].shift(1) < 0.1) # Example using relative difference of 10%
    pattern_mask &= (df['close'].shift(1) < df['open'].shift(2)) & (df['close'] < df['open'].shift(2))

    return pattern_mask

# Ref: https://thepatternsite.com/ThreeLineStrikeBear.html
# The following function implements the detection of the Bearish Three-Line Strike candlestick pattern.
# It identifies the pattern based on the configuration of four consecutive candles: three black candles 
# with decreasing lows, followed by a tall white candle that opens below the previous close and closes 
# above the first day's open.

def do_detect_bearish_three_line_strike(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Three-Line Strike candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).

    Returns:
        pandas.Series: Boolean Series indicating Bearish Three Line Strike patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    #Efficient calculation using boolean indexing and rolling window comparisons.
    pattern = (
        (df['close'] < df['open']) &  # Black candles
        (df['low'].rolling(3).min() > df['low'].shift(1).rolling(3).min()) & # Decreasing lows
        (df['open'].shift(-1) < df['close'].shift(1)) & # Next candle opens below previous close
        (df['close'].shift(-1) > df['open'].shift(3)) # Next candle closes above first candle's open
    ).shift(1)

    #To align index, we shift the pattern back by 1.
    return pattern


# Ref: https://thepatternsite.com/TriStarBear.html
# This function detects the Bearish Tri-Star candlestick pattern.
# The pattern consists of three doji candles, with the middle one having a body above the other two.
# The function assumes the input DataFrame has columns 'open', 'high', 'low', 'close', 'volume', and 'date'.
# A doji is defined as a candle where the opening and closing prices are very close.  This function uses a simple threshold to check this condition.

def do_detect_bearish_tri_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Tri-Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Tri-Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size of each candle.
    df['body_size'] = abs(df['close'] - df['open'])

    # Define a threshold for doji candles (adjust as needed).
    doji_threshold = 0.01  # Example threshold: body size less than 1% of the candle's range.

    # Identify potential doji candles.
    df['is_doji'] = df['body_size'] / (df['high'] - df['low']) < doji_threshold

    # Check for the Bearish Tri-Star pattern.
    bearish_tri_star = (df['is_doji'].shift(2) & 
                        df['is_doji'].shift(1) & 
                        df['is_doji'] &
                        (df['open'].shift(1) > df['open'].shift(2)) &
                        (df['open'].shift(1) > df['open']) )

    return bearish_tri_star

# Ref: https://thepatternsite.com/BelowStomach.html
# The below the stomach candlestick is a two-candle pattern that appears after an uptrend.
# It consists of a tall white candle followed by a candle whose body is below the midpoint of the previous candle's body.
# The second candle may be black, but this is not a strict requirement.
# This pattern is considered a bearish reversal pattern, though its reliability is lower than that of the "Above the Stomach" pattern.

def do_detect_below_the_stomach(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Below the Stomach candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating where the pattern is detected.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the midpoint of the first candle's body
    midpoint = (df['high'].shift(1) + df['low'].shift(1)) / 2

    #Check if the second candle's body is below the midpoint
    below_midpoint = df['close'] < midpoint

    #Check if the previous candle was a tall white candle (Close > Open)
    tall_white_candle = df['close'].shift(1) > df['open'].shift(1)

    #Combine conditions to identify the pattern.
    pattern_detected = tall_white_candle & below_midpoint

    return pattern_detected


# Ref: https://thepatternsite.com/BeltHoldBear.html
# The bearish belt hold is characterized by a single black candlestick in an uptrend.
# The candlestick opens near its high and closes near its low.
# This function detects this pattern based on the provided criteria.


def do_detect_bearish_belt_hold(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Bearish Belt Hold candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the body size of each candlestick.
    body_size = df['close'] - df['open']

    # Identify black candlesticks (close < open).
    is_black = body_size < 0

    # Calculate the total candle range.
    candle_range = df['high'] - df['low']

    # Check if the open is near the high and the close is near the low.
    open_near_high = (df['high'] - df['open']) / candle_range < 0.1  # Adjust 0.1 as needed
    close_near_low = (df['close'] - df['low']) / candle_range < 0.1 # Adjust 0.1 as needed


    # Combine conditions to detect the pattern.
    is_bearish_belt_hold = is_black & open_near_high & close_near_low

    return is_bearish_belt_hold


# Ref: https://thepatternsite.com/BeltHoldBull.html
# This function detects the 'Bullish Belt Hold' candlestick pattern.
# A bullish belt hold is a single white candle with no lower shadow, closing near the high.
# It's considered a bullish reversal pattern.  The function identifies the pattern
# based on the characteristics described in the documentation.

def do_detect_bullish_belt_hold(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Bullish Belt Hold candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the body size and lower shadow for each candle.
    body_size = df['close'] - df['open']
    lower_shadow = df['open'] - df['low']

    # Identify Bullish Belt Hold candles: white candle (body_size > 0), 
    # no lower shadow (lower_shadow <= 0), and closing near the high.
    #  The condition 'close' being near the high is not explicitly defined in the source,
    # therefore, we use the heuristic that the close is within 1% of the high.

    is_bullish_belt_hold = (body_size > 0) & (lower_shadow <= 0) & ((df['high'] - df['close']) / df['high'] <= 0.01)

    return is_bullish_belt_hold


# Ref: https://thepatternsite.com/BlkCandle.html
# The following function detects the Black Candle pattern based on the description provided in the URL.
# A black candle is characterized by a black-colored candle of average size with shadows smaller than the body height.
# No specific price trend is required before the pattern.

def do_detect_black_candle(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Black Candle pattern in a DataFrame.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of a black candle pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate body height and shadow lengths
    df['body_height'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.apply(lambda row: max(row['open'], row['close']), axis=1)
    df['lower_shadow'] = df.apply(lambda row: min(row['open'], row['close']), axis=1) - df['low']

    # Define criteria for black candle (shadows smaller than body height)
    is_black_candle = (df['close'] < df['open']) & (df['upper_shadow'] < df['body_height']) & (df['lower_shadow'] < df['body_height'])

    return is_black_candle



# Ref: https://thepatternsite.com/LongBlack.html
# This function detects the "Long Black Day" candlestick pattern.
# A long black day is defined as a tall black candle with shadows shorter than the body.
# The body is considered "tall" if it is at least three times the average body height of recent candles.

def do_detect_long_black_day(df: pd.DataFrame, recent_candle_count: int = 10) -> pd.Series:
    """
    Detects the Long Black Day candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        recent_candle_count: Number of recent candles to consider for average body height calculation.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.  Returns an empty Series if the dataframe is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate average body height
    df['body'] = abs(df['close'] - df['open'])
    avg_body_height = df['body'].rolling(window=recent_candle_count, min_periods=1).mean()

    # Identify long black days
    is_long_black_day = (df['close'] < df['open']) & (df['body'] >= 3 * avg_body_height) & (df['high'] - df['close'] < df['body']) & (df['open'] - df['low'] < df['body'])

    return is_long_black_day

# Ref: https://thepatternsite.com/BlackMarubozu.html
# The black marubozu is a tall black candle with no shadows.  This function detects this pattern.
# The function operates on a Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The index of the input DataFrame is preserved in the output Series. The input DataFrame is not modified.

def do_detect_black_marubozu(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close', 'volume', 'date').

    Returns:
        pandas.Series: Boolean Series indicating Black Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black_marubozu = (df['close'] < df['open']) & (df['high'] == df['close']) & (df['low'] == df['open'])
    return is_black_marubozu


# Ref: https://thepatternsite.com/BlkCandleShort.html
# This function detects the "Short Black Candle" pattern based on the criteria
# described in the provided documentation.  The pattern is characterized by a
# single short black candle with relatively short upper and lower shadows.  The
# function identifies the pattern by comparing the candle body length to the
# lengths of its upper and lower shadows.


def do_detect_short_black_candle(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Short Black Candle pattern in OHLCV data.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body length
    body_length = df['close'] - df['open']

    # Calculate upper and lower shadow lengths
    upper_shadow = df['high'] - df['close']
    lower_shadow = df['open'] - df['low']

    # Check for short black candle conditions
    is_short_black = (body_length < 0) & (upper_shadow < abs(body_length)) & (lower_shadow < abs(body_length))

    return is_short_black


# Ref: https://thepatternsite.com/SpinTopBlack.html
# This function detects the "Black Spinning Top" candlestick pattern.
# A black spinning top is characterized by a small black body with shadows taller than the body.
# The function takes a Pandas DataFrame as input, and returns a Pandas Series of booleans.
# True indicates the presence of a black spinning top pattern, False otherwise.

def do_detect_black_spinning_top(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Black Spinning Top candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Black Spinning Top patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size
    body_size = abs(df["close"] - df["open"])

    # Calculate upper and lower shadow sizes
    upper_shadow = df["high"] - df.loc[df["close"] > df["open"], "close"].fillna(df["open"])
    upper_shadow = upper_shadow.fillna(0)
    lower_shadow = df.loc[df["close"] < df["open"], "open"].fillna(df["close"]) - df["low"]
    lower_shadow = lower_shadow.fillna(0)
    
    #Condition for black spinning top: small black body and shadows taller than the body
    is_black_spinning_top = (df["close"] < df["open"]) & (body_size < upper_shadow) & (body_size < lower_shadow)

    return is_black_spinning_top


# Ref: https://thepatternsite.com/BearBreakaway.html
# This function detects the Bearish Breakaway candlestick pattern.
# The pattern consists of five candles:
# 1. A tall white candle.
# 2. A white candle with a gap between its body and the body of the first candle.
# 3. A candle with a higher close than the second candle (color doesn't matter).
# 4. A white candle with a higher close than the third candle.
# 5. A tall black candle that closes within the gap between the bodies of the first two candles.

def do_detect_bearish_breakaway(df: pd.DataFrame, range_threshold: float = 0.03) -> pd.Series:
    """
    Detects Bearish Breakaway pattern.

    Args:
        df: DataFrame with OHLC data.
        range_threshold: Minimum percentage move beyond the previous range.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Breakaway patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and ranges
    df['body'] = abs(df['close'] - df['open'])
    df['range'] = df['high'] - df['low']

    # Function to check if a candle is tall (body size > threshold)
    is_tall = lambda x: x['body'] > range_threshold * x['range']

    # Function to check for a gap between candles
    has_gap = lambda x: (x['open'] > df['close'].shift(1)) | (x['close'] < df['open'].shift(1))
    
    # Apply functions to create boolean columns indicating pattern conditions
    df['is_tall_white'] = (df['close'] > df['open']) & df['is_tall'].apply(is_tall)
    df['is_white'] = (df['close'] > df['open'])
    df['has_gap'] = df['has_gap'].apply(has_gap)
    df['higher_close'] = df['close'] > df['close'].shift(1)
    df['is_tall_black'] = (df['close'] < df['open']) & df['is_tall'].apply(is_tall)


    # Detect the pattern using boolean indexing
    pattern = (df['is_tall_white'].shift(4) &
               df['is_white'].shift(3) &
               df['has_gap'].shift(3) &
               df['higher_close'].shift(2) &
               df['is_white'].shift(1) &
               df['higher_close'].shift(1) &
               df['is_tall_black'] &
               (df['close'] > df['open'].shift(3)) &  #Close within gap
               (df['close'] < df['close'].shift(3)) &  #Close within gap
               (df['open'] < df['close'].shift(1)) )

    return pattern
# Ref: https://thepatternsite.com/BullBreakaway.html
# This function detects the Bullish Breakaway candlestick pattern.
# The pattern consists of five candles:
# 1. A tall black candle.
# 2. Another black candle opening lower, leaving a gap between the two bodies (shadows can overlap).
# 3. A candle of any color with a lower close than the previous candle.
# 4. A black candle with a lower close than the previous candle.
# 5. A tall white candle closing within the body gap of the first two candles.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_bullish_breakaway(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Breakaway candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date column.

    Returns:
        Boolean Series indicating Bullish Breakaway patterns.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    #Calculate the body sizes of each candle
    df["body_size"] = abs(df["close"] - df["open"])

    #Helper function to check if a condition holds true for a given window of rows.
    def check_pattern(window):
        if len(window) < 5:
            return False
        c1, c2, c3, c4, c5 = window
        # Check conditions for candle 1
        is_c1_black = c1["close"] < c1["open"]
        is_c1_tall = c1["body_size"] > 0 #Arbitrary value to avoid division by zero.  This condition should be made more stringent.

        # Check conditions for candle 2
        is_c2_black = c2["close"] < c2["open"]
        is_c2_gap = c2["open"] < c1["open"] #Simplified gap condition, shadows can overlap.

        # Check conditions for candle 3
        is_c3_lower_close = c3["close"] < c2["close"]

        # Check conditions for candle 4
        is_c4_black = c4["close"] < c4["open"]
        is_c4_lower_close = c4["close"] < c3["close"]

        # Check conditions for candle 5
        is_c5_white = c5["close"] > c5["open"]
        is_c5_within_gap = c1["open"] > c5["close"] > c2["open"]

        return is_c1_black and is_c1_tall and is_c2_black and is_c2_gap and is_c3_lower_close and is_c4_black and is_c4_lower_close and is_c5_white and is_c5_within_gap

    bullish_breakaway = df.rolling(5).apply(check_pattern, raw=True)
    return bullish_breakaway

# Ref: https://thepatternsite.com/AbandonBabyBull.html
# This function detects the Bullish Abandoned Baby candlestick pattern.
# It requires a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The function identifies the pattern based on the following criteria:
# 1. Three candles in a downward trend.
# 2. The first candle is a black candle (open > close).
# 3. The second candle is a doji (open ≈ close) with a gap below the shadows of the adjacent candles.
# 4. The third candle is a white candle (open < close) whose lower shadow is above the doji's top.
# An upward breakout is confirmed when the price closes above the top of the pattern.


def do_detect_bullish_abandoned_baby(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Abandoned Baby candlestick pattern.

    Args:
        df: DataFrame with OHLCV data and a 'date' column.

    Returns:
        Boolean Series indicating Bullish Abandoned Baby patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadows
    df['body'] = df['close'] - df['open']
    df['upper_shadow'] = df['high'] - df.loc[ : , ['open', 'close']].max(axis=1)
    df['lower_shadow'] = df.loc[ : , ['open', 'close']].min(axis=1) - df['low']

    # Identify potential Bullish Abandoned Baby patterns
    is_bullish_abandoned_baby = (
        (df['body'].shift(2) < 0) &  # First candle is black
        (abs(df['body'].shift(1)) < 0.01 * (df['high'].shift(1) - df['low'].shift(1))) &  # Second candle is a doji
        (df['open'].shift(1) < df['close'].shift(2)) & (df['close'].shift(1) > df['open'].shift(2)) & # Gap check
        (df['body'] > 0) &  # Third candle is white
        (df['low'] > df['close'].shift(1)) # Lower shadow of the third candle above doji top
    )

    return is_bullish_abandoned_baby


# Ref: https://thepatternsite.com/BeltHoldBull.html
# The following function detects the Bullish Belt Hold candlestick pattern.
# A Bullish Belt Hold is characterized by a single white candle with no lower shadow and closing near the high.
# The function takes a Pandas DataFrame as input, containing 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# It returns a Pandas Series of boolean values, indicating the presence (True) or absence (False) of the pattern for each row in the DataFrame.

def do_detect_bullish_belt_hold(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Bullish Belt Hold candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Belt Hold patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    #Check for white candle
    is_white = df['close'] > df['open']

    #Check for no lower shadow
    no_lower_shadow = df['low'] == df['open']

    #Check for closing near high (simplified condition: close within 1% of high)
    close_near_high = (df['high'] - df['close']) / df['high'] <= 0.01

    # Combine conditions to detect Bullish Belt Hold
    bullish_belt_hold = is_white & no_lower_shadow & close_near_high

    return bullish_belt_hold


# Ref: https://thepatternsite.com/BullBreakaway.html
# This function detects the Bullish Breakaway candlestick pattern.
# The pattern consists of five candles:
# 1. A tall black candle.
# 2. Another black candle opening lower, leaving a gap between the two bodies.
# 3. A candle of any color with a lower close than the previous candle.
# 4. A black candle with a lower close than the previous candle.
# 5. A tall white candle closing within the body gap of the first two candles.
# The function returns a boolean Series indicating whether a Bullish Breakaway pattern is detected for each row in the input DataFrame.

def do_detect_bullish_breakaway(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Bullish Breakaway candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).

    Returns:
        A pandas Series with True where a Bullish Breakaway pattern is detected and False otherwise.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_white'] = df['close'] > df['open']

    # Function to check for Bullish Breakaway condition on a window of 5 candles
    def is_bullish_breakaway(window):
        if len(window) != 5:
            return False
        c1, c2, c3, c4, c5 = window
        
        #Check for first candle condition
        is_c1_black = not c1['is_white'] and c1['body_size'] > 0 # added body size condition
        
        #Check for second candle condition
        is_c2_black = not c2['is_white'] and c2['open'] < c1['open'] and c2['close'] < c1['close'] # added close condition
        
        #Check for gap condition
        has_gap = c2['open'] < c1['close']

        #Check for third and fourth candle conditions
        is_c3_lower = c3['close'] < c2['close']
        is_c4_lower = c4['close'] < c3['close'] and not c4['is_white']

        #Check for fifth candle condition
        is_c5_white = c5['is_white'] and c5['close'] > c2['open'] and c5['close'] < c1['open'] # added close condition within gap
        
        return is_c1_black and is_c2_black and has_gap and is_c3_lower and is_c4_lower and is_c5_white

    # Apply the function to a rolling window of 5 candles
    bullish_breakaway = df[['open', 'high', 'low', 'close', 'volume','is_white','body_size']].rolling(window=5).apply(lambda x: is_bullish_breakaway(x), raw=True)

    # Shift the result to align with the last candle of the 5-candle pattern
    bullish_breakaway = bullish_breakaway.shift(-4)
    bullish_breakaway = bullish_breakaway.fillna(False)

    return bullish_breakaway.iloc[:,0]


# Ref: https://thepatternsite.com/DojiStarBull.html
# The following function detects the Bullish Doji Star candlestick pattern.
# A Bullish Doji Star consists of two candles: a long black candle followed by a doji that gaps below the previous candle's body.
# The doji's shadows should not be unusually long.  The function uses a simplified definition and may require tuning for optimal results.


def do_detect_bullish_doji_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Doji Star patterns (True) or not (False).
        Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and gaps
    df['body_size'] = abs(df['close'] - df['open'])
    df['gap'] = df['open'].shift(-1) - df['close']

    # Identify doji candles (simplified definition)
    df['is_doji'] = (abs(df['close'] - df['open']) < 0.001 * df['high'])  # simplified; a tighter check might be needed.

    # Detect Bullish Doji Star pattern
    bullish_doji_star = (df['body_size'].shift(1) > df['body_size'].shift(1) * 0.1) & \
                        (df['gap'] < 0) & \
                        (df['close'].shift(1) > df['open'].shift(0)) & \
                        (df['is_doji'])


    return bullish_doji_star


# Ref: https://thepatternsite.com/BullEngulfing.html
# The following function implements the detection of the Bullish Engulfing candlestick pattern.
# It takes a Pandas DataFrame as input and returns a Pandas Series of booleans, indicating the presence
# of the pattern at each row.  The function considers only the body of the candles, ignoring the wicks.


def do_detect_bullish_engulfing(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects Bullish Engulfing candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date column.  Index must be preserved.

    Returns:
        Pandas Series of booleans indicating Bullish Engulfing pattern.
        Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the body of each candle
    df["body"] = df["close"] - df["open"]

    # Shift the data to compare with the previous candle.
    df["prev_body"] = df["body"].shift(1)
    df["prev_open"] = df["open"].shift(1)
    df["prev_close"] = df["close"].shift(1)


    # Condition for bullish engulfing:
    # 1. Previous candle must be black (close < open)
    # 2. Current candle must be white (close > open)
    # 3. Current candle's body must engulf or overlap the previous candle's body

    mask = (df["prev_close"] < df["prev_open"]) & \
           (df["close"] > df["open"]) & \
           (df["open"] <= df["prev_close"]) & \
           (df["close"] >= df["prev_open"])

    return mask



# Ref: https://thepatternsite.com/HaramiBull.html
# The bullish harami pattern consists of two candles.  The first is a long black candle,
# and the second is a white candle whose body is entirely contained within the body of the
# first candle.  This function implements this detection logic.

def do_detect_bullish_harami(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Bullish Harami candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of a Bullish Harami pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes
    df['body_size'] = abs(df['close'] - df['open'])

    # Identify potential Bullish Harami patterns
    bullish_harami = (df['close'].shift(1) < df['open'].shift(1)) &  # Previous candle is black
                      (df['close'] > df['open']) &  # Current candle is white
                      (df['open'] > df['open'].shift(1)) &  # Current open is higher than previous open
                      (df['close'] < df['close'].shift(1))   # Current close is lower than previous close


    #Check that current candle body is fully within previous candle body
    bullish_harami = bullish_harami & (df['open'] > df['open'].shift(1)) & (df['close'] < df['close'].shift(1))
    
    return bullish_harami

# Ref: https://thepatternsite.com/HaramiCrossBull.html
# This function detects the Bullish Harami Cross candlestick pattern.
# A bullish harami cross consists of two candles:
# 1. A long black candle.
# 2. A doji that is entirely contained within the high-low range of the first candle.

def do_detect_bullish_harami_cross(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Harami Cross candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.

    Returns:
        Boolean Series indicating Bullish Harami Cross patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and ranges.
    df['body_size'] = abs(df['close'] - df['open'])
    df['candle_range'] = df['high'] - df['low']

    # Identify long black candles (open > close).
    is_long_black = df['open'] > df['close'] & (df['body_size'] > 0)

    # Shift data for comparison with the next candle.
    shifted_high = df['high'].shift(-1)
    shifted_low = df['low'].shift(-1)
    shifted_open = df['open'].shift(-1)
    shifted_close = df['close'].shift(-1)

    # Check for doji within the range of the long black candle.
    is_doji = abs(shifted_open - shifted_close) < 0.0001  # Assuming small difference as 'pennies'.
    is_contained = (shifted_high < df['high']) & (shifted_low > df['low'])

    # Combine conditions to detect the pattern.
    bullish_harami_cross = is_long_black & is_doji & is_contained
    
    return bullish_harami_cross

# Ref: https://thepatternsite.com/KickingBull.html
# The bullish kicking candlestick pattern consists of two marubozu candlesticks: a black one followed by a white one with an upward gap between them.
# This function detects this pattern in a given DataFrame.

def do_detect_bullish_kicking(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Bullish Kicking candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the body size for each candle.
    df['body_size'] = abs(df['close'] - df['open'])

    # Identify marubozu candles (candles with no upper or lower shadows).
    df['is_marubozu'] = (df['high'] == max(df['high'], df['close'],df['open']) ) & (df['low'] == min(df['high'],df['close'],df['open']) )

    #Check for gaps.  A gap is defined as a difference between the open and previous close.
    df['gap'] = df['open'] - df['close'].shift(1)


    # Detect the Bullish Kicking pattern: Black marubozu, upward gap, white marubozu.
    bullish_kicking = (df['is_marubozu'].shift(1) & (df['close'].shift(1) < df['open'].shift(1)) &
                       df['is_marubozu'] & (df['open'] > df['close'].shift(1)) & (df['close'] > df['open']) )


    return bullish_kicking
# Ref: https://thepatternsite.com/MeetingLinesBull.html
# This function detects the Bullish Meeting Lines candlestick pattern.
# The pattern consists of two candles: a tall black candle followed by a tall white candle.
# The closes of the two candles should be near one another.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_bullish_meeting_lines(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Bullish Meeting Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Meeting Lines patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_black'] = df['close'] < df['open']
    df['is_white'] = df['close'] > df['open']


    # Identify potential Bullish Meeting Lines patterns
    bullish_meeting_lines = (
        df['is_black'].shift(1) &  # Previous candle is black
        df['is_white'] &  # Current candle is white
        abs(df['close'] - df['close'].shift(1)) < df['body_size']  # Closing prices are close
    )

    return bullish_meeting_lines

# Ref: https://thepatternsite.com/SeparateLinesBull.html
# This function detects the Bullish Separating Lines candlestick pattern.
# The pattern consists of two candles: a tall black candle followed by a tall white candle,
# sharing a common opening price.  The function assumes the input DataFrame has columns
# 'open', 'high', 'low', 'close', 'volume', and 'date'.

def do_detect_bullish_separating_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Separating Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Separating Lines patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['candle_color'] = (df['close'] > df['open']).astype(int)  # 1 for white, 0 for black

    # Detect the pattern: tall black candle followed by tall white candle with nearly the same open price
    bullish_separating_lines = (df['candle_color'].shift(1) == 0) & \
                              (df['candle_color'] == 1) & \
                              (abs(df['open'] - df['open'].shift(1)) < 0.01 * df['high'].shift(1)) & # Adjust tolerance as needed

    return bullish_separating_lines

# Ref: https://thepatternsite.com/SidebySideWhiteLinesBull.html
# This function detects the "Bullish Side by Side White Lines" candlestick pattern.
# The pattern consists of three consecutive white candles in an upward trend.
# The last two candles should have bodies of similar size, opening near the same price,
# and above the high of the first candle's body.

def do_detect_bullish_side_by_side_white_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Side by Side White Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Side by Side White Lines patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes
    df['body'] = df['close'] - df['open']
    
    #Identify white candles
    df['is_white'] = df['body'] > 0

    # Check for three consecutive white candles
    is_bullish_pattern = (df['is_white'] & df['is_white'].shift(1) & df['is_white'].shift(2))

    # Check body size similarity for the last two candles
    similar_body_size = abs(df['body'] - df['body'].shift(1)) < abs(df['body']) * 0.5
    similar_body_size = similar_body_size.shift(-1)

    # Check open price similarity for the last two candles
    similar_open_price = abs(df['open'] - df['open'].shift(1)) < abs(df['open']) * 0.1
    similar_open_price = similar_open_price.shift(-1)
    
    # Check if last two open prices are above the high of the first candle.
    above_first_high = df['open'] > df['high'].shift(2)
    above_first_high = above_first_high.shift(-1).shift(-1)

    #Combine conditions
    bullish_side_by_side_white_lines = is_bullish_pattern & similar_body_size & similar_open_price & above_first_high

    return bullish_side_by_side_white_lines


# Ref: https://thepatternsite.com/ThreeLineStrikeBull.html
# This function detects the Bullish Three-Line Strike candlestick pattern.
# The pattern consists of three consecutive white candles with increasing closes,
# followed by a fourth candle (black candle) that opens higher than the previous
# high but closes below the open of the first candle.

def do_detect_bullish_three_line_strike(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Bullish Three-Line Strike candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date/index.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Three-Line Strike patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate conditions for each candle
    cond1 = df["close"] > df["open"]  # White candles
    cond2 = df["close"].shift(-1) > df["close"]
    cond3 = df["close"].shift(-2) > df["close"].shift(-1)
    cond4 = df["open"].shift(-3) > df["high"].shift(-1) # Black candle opens higher than previous high
    cond5 = df["close"].shift(-3) < df["open"]  # Black candle closes below the open of the first candle

    # Combine conditions using & operator
    mask = cond1 & cond2 & cond3 & cond4 & cond5

    # Shift the mask back to align with the first candle of the pattern
    return mask.shift(3).fillna(False)


# Ref: https://thepatternsite.com/TriStarBull.html
# The following function implements the detection of the Bullish Tri-Star candlestick pattern.
# It identifies three consecutive doji candles following a downward trend, with the middle doji's body below the other two.
# The function takes a Pandas DataFrame as input, and returns a Pandas Series of boolean values.
# True indicates the presence of the pattern, False otherwise.

def do_detect_bullish_tri_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Tri-Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).

    Returns:
        Pandas Series: Boolean Series indicating Bullish Tri-Star patterns.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body and range
    df['body'] = abs(df['close'] - df['open'])
    df['range'] = df['high'] - df['low']

    # Define a function to check if a candle is a doji
    def is_doji(row):
        return row['body'] / row['range'] < 0.1  # Adjust threshold if needed


    # Apply is_doji function to identify doji candles
    df['is_doji'] = df.apply(is_doji, axis=1)

    # Identify sequences of three consecutive doji candles
    three_doji = df['is_doji'].rolling(3).apply(lambda x: all(x), raw=True)

    # Check for the central doji condition
    bullish_tri_star = pd.Series(False, index=df.index)
    for i in range(1, len(df) - 1):
      if three_doji[i+1]:
        if df['close'][i] < df['close'][i-1] and df['close'][i] < df['close'][i+2]:
          bullish_tri_star.loc[i+1] = True

    return bullish_tri_star


# Ref: https://thepatternsite.com/Busted.html
# This function detects the "Busted Pattern" as described in the provided HTML documentation.
# A busted chart pattern is one in which price breaks out in one direction from a chart pattern, 
# but moves no more than 10% before reversing and breaking out in the new direction.
# A "breakout" means a close above the top or a close below the bottom of a chart pattern.
# This function assumes the input DataFrame has 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

def do_detect_busted_pattern(df: pd.DataFrame) -> pd.Series:
    """
    Detects Busted Chart Patterns.

    Args:
        df: DataFrame with OHLC data, volume and date.

    Returns:
        A pandas Series indicating busted patterns (True) or not (False).
        FIXME: Requires a more sophisticated definition of chart patterns and breakout detection.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Placeholder:  This needs a much more robust implementation to actually detect various chart patterns
    # and their busts. This is just a minimal example.  Replace with proper pattern recognition.
    busted = pd.Series(False, index=df.index)
    return busted


# Ref: https://thepatternsite.com/BlkCandle.html
# The black candlestick shows near random direction under actual market conditions since it acts as a continuation 52% of the time.
# In theory, it can act as either a continuation candlestick or a reversal.
# The only thing remarkable about this candlestick is its frequency: 3rd out of 103 where 1 is the most common.
# Performance over 10 days is about what you would expect: close to last place (ranking 82).
# A bear market after a downward breakout is where the black candlestick gets its strength.
# It meets the predicted price target 84% of the time, drops 6% in 10 days, on average, and ranks 19th for that performance under those conditions.

def do_detect_black_candle(df: pd.DataFrame) -> pd.Series:
    """
    Detects the black candle pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        A pandas Series of booleans indicating the presence of a black candle pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_small_shadow = (df["high"] - df["close"]) < (df["close"] - df["open"])  & (df["open"] - df["low"]) < (df["close"] - df["open"])

    return is_black & is_small_shadow


# Ref: https://thepatternsite.com/BlkCandleShort.html
# The short black candle is characterized by a short black candle body with shorter shadows (upper and lower) than the body height.
# The function detects this pattern in a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The function returns a boolean Series indicating the presence of the pattern for each row in the input DataFrame.

def do_detect_short_black_candle(df: pd.DataFrame) -> pd.Series:
    """
    Detects the 'Short Black Candle' pattern.

    Args:
        df: DataFrame with OHLC data and date.

    Returns:
        Boolean Series indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size and shadow lengths
    body_size = df["close"] - df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]

    # Identify short black candles
    is_short_black = (body_size < 0) & (upper_shadow < abs(body_size)) & (lower_shadow < abs(body_size))

    return is_short_black

# Ref: https://thepatternsite.com/ShortWhiteCandle.html
# This function detects the "Short White Candle" candlestick pattern.
# A short white candle is characterized by a small body (close - open) 
# and short shadows (high - close and open - low).  The definition is
# somewhat subjective, so the function uses relative comparisons
# rather than absolute thresholds for the body and shadows lengths.
# The function compares the body length to the candle's overall range
# (high - low) and the shadow lengths to the body length.


def do_detect_short_white_candle(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Short White Candle candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain "open", "high", "low", and "close" columns.

    Returns:
        pandas.Series: Boolean Series indicating Short White Candle patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    body = df["close"] - df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]
    total_range = df["high"] - df["low"]

    # Condition for Short White Candle:
    # 1. White candle (close > open)
    # 2. Short body relative to total range.
    # 3. Short shadows relative to body length.  This prevents very long shadows.
    # 4. non-zero total range; this prevents division by zero errors.

    is_short_white_candle = (body > 0) & (body / total_range < 0.2) & (upper_shadow / body < 1) & (lower_shadow / body < 1) & (total_range != 0)

    return is_short_white_candle


# Ref: https://thepatternsite.com/WhiteCandle.html
# This function detects the "White Candle" pattern based on the description provided in the given HTML.
# A white candle is defined as a candle with a white body and shadows shorter than the body.
# The function takes a Pandas DataFrame as input and returns a Pandas Series indicating the presence of the pattern.


def do_detect_white_candle(df: pd.DataFrame) -> pd.Series:
    """
    Detects the White Candle pattern in a DataFrame of OHLCV data.

    Args:
        df: Pandas DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        Pandas Series with boolean values indicating the presence of the pattern for each row.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size and shadow lengths
    df['body_size'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.apply(lambda row: max(row['open'], row['close']), axis=1)
    df['lower_shadow'] = df.apply(lambda row: min(row['open'], row['close']), axis=1) - df['low']

    # Check for white candle criteria
    is_white_candle = (df['close'] > df['open']) & (df['upper_shadow'] < df['body_size']) & (df['lower_shadow'] < df['body_size'])

    return is_white_candle



# Ref: https://thepatternsite.com/CloseBlkMarubozu.html
# The Closing Black Marubozu is a tall black candlestick with an upper shadow but no lower one.
# It is identified by a single black candle with an upper shadow but no lower shadow.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_closing_black_marubozu(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Closing Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Closing Black Marubozu patterns (True) or not (False).
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    has_upper_shadow = df["high"] > df["open"]
    no_lower_shadow = df["low"] == df["close"]

    is_closing_black_marubozu = is_black & has_upper_shadow & no_lower_shadow
    return is_closing_black_marubozu


# Ref: https://thepatternsite.com/ClosingWhiteMarubozu.html
# The closing white marubozu is a tall white candle with no upper shadow, but it does have a lower shadow.
# It acts as a continuation of the prevailing price trend.

def do_detect_closing_white_marubozu(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Closing White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.

    Returns:
        pandas.Series: Boolean Series indicating Closing White Marubozu patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Check for tall white candle with no upper shadow but with a lower shadow.
    is_closing_white_marubozu = (df['close'] > df['open']) & (df['high'] == df['close']) & (df['low'] < df['open'])
    return is_closing_white_marubozu


# Ref: https://thepatternsite.com/CollapseDojiStar.html
# The collapsing doji star is a three-candle pattern occurring within an upward trend.
# It consists of a white candle followed by a doji that gaps below the previous candle's low.
# The pattern concludes with a black candle that also gaps below the doji's low.
# No shadows should overlap.

def do_detect_collapsing_doji_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Collapsing Doji Star candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        Boolean Series indicating Collapsing Doji Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadow lengths
    df['body'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.apply(lambda x: max(x['open'], x['close']), axis=1)
    df['lower_shadow'] = df.apply(lambda x: min(x['open'], x['close']), axis=1) - df['low']
    
    # Function to check for doji candle
    def is_doji(row):
        return abs(row['close'] - row['open']) < 0.001 * (row['high'] - row['low'])

    # Detect the pattern
    pattern = (
        (df['close'].shift(2) > df['open'].shift(2))  # White candle
        & (df['low'].shift(1) > df['low'].shift(2) ) # Gap below previous low
        & is_doji(df.iloc[:,0:5].shift(1))    # Doji
        & (df['low'] > df['low'].shift(1))  # Gap below doji
        & (df['close'] < df['open']) #Black candle
        & (df['upper_shadow'].shift(1) < (df['low'].shift(1) - df['high'].shift(2))) # upper shadow check
        & (df['lower_shadow'].shift(1) < (df['low'].shift(2) - df['low'].shift(1)) ) # lower shadow check
        & (df['upper_shadow'] < (df['high'].shift(1)-df['low'].shift(1)) ) # upper shadow check
        & (df['lower_shadow'] < (df['high'] - df['low'].shift(1)) ) # lower shadow check

    )


    return pattern
# Ref: https://thepatternsite.com/ConcealBaby.html
# The Concealing Baby Swallow pattern consists of four consecutive black candles.
# The first two are long black marubozu candles.
# The third candle gaps down but its high trades into the body of the previous day's candle.
# The fourth candle engulfs the third candle, having a higher high and lower low.


def do_detect_concealing_baby_swallow(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Concealing Baby Swallow candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating Concealing Baby Swallow patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    #Helper functions to simplify pattern checks
    def is_black_marubozu(row):
        return row['open'] > row['close'] and abs(row['open'] - row['close']) > 0.0001 and row['high'] - max(row['open'], row['close']) < 0.0001 and min(row['open'], row['close']) - row['low'] < 0.0001

    def is_engulfing(row, prev_row):
        return row['high'] >= prev_row['high'] and row['low'] <= prev_row['low'] and row['open'] > row['close']

    #Check for the pattern in the last four days of the dataframe
    mask = df.apply(is_black_marubozu, axis=1) & df.shift(-1).apply(is_black_marubozu, axis=1) & df.shift(-2).apply(lambda x: x['open'] < x['close'] and x['open'] < df.shift(-1)['close'] and x['high'] >= df.shift(-1)['open'] , axis=1) & df.shift(-3).apply(lambda x: is_engulfing(x, df.shift(-2)), axis=1)
    
    return mask

# Ref: https://thepatternsite.com/DarkCloudCover.html
# This function detects the Dark Cloud Cover candlestick pattern.
# The pattern consists of two candles:
# 1. A long white (bullish) candle.
# 2. A black (bearish) candle that opens above the high of the white candle
#    and closes below the midpoint of the white candle's body.

def do_detect_dark_cloud_cover(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Dark Cloud Cover candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series of booleans indicating Dark Cloud Cover patterns.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the midpoint of the previous candle's body.
    midpoint = (df['open'].shift(1) + df['close'].shift(1)) / 2

    # Check for the Dark Cloud Cover conditions.
    dark_cloud_cover = (
        (df['close'] < midpoint) &  # Current candle closes below midpoint of previous candle
        (df['open'] > df['high'].shift(1)) &  # Current candle opens above previous candle's high
        (df['close'].shift(1) > df['open'].shift(1))  # Previous candle is bullish (white)

    )
    return dark_cloud_cover


# Ref: https://thepatternsite.com/Deliberation.html
# The Deliberation pattern consists of three consecutive candlesticks in an upward trend.
# The first two candles have tall bodies, while the third candle has a small body, opening near the second candle's close.
# Each candle opens and closes higher than the previous one.  The description is somewhat ambiguous regarding the relative
# sizes of the bodies and wicks. This implementation prioritizes the relative body sizes and opening/closing prices.

def do_detect_deliberation(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Deliberation candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date index.

    Returns:
        A pandas Series with True where the pattern is detected, False otherwise.  The index is preserved.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes
    df['body'] = abs(df['close'] - df['open'])

    # Check for three consecutive candles
    deliberation = (df['body'].shift(2) > df['body'].shift(1)) & \
                   (df['body'].shift(1) > df['body']) & \
                   (df['open'].shift(2) < df['open'].shift(1)) & \
                   (df['open'].shift(1) < df['open']) & \
                   (df['close'].shift(2) < df['close'].shift(1)) & \
                   (df['close'].shift(1) < df['close']) & \
                   (df['open'] > df['close'].shift(1))


    return deliberation

# Ref: https://thepatternsite.com/Dragonfly.html
# This function detects the Dragonfly Doji candlestick pattern.
# A Dragonfly Doji is characterized by a long lower shadow, a small body (open and close are very close), and a very short or nonexistent upper shadow.
# The closing price is at or near the high of the candle.  This pattern is considered a bullish reversal signal.

def do_detect_dragonfly_doji(df: pd.DataFrame, lower_wick_threshold: float = 0.7) -> pd.Series:
    """
    Detects Dragonfly Doji pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain "open", "high", "low", "close" columns.
        lower_wick_threshold: Minimum ratio of lower wick length to total candle range.

    Returns:
        pandas.Series: Boolean Series indicating Dragonfly Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the lower wick length and total candle range.
    lower_wick = df["low"] - df["open"]
    total_range = df["high"] - df["low"]
    
    #Condition for the body to be small (open and close are almost equal) is not explicitly mentioned in the text so we omit it for now.
    #Condition to check if close is near the high is also not explicitly mentioned in the text, so it's not included here.

    #Identify Dragonfly Doji based on lower wick threshold.
    is_dragonfly_doji = (lower_wick / total_range) >= lower_wick_threshold

    return is_dragonfly_doji

# Ref: https://thepatternsite.com/GappingDownDoji.html
# This function detects the "Gapping Down Doji" candlestick pattern.
# A gapping down doji is a doji candlestick that opens lower than the previous day's close.
# The function checks if the open and close prices are approximately equal and if there is a gap between the current day's open and the previous day's close.


def do_detect_gapping_down_doji(df: pandas.DataFrame) -> pandas.Series:
    """
    Detects the Gapping Down Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Gapping Down Doji patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the difference between open and close prices.  A small difference indicates a doji.
    doji_condition = abs(df["open"] - df["close"]) <= 0.01 * df["high"].abs()

    # Check for a gap: current open is lower than the previous close.  Shift the close prices by 1.
    gap_condition = df["open"] < df["close"].shift(1)

    # Combine the conditions to identify the gapping down doji pattern.
    return (doji_condition & gap_condition)

# Ref: https://thepatternsite.com/GappingUpDoji.html
# This function detects the "Gapping Up Doji" candlestick pattern.
# A gapping up doji is characterized by a gap up in price followed by a doji candlestick
# in an upward trending market.  The doji has an opening price and a closing price that
# are very close to each other.

def do_detect_gapping_up_doji(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Gapping Up Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data and a date column.  The dataframe must contain 
             columns named "open", "high", "low", "close", "volume", and "date".

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the gap between consecutive days' closing prices.
    df['gap'] = df['open'] - df['close'].shift(1)

    # Identify doji candles (opening and closing prices are very close).
    df['is_doji'] = abs(df['open'] - df['close']) < 0.01 * df['high']  # Adjust the 0.01 threshold as needed.

    # Determine the upward trend using closing prices (simplified).
    df['uptrend'] = df['close'] > df['close'].shift(1)


    # Combine conditions to detect the gapping up doji pattern.
    pattern_detected = (df['gap'] > 0) & (df['is_doji']) & (df['uptrend'].shift(1))

    return pattern_detected

# Ref: https://thepatternsite.com/Gravestone.html
# This function detects the Gravestone Doji candlestick pattern.
# A Gravestone Doji is characterized by a tall upper shadow and a very short or non-existent lower shadow, with the open and close prices being nearly identical near the low of the candle.

def do_detect_gravestone_doji(df: pd.DataFrame, upper_wick_threshold: float = 0.7) -> pd.Series:
    """
    Detects Gravestone Doji pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns "open", "high", "low", "close".
        upper_wick_threshold: Minimum ratio of upper wick length to total candle range.

    Returns:
        pandas.Series: Boolean Series indicating Gravestone Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the upper and lower wick lengths
    df["upper_wick"] = df["high"] - df["close"]
    df["lower_wick"] = df["open"] - df["low"]
    df["body"] = abs(df["close"] - df["open"])

    # Calculate the total candle range
    df["total_range"] = df["high"] - df["low"]

    # Calculate the ratio of upper wick length to total candle range
    df["upper_wick_ratio"] = df["upper_wick"] / df["total_range"]

    # Identify Gravestone Doji candles based on criteria
    is_gravestone_doji = (df["upper_wick_ratio"] >= upper_wick_threshold) & (df["lower_wick"] <= df["body"]) & (df["body"] <= 0.01 * df["total_range"])


    return is_gravestone_doji

# Ref: https://thepatternsite.com/LongLegDoji.html
# This function detects the "Long Legged Doji" candlestick pattern.
# A long legged doji is characterized by an opening and closing price that are very close to each other,
# accompanied by significantly long upper and lower shadows.  The shadows represent indecision between buyers and sellers.
# The function identifies this pattern by comparing the body size to the upper and lower shadow lengths.
# Specific thresholds for shadow length are not provided in the documentation, so reasonable defaults are used.


def do_detect_long_legged_doji(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Long Legged Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Long Legged Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size
    body_size = abs(df["close"] - df["open"])

    # Calculate upper and lower shadow lengths
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]
    
    # Check for doji (body size is small relative to shadows) and long shadows
    is_doji = body_size < 0.1 * (upper_shadow + lower_shadow)  # Adjust 0.1 if needed
    is_long_legged = (upper_shadow > 0.7 * (df['high'] - df['low'])) & (lower_shadow > 0.7 * (df['high'] - df['low'])) # Adjust 0.7 if needed

    #Combine conditions
    is_long_legged_doji = is_doji & is_long_legged

    return is_long_legged_doji


# Ref: https://thepatternsite.com/NorthernDoji.html
# This function detects the 'Doji, northern' candlestick pattern.
# A northern doji is characterized by a doji candlestick (open and close prices are nearly equal) appearing within an upward price trend.
# The function identifies this pattern by checking if the absolute difference between the open and close prices is within a specified threshold,
# and if the close price is higher than the previous day's close.
# The threshold is a parameter that can be adjusted to control the sensitivity of the pattern detection.
# The function returns a pandas Series of boolean values, where True indicates the presence of a northern doji on that day.

def do_detect_northern_doji(df: pd.DataFrame, threshold: float = 0.01) -> pd.Series:
    """
    Detects the 'Doji, northern' candlestick pattern.

    Args:
        df: DataFrame with "open", "high", "low", "close", "volume", and "date" columns.
        threshold: The maximum absolute difference between open and close prices to be considered a doji.

    Returns:
        A pandas Series of booleans indicating the presence of a northern doji pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the absolute difference between open and close prices.
    df['doji_diff'] = abs(df['open'] - df['close']) / df['high']

    # Identify doji candles.
    df['is_doji'] = df['doji_diff'] <= threshold

    # Check if the close price is higher than the previous day's close.
    df['upward_trend'] = df['close'] > df['close'].shift(1)

    # Combine conditions to detect northern doji.
    df['is_northern_doji'] = df['is_doji'] & df['upward_trend']

    return df['is_northern_doji']


# Ref: https://thepatternsite.com/SouthernDoji.html
# This function detects the Southern Doji candlestick pattern.
# A Southern Doji is a doji candlestick appearing in a downward price trend.
# The function identifies it by comparing the opening and closing prices,
# and checking if the candle is within a downward trend.

def do_detect_southern_doji(df: pd.DataFrame) -> pd.Series:
    """
    Detects Southern Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating Southern Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the difference between open and close prices.
    df['doji_diff'] = abs(df['open'] - df['close'])
    
    # Identify potential doji candles.  A small threshold is used to account for minor discrepancies.
    df['is_doji'] = df['doji_diff'] <= 0.01 * (df['high'] - df['low'])

    # Identify downward trends using a simple moving average (SMA)  Over a short period to define a recent downward trend.
    df['sma_short'] = df['close'].rolling(window=3).mean()  
    df['is_downtrend'] = df['close'] < df['sma_short']
    
    # Combine conditions to detect Southern Doji.
    is_southern_doji = df['is_doji'] & df['is_downtrend']

    return is_southern_doji

# Ref: https://thepatternsite.com/DojiStarBear.html
# This function detects the 'Doji star, bearish' candlestick pattern.
# The pattern consists of two candles:
# 1. A long white candle.
# 2. A doji candle that gaps higher than the previous candle's high.
# The doji's body should be small relative to the previous candle's body, and its shadows should be short.

def do_detect_bearish_doji_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Doji Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size and upper/lower wick lengths
    df['body_size'] = abs(df['close'] - df['open'])
    df['upper_wick'] = df['high'] - df.loc[:,['open','close']].max(axis=1)
    df['lower_wick'] = df.loc[:,['open','close']].min(axis=1) - df['low']
    
    # Shift data for comparison with previous candle
    df['prev_body_size'] = df['body_size'].shift(1)
    df['prev_high'] = df['high'].shift(1)
    df['prev_close'] = df['close'].shift(1)


    # Identify potential patterns
    is_doji_star = (
        (df['open'].shift(1) < df['close'].shift(1))  # Previous candle is white
        & (df['open'] > df['prev_high'])           # Gap higher
        & (df['open'] < df['close'] + 0.01)      # Doji condition (approx equal open and close)
        & (df['close'] > df['open'] - 0.01)
        & (df['upper_wick'] + df['lower_wick'] < df['prev_body_size'])  # Short shadows
    )

    return is_doji_star


# Ref: https://thepatternsite.com/DojiStarBull.html
# This function detects the 'Doji star, bullish' pattern in OHLCV data.
# The pattern consists of a tall black candle followed by a doji that gaps below the prior candle's body.
# The doji's shadows should not be unusually long.  A downward breakout confirms the pattern as bearish continuation.


def do_detect_bullish_doji_star(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Bullish Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLCV data ('open', 'high', 'low', 'close', 'volume', 'date').

    Returns:
        pandas.Series: Boolean Series indicating Bullish Doji Star patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and shadow lengths
    df['body_size'] = abs(df['close'] - df['open'])
    df['upper_wick'] = df['high'] - df.apply(lambda x: max(x['open'], x['close']), axis=1)
    df['lower_wick'] = df.apply(lambda x: min(x['open'], x['close']), axis=1) - df['low']

    # Identify potential doji candles (simplified check)
    df['is_doji'] = (abs(df['close'] - df['open']) < 0.01 * (df['high'] - df['low']) )

    # Detect the pattern
    bullish_doji_star = (
        (df['body_size'].shift(1) > 0.03*(df['high'].shift(1)-df['low'].shift(1))) & #Tall black candle
        (df['close'].shift(1) > df['open']) & #Black candle
        (df['open'].shift(1) > df['open']) & #Gap below
        (df['is_doji']) & #Doji
        (df['lower_wick'] < df['body_size']) & (df['upper_wick'] < df['body_size'])
    )

    return bullish_doji_star

# Ref: https://thepatternsite.com/CollapseDojiStar.html
# This function detects the "Collapsing Doji Star" candlestick pattern.
# The pattern consists of three candles:
# 1. A white (up) candle in an upward trend.
# 2. A doji candle that gaps below the previous candle's low.
# 3. A black (down) candle that gaps below the doji's low.
# No shadows on the three candles should overlap.

def do_detect_collapsing_doji_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Collapsing Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Collapsing Doji Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadow lengths
    df['body'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.apply(lambda row: max(row['open'], row['close']), axis=1)
    df['lower_shadow'] = df.apply(lambda row: min(row['open'], row['close']), axis=1) - df['low']

    # Identify Doji candles (body size is very small)
    df['is_doji'] = df['body'] < 0.001 * (df['high'] - df['low'])  # Adjust the threshold as needed

    # Detect Collapsing Doji Star pattern. Adjust gap threshold (0.001) as needed
    pattern = (
        (df['close'].shift(2) > df['open'].shift(2)) &  # White candle 
        (df['low'].shift(1) > df['low'].shift(2)) &  # Gap below the previous candle's low.
        df['is_doji'].shift(1) &
        (df['low'] > df['low'].shift(1) + 0.001*(df['high'].shift(1)-df['low'].shift(1))) & # Gap below doji low.
        (df['close'].shift(1) > df['open'].shift(1)) & #White candle
        (df['close'] < df['open']) # Black Candle
    )

    return pattern

# Ref: https://thepatternsite.com/EveningDojiStar.html
# The evening doji star is a three-candlestick bearish reversal pattern.
# It consists of a long white candle followed by a doji candle whose body is above the previous candle's body,
# and finally a long black candle closing below the midpoint of the first candle's body.  This function implements
# a simplified detection algorithm based on the description.  More robust criteria could be added for greater accuracy.


def do_detect_evening_doji_star(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Evening Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date index.

    Returns:
        A pandas Series indicating Evening Doji Star patterns (True) or not (False).  Returns an empty Series if df is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the midpoint of the first candle's body.
    midpoint = (df['open'].shift(2) + df['close'].shift(2)) / 2

    # Conditions for Evening Doji Star pattern
    is_evening_doji_star = (
        # Tall white candle (First candle)
        (df['close'].shift(2) > df['open'].shift(2)) &
        # Doji (Second candle)  Simplified doji criteria
        (abs(df['close'].shift(1) - df['open'].shift(1)) < 0.001) & #Near zero body
        # The doji gaps above previous candle's body
        (df['open'].shift(1) > df['close'].shift(2)) &
        (df['close'].shift(1) > df['close'].shift(2)) &
        # Tall black candle (Third candle) closing below midpoint
        (df['close'] < df['open']) & (df['close'] < midpoint)
    )

    return is_evening_doji_star


# Ref: https://thepatternsite.com/DownGap3Methods.html
# This function detects the Downside Gap Three Methods candlestick pattern.
# The pattern consists of three candles: two long black candles with a gap between them,
# followed by a white candle that opens within the body of the second black candle
# and closes within the body of the first black candle, closing the gap.


def do_detect_downside_gap_three_methods(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Downside Gap Three Methods candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Downside Gap Three Methods patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    def is_downside_gap_three_methods(row):
        #Check if there are enough rows
        if row.name < 2:
            return False
        
        #Check for two consecutive long black candles with a gap
        if (df["close"].iloc[row.name-2] > df["open"].iloc[row.name-2] and
            df["close"].iloc[row.name-1] > df["open"].iloc[row.name-1] and
            df["open"].iloc[row.name-1] > df["close"].iloc[row.name-2] ): # gap condition
            #Check for a white candle closing the gap.
            if (df["close"].iloc[row.name] < df["open"].iloc[row.name] and
                df["open"].iloc[row.name] >= df["low"].iloc[row.name-1] and
                df["close"].iloc[row.name] <= df["open"].iloc[row.name-2]):
                return True
        return False


    return df.apply(is_downside_gap_three_methods, axis=1)


# Ref: https://thepatternsite.com/DownsideTasukiGap.html
# The Downside Tasuki Gap pattern consists of three candles:
# 1. A black candle in a downward trend.
# 2. Another black candle that gaps lower, with no shadow overlap.
# 3. A white candle that opens within the body of the second candle and closes within the gap between the first and second candles.


def do_detect_downside_tasuki_gap(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Downside Tasuki Gap candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    def detect_pattern(row):
        try:
            #Check for three candles
            open1, high1, low1, close1 = df["open"][row.name -2], df["high"][row.name-2], df["low"][row.name-2], df["close"][row.name-2]
            open2, high2, low2, close2 = df["open"][row.name -1], df["high"][row.name-1], df["low"][row.name-1], df["close"][row.name-1]
            open3, high3, low3, close3 = df["open"][row.name], df["high"][row.name], df["low"][row.name], df["close"][row.name]

            #Check for downward trend in the first two candles
            is_downward = close1 < open1 and close2 < open2
            
            #Check for gap between first and second candles.
            is_gap = low2 > close1

            #Check for no shadow overlap
            is_no_overlap = max(low1,low2) > min(high1,high2)

            #Check for closing condition
            is_close_condition = open3 > close2 and close3 < open2 and close3 > close1


            return is_downward and is_gap and is_no_overlap and is_close_condition
        except KeyError:
            return False
        except IndexError:
            return False


    result = df.iloc[2:].apply(detect_pattern, axis=1)
    return result

# Ref: https://thepatternsite.com/Dragonfly.html
# The Dragonfly Doji is characterized by a long lower shadow, a small body (open and close are very close), and a very short or nonexistent upper shadow.
# This function identifies Dragonfly Doji patterns within a given DataFrame.
# The function considers a candle to be a Dragonfly Doji if the lower shadow is significantly longer than the body and the upper shadow is very short.


def do_detect_dragonfly_doji(df: pd.DataFrame, lower_wick_threshold: float = 0.7) -> pd.Series:
    """
    Detects Dragonfly Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.
        lower_wick_threshold: Minimum ratio of lower wick length to total candle range.

    Returns:
        pandas.Series: Boolean Series indicating Dragonfly Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    df["lower_wick"] = df["low"] - df["min"](df["open"], df["close"])
    df["body"] = abs(df["open"] - df["close"])
    df["total_range"] = df["high"] - df["low"]
    is_dragonfly = (df["lower_wick"] / df["total_range"] >= lower_wick_threshold) & (df["body"] / df["total_range"] <= 0.1) & (df["high"] - max(df["open"], df["close"]) <= df["body"])

    return is_dragonfly

# Ref: https://thepatternsite.com/BearEngulfing.html
# This function detects the Bearish Engulfing candlestick pattern.
# A bearish engulfing pattern occurs in an upward trend when a black candle's body completely engulfs the preceding white candle's body.
# The function identifies this pattern by comparing the open, high, low, and close prices of consecutive candles.
# The function returns a pandas Series of booleans, where True indicates the presence of a bearish engulfing pattern.


def do_detect_bearish_engulfing(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Engulfing candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series with True where a bearish engulfing pattern is detected.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_black'] = df['close'] < df['open']
    df['is_white'] = df['close'] > df['open']


    #Detect engulfing pattern
    is_bearish_engulfing = (df['is_white'].shift(1) & df['is_black'] &
                            (df['open'] > df['close'].shift(1)) & (df['close'] < df['open'].shift(1)) &
                            (df['body_size'] > df['body_size'].shift(1)))

    return is_bearish_engulfing


# Ref: https://thepatternsite.com/BullEngulfing.html
# The following function detects the "Bullish Engulfing" candlestick pattern.
# It takes a Pandas DataFrame as input, with columns 'open', 'high', 'low', 'close', 'volume', and 'date'.
# The function returns a Pandas Series of boolean values, indicating whether a bullish engulfing pattern is present at each row.
# True indicates a bullish engulfing pattern.

def do_detect_bullish_engulfing(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Engulfing candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Engulfing patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size for each candle.
    body_size = df['close'] - df['open']

    # Condition for a bullish engulfing pattern:
    # 1. The current candle must be white (close > open).
    # 2. The previous candle must be black (close < open).
    # 3. The current candle's body must engulf the previous candle's body (current_close >= previous_open and current_open <= previous_close).


    is_bullish_engulfing = (body_size > 0) & (body_size.shift(1) < 0) & (df['close'] >= df['open'].shift(1)) & (df['open'] <= df['close'].shift(1))

    return is_bullish_engulfing


# Ref: https://thepatternsite.com/EveningDojiStar.html
# The Evening Doji Star pattern consists of three candles:
# 1. A long white candle in an upward trend.
# 2. A doji candle whose body gaps above the previous candle's body.
# 3. A long black candle that closes at or below the midpoint of the first candle's body.


def do_detect_evening_doji_star(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Evening Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data and a date column.  Must have columns named "open", "high", "low", "close", "volume", and "date".

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and midpoints
    df['body_size'] = abs(df['close'] - df['open'])
    df['midpoint'] = (df['high'] + df['low']) / 2

    # Detect the pattern
    is_evening_doji_star = (
        (df['body_size'].shift(2) > df['body_size'].shift(1)) &  # Long white candle
        (df['close'].shift(1) == df['open'].shift(1)) &  # Doji candle
        (df['open'].shift(1) > df['close'].shift(2)) &  # Gap above previous candle
        (df['close'] < df['midpoint'].shift(2))  # Long black candle closing below midpoint
    )

    return is_evening_doji_star


# Ref: https://thepatternsite.com/EveningStar.html
# This function detects the Evening Star candlestick pattern.
# The pattern consists of three candles:
# 1. A long white candle (upward trend).
# 2. A small-bodied candle (any color), gapping above the previous candle's body.
# 3. A long black candle, opening below the second candle, closing at least midway down the first candle's body.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_evening_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Evening Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Evening Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and gaps
    df["body_size"] = abs(df["close"] - df["open"])
    df["gap"] = df["open"].shift(-1) - df["close"]

    # Define conditions for each candle
    cond1 = (df["close"] > df["open"]) & (df["body_size"] > df["body_size"].shift(1)) #Tall white candle
    cond2 = (df["gap"] > 0) & (df["body_size"] < df["body_size"].shift(1)) & (df["body_size"] < df["body_size"].shift(-1)) # Small body gapping up
    cond3 = (df["open"].shift(-1) < df["close"]) & (df["close"].shift(-1) < df["open"] + 0.5 * df["body_size"]) & (df["body_size"].shift(-1) > df["body_size"])# Long black candle closing at least midway down the first candle

    # Combine conditions to detect the pattern
    pattern = cond1 & cond2.shift(1) & cond3.shift(2)

    return pattern

# Ref: https://thepatternsite.com/eventpatterns.html
# This function detects event patterns based on the provided HTML documentation.
# The function identifies several event patterns, including earnings surprises, stock rating upgrades/downgrades, 
# stock splits, and Dutch auction tender offers.  However, the specific logic for each event type
# is not defined in the provided HTML; it requires further specification or data sources.
# This implementation provides a basic structure for detecting event patterns; it needs to be expanded 
# with specific criteria for each pattern.
# FIXME: This function requires further implementation to define the specific detection logic for each event pattern.

def do_detect_event_patterns(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects event patterns in a DataFrame.

    Args:
        df: A Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A Pandas Series of booleans indicating the presence of event patterns for each row.  Returns an empty Series if df is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)
    
    # Placeholder for event pattern detection logic.  This needs to be replaced with actual detection logic based on the specification for event patterns.
    # For example, you would need external data to check for earnings surprises, rating upgrades/downgrades, etc.
    # The logic below is just an example and will need to be significantly enhanced.
    event_pattern_detected = pandas.Series(data=False, index=df.index)  

    return event_pattern_detected

# Ref: https://thepatternsite.com/Falling3Methods.html
# The Falling Three Methods pattern consists of five candles.
# The first candle is a long black candle.
# The next three candles are smaller candles that trend upward and stay within the high-low range of the first candle.  The second candle can be either black or white.
# The fifth candle is another long black candle that closes below the close of the first candle.

def do_detect_falling_three_methods(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Falling Three Methods candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Falling Three Methods patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    #Helper function to calculate candle body size
    def candle_body(row):
        return abs(row["close"] - row["open"])

    # Calculate candle body sizes
    df["body"] = df.apply(candle_body, axis=1)

    #Check for pattern
    def is_falling_three_methods(row):
        try:
            c0 = row["body"]
            c1 = df["body"].iloc[row.name -1]
            c2 = df["body"].iloc[row.name -2]
            c3 = df["body"].iloc[row.name -3]
            c4 = df["body"].iloc[row.name -4]

            #Check candle body lengths
            if c0 < c4 * 0.5 or c0 < c1 * 0.5:
                return False

            #Check for upward trend and range
            if df["high"].iloc[row.name-1] > df["high"].iloc[row.name-2] or df["high"].iloc[row.name-2] > df["high"].iloc[row.name-3]:
                return False

            if df["low"].iloc[row.name-1] < df["low"].iloc[row.name-4] or df["low"].iloc[row.name-2] < df["low"].iloc[row.name-4] or df["low"].iloc[row.name-3] < df["low"].iloc[row.name-4]:
                return False
            
            if not (df["close"].iloc[row.name -1] > df["open"].iloc[row.name-1] and df["close"].iloc[row.name -2] > df["open"].iloc[row.name-2] and df["close"].iloc[row.name -3] > df["open"].iloc[row.name-3]):
                return False
            
            if df["low"].iloc[row.name] > df["low"].iloc[row.name-4]:
                return False
            
            if df["high"].iloc[row.name-1] > df["high"].iloc[row.name-4] or df["high"].iloc[row.name-2] > df["high"].iloc[row.name-4] or df["high"].iloc[row.name-3] > df["high"].iloc[row.name-4]:
                return False

            return True
        except IndexError:
            return False

    # Apply the pattern detection function
    result = df.iloc[4:].apply(is_falling_three_methods, axis=1)
    return result


# Ref: https://thepatternsite.com/FallingWindow.html
# The Falling Window pattern is characterized by a gap in a downward trend where yesterday's low is above today's high.
# This function identifies this pattern in a given DataFrame.


def do_detect_falling_window(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Falling Window candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Falling Window patterns (True) or not (False).
        Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Check if yesterday's low is above today's high
    is_falling_window = df['low'].shift(1) > df['high']
    return is_falling_window

# Ref: https://thepatternsite.com/GappingDownDoji.html
# The following function detects the "Gapping Down Doji" candlestick pattern.
# A gapping down doji is characterized by a gap down from the previous candle, followed by a doji candle.
# The doji candle has an open and close price that are very close to each other.
# This function identifies this pattern based on the provided criteria.

def do_detect_gapping_down_doji(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Gapping Down Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating whether a Gapping Down Doji pattern is detected for each row.
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the gap between consecutive candles.
    gap = df["open"] - df["close"].shift(1)

    # Identify doji candles (open and close prices are very close).  A tolerance is needed here; the original definition is vague.
    is_doji = abs(df["open"] - df["close"]) < 0.01 * df["high"] # A 1% tolerance relative to the high is arbitrary, but it represents a relatively small price change.

    # Combine gap and doji conditions.
    is_gapping_down_doji = (gap < 0) & is_doji & (df["open"].shift(1) > df["close"].shift(1)) # The gap must be down, and the previous candle should also be down

    return is_gapping_down_doji


# Ref: https://thepatternsite.com/GappingUpDoji.html
# This function detects the "Gapping Up Doji" candlestick pattern.
# A gapping up doji is characterized by a gap up in price from the previous candle,
# followed by a doji candle (open and close prices are nearly identical).
# The pattern is considered a bearish reversal, occurring in an upward price trend.
# The function identifies the pattern based on the specified conditions.


def do_detect_gapping_up_doji(df: pandas.DataFrame) -> pandas.Series:
    """
    Detects the Gapping Up Doji candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the gap between consecutive candles.
    gap = df["open"] - df["close"].shift(1)

    # Identify doji candles (open and close prices are nearly identical).  A tolerance is added for practicality.
    is_doji = abs(df["open"] - df["close"]) < 0.01 * df["high"]  

    # Combine conditions: gap up and doji.
    is_gapping_up_doji = (gap > 0) & is_doji

    return is_gapping_up_doji


# Ref: https://thepatternsite.com/Gravestone.html
# This function detects the Gravestone Doji candlestick pattern.
# A Gravestone Doji is characterized by a long upper shadow, a very short or non-existent lower shadow, and an opening and closing price near the low of the day.  


def do_detect_gravestone_doji(df: pandas.DataFrame) -> pandas.Series:
    """
    Detects Gravestone Doji candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        Boolean pandas Series indicating Gravestone Doji patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the body size and upper/lower shadow lengths.
    body_size = abs(df['close'] - df['open'])
    upper_shadow = df['high'] - max(df['open'], df['close'])
    lower_shadow = min(df['open'], df['close']) - df['low']

    # Define thresholds for identifying a Gravestone Doji. Adjust these as needed.
    upper_shadow_threshold = 0.7  # Minimum ratio of upper shadow to total range.
    lower_shadow_threshold = 0.1  # Maximum ratio of lower shadow to total range.


    # Apply conditions to identify Gravestone Doji patterns.
    is_gravestone_doji = (
        (upper_shadow / (df['high'] - df['low'])) >= upper_shadow_threshold
    ) & (
        (lower_shadow / (df['high'] - df['low'])) <= lower_shadow_threshold
    ) & (body_size <= (df['high'] - df['low']) * 0.1)


    return is_gravestone_doji


# Ref: https://thepatternsite.com/HikkakeBear.html
# This function detects the bearish Hikkake candlestick pattern.
# The pattern consists of three candles:
# 1. An inside day (lower high and higher low than the previous day).
# 2. A candle with a higher high and higher low than the inside day.
# 3. Confirmation: The price closes below the low of the inside day within three days.


def do_calculate_bearish_hikkake(df: pd.DataFrame) -> pd.Series:
    """
    Detects the bearish Hikkake candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Hikkake patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    #Helper function to identify inside days
    def is_inside_day(row):
        prev_row = df.iloc[row-1] if row > 0 else None
        if prev_row is None: return False
        return prev_row['high'] > row['high'] and prev_row['low'] < row['low']
    
    #Apply helper function to identify inside days.  We need to account for index alignment.
    inside_days = df.index[df.rolling(2, min_periods=1).apply(lambda x: is_inside_day(x.name), raw=True)].tolist()
    
    bearish_hikkake = pd.Series(False, index=df.index)
    
    for i in inside_days:
        #Check for the next higher high, higher low and confirmation
        if i + 1 < len(df) and df['high'].iloc[i+1] > df['high'].iloc[i] and df['low'].iloc[i+1] > df['low'].iloc[i] and any(df['close'].iloc[i+1:min(i+4, len(df))] < df['low'].iloc[i]):
            bearish_hikkake.iloc[i] = True
    return bearish_hikkake


# Ref: https://thepatternsite.com/HikkakeBull.html
# This function detects the bullish hikkake candlestick pattern.
# A bullish hikkake consists of three candles:
# 1. An inside day (lower high and higher low than the previous day).
# 2. A candle with a lower high and lower low than the inside day.
# 3. Confirmation: Price rises above the high of the inside day within three days.
# The function returns a boolean Series indicating the presence of the pattern.


def do_detect_bullish_hikkake(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects bullish hikkake candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date.

    Returns:
        Boolean Series indicating bullish hikkake patterns.  Returns an empty Series if the dataframe is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate daily ranges
    df['daily_range'] = df['high'] - df['low']

    # Identify inside day
    df['inside_day'] = (df['high'].shift(1) > df['high']) & (df['low'].shift(1) < df['low'])

    # Identify hikkake pattern
    df['hikkake'] = df['inside_day'] & (df['high'].shift(-1) < df['high']) & (df['low'].shift(-1) < df['low'])

    #Check confirmation: Price rises above the high of the inside day within three days
    df['confirmation'] = df.apply(lambda row: (df['high'][(df.index.get_loc(row.name) +1):(df.index.get_loc(row.name) + 4)].max() > row['high']) if row['hikkake'] else False, axis=1)

    return df['confirmation']


# Ref: https://thepatternsite.com/Hammer.html
# The following function detects the Hammer candlestick pattern.
# A hammer is characterized by a small body with a long lower shadow,
# at least two or three times the height of the body, and little or no upper shadow.
# It typically appears in a downward trend and signals a potential bullish reversal.
# The function takes a Pandas DataFrame as input and returns a Pandas Series
# of boolean values indicating the presence (True) or absence (False) of the Hammer pattern.

def do_detect_hammer(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Hammer candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close') and date ('date').

    Returns:
        Pandas Series indicating Hammer patterns (True) or not (False).
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size
    df['body_size'] = abs(df['close'] - df['open'])

    # Calculate the lower shadow length
    df['lower_shadow'] = df['open'] - df['low']

    # Calculate the upper shadow length
    df['upper_shadow'] = df['high'] - df['close']
    
    # Check the hammer criteria:
    # 1. Long lower shadow: at least twice the body size.
    # 2. Short upper shadow: less than half the body size
    # 3. Small body size (relative to lower shadow).


    is_hammer = (df['lower_shadow'] >= 2 * df['body_size']) & (df['upper_shadow'] <= 0.5 * df['body_size'])

    return is_hammer


# Ref: https://thepatternsite.com/HammerInv.html
# This function detects the inverted hammer candlestick pattern.
# An inverted hammer is a two-line pattern where the first candle is tall and black,
# and the second candle is short with a long upper shadow and little or no lower shadow.
# The second candle's open must be below the first candle's close.  The second candle cannot be a doji.

def do_detect_inverted_hammer(df: pd.DataFrame) -> pd.Series:
    """
    Detects the inverted hammer candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date column.

    Returns:
        Boolean Series indicating inverted hammer patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body and shadows
    df['body'] = df['close'] - df['open']
    df['upper_shadow'] = df['high'] - max(df['open'], df['close'])
    df['lower_shadow'] = min(df['open'], df['close']) - df['low']

    # Check for inverted hammer conditions
    inverted_hammer = (
        (df['body'] < 0) &  # First candle is black
        (abs(df['body']).shift(-1) < abs(df['body'])) & # Second candle body is smaller than the first
        (df['upper_shadow'].shift(-1) > abs(df['body']).shift(-1)) & # Second candle has long upper shadow
        (df['lower_shadow'].shift(-1) < abs(df['body']).shift(-1) * 0.2) & # Second candle has short lower shadow
        (df['open'].shift(-1) < df['close']) # Second candle opens below first candle close
    )


    return inverted_hammer

# Ref: https://thepatternsite.com/HangingMan.html
# The Hanging Man candlestick pattern is characterized by a small real body at the top of a long lower shadow, appearing within an uptrend.  
# This function identifies potential Hanging Man patterns based on the provided criteria.
# The function considers a candle a Hanging Man if its body is small relative to its lower shadow and it appears within an upward trend.
# The parameters hammer_body_factor and hammer_wick_factor can be adjusted to fine tune the pattern detection.

def do_detect_hanging_man(df: pd.DataFrame, hammer_body_factor: float = 0.2, hammer_wick_factor: float = 2.0) -> pd.Series:
    """
    Detects Hanging Man candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        hammer_body_factor: Factor determining the maximum body size relative to the lower wick.
        hammer_wick_factor: Factor determining the minimum lower wick length relative to the body size.

    Returns:
        pandas.Series: Boolean Series indicating hanging man patterns.  FIXME: Handle edge cases and missing data.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size and lower wick length
    df['body_size'] = abs(df['close'] - df['open'])
    df['lower_wick'] = df['low'] - df['min'](df['open'], df['close'])

    # Identify potential hanging man candles
    is_hanging_man = (df['body_size'] <= df['lower_wick'] * hammer_body_factor) & (df['lower_wick'] >= df['body_size'] * hammer_wick_factor)

    # Check for uptrend (simplified: positive change in the last few periods)
    df['uptrend'] = df['close'].diff(periods=3) > 0

    # Combine criteria to detect Hanging Man pattern
    hanging_man_pattern = is_hanging_man & df['uptrend']
    
    return hanging_man_pattern

# Ref: https://thepatternsite.com/HaramiBear.html
# The following function detects the 'Harami, bearish' candlestick pattern.
# It takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# The function returns a pandas Series of booleans, indicating the presence (True) or absence (False)
# of the pattern for each row in the input DataFrame.  The index of the returned Series is preserved.
# The input DataFrame is not modified.  An empty Series is returned only if the DataFrame is empty.


def do_detect_bearish_harami(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Bearish Harami candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Harami patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['body_color'] = (df['close'] > df['open']).astype(int)

    # Shift data to compare with previous candle
    df['prev_open'] = df['open'].shift(1)
    df['prev_close'] = df['close'].shift(1)
    df['prev_body_size'] = df['body_size'].shift(1)
    df['prev_body_color'] = df['body_color'].shift(1)


    # Apply conditions for bearish harami
    is_bearish_harami = (
        (df['prev_body_color'] == 1)  # Previous candle is white (up)
        & (df['body_color'] == 0)     # Current candle is black (down)
        & (df['open'] >= df['prev_open'])  # Current candle open is above or equal to previous open
        & (df['close'] <= df['prev_close'])  # Current candle close is below previous close
        & (df['high'] <= df['prev_close'])  # Current candle high is below or equal previous close
        & (df['low'] >= df['prev_open'])   # Current candle low is above or equal to previous open
    )


    return is_bearish_harami

# Ref: https://thepatternsite.com/HaramiBull.html
# This function detects the bullish harami candlestick pattern.
# A bullish harami consists of two candles: a long black candle followed by a smaller white candle
# whose body is entirely contained within the body of the black candle.


def do_detect_bullish_harami(df: pd.DataFrame) -> pd.Series:
    """
    Detects the bullish harami candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date column.

    Returns:
        pandas.Series: Boolean Series indicating bullish harami patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = df['close'] - df['open']
    df['is_black'] = df['body_size'] < 0
    df['is_white'] = df['body_size'] > 0

    # Shift data to compare consecutive candles
    shifted_body_size = df['body_size'].shift(1)
    shifted_is_black = df['is_black'].shift(1)
    shifted_is_white = df['is_white'].shift(1)
    shifted_open = df['open'].shift(1)
    shifted_close = df['close'].shift(1)

    # Detect bullish harami
    bullish_harami = (shifted_is_black) & (df['is_white']) & (df['open'] > shifted_open) & (df['close'] < shifted_close)

    return bullish_harami


# Ref: https://thepatternsite.com/HaramiCrossBear.html
# The following function detects the "Harami cross, bearish" candlestick pattern.
# It checks for a tall white candle followed by a doji candle whose range is entirely within the range of the preceding candle.
# The function takes a Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# It returns a pandas Series of booleans, indicating the presence (True) or absence (False) of the pattern for each row.


def do_detect_harami_cross_bearish(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Harami Cross candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Harami Cross patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and ranges
    df['body_size'] = abs(df['close'] - df['open'])
    df['candle_range'] = df['high'] - df['low']

    # Identify tall white candles (previous day)
    is_tall_white = (df['close'] > df['open']) & (df['body_size'] > df['candle_range'] / 2) #The definition of a tall white candle is not precise here

    # Shift data for comparison with the next candle
    shifted_is_tall_white = is_tall_white.shift(1)
    shifted_high = df['high'].shift(1)
    shifted_low = df['low'].shift(1)


    # Detect doji candles (current day)
    is_doji = abs(df['close'] - df['open']) < df['candle_range'] / 10  # Definition of doji needs to be refined.

    # Combine conditions to identify the pattern
    harami_cross_bearish = shifted_is_tall_white & is_doji & (df['high'] < shifted_high) & (df['low'] > shifted_low)

    return harami_cross_bearish


# Ref: https://thepatternsite.com/HaramiCrossBull.html
# This function detects the 'Harami cross, bullish' candlestick pattern.
# It checks for a tall black candle followed by a doji within the first candle's range.
# The function operates on a Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The index of the input DataFrame is preserved in the output.  The input DataFrame is not modified.


def do_detect_bullish_harami_cross(df: pd.DataFrame) -> pd.Series:
    """
    Detects the bullish harami cross candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        Boolean Series indicating bullish harami cross patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body and range
    df['body'] = abs(df['close'] - df['open'])
    df['range'] = df['high'] - df['low']

    # Identify doji candles (opening and closing price nearly equal)
    df['is_doji'] = abs(df['close'] - df['open']) < 0.01 * df['range']


    # Identify tall black candles (open > close, and a significant body)
    df['is_tall_black'] = (df['open'] > df['close']) & (df['body'] > 0.5 * df['range'])

    # Detect the pattern: tall black candle followed by an inside doji
    bullish_harami_cross = (df['is_tall_black'].shift(1) & df['is_doji'] &
                            (df['open'].shift(1) > df['high']) & (df['close'].shift(1) < df['low']))

    return bullish_harami_cross

# Ref: https://thepatternsite.com/HighWave.html
# This function detects the High Wave candlestick pattern.
# A High Wave is characterized by tall upper and lower shadows with a small body that is not a doji.
# The body's color is not specified in the definition.


def do_detect_high_wave(df: pd.DataFrame) -> pd.Series:
    """
    Detects the High Wave candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating High Wave patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size and shadow lengths
    df['body_size'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.apply(lambda x: max(x['close'], x['open']), axis=1)
    df['lower_shadow'] = df.apply(lambda x: min(x['close'], x['open']), axis=1) - df['low']

    # Define thresholds (these could be made configurable)
    body_threshold = 0.2 # Arbitrary small body size threshold
    shadow_threshold = 2 # Arbitrary threshold for tall shadows relative to body size

    # Identify High Wave pattern. The body is not a doji.
    is_high_wave = (df['body_size'] < body_threshold) & (df['upper_shadow'] > shadow_threshold * df['body_size']) & (df['lower_shadow'] > shadow_threshold * df['body_size'])

    return is_high_wave


# Ref: https://thepatternsite.com/HomingPigeon.html
# The Homing Pigeon pattern consists of two candles.
# The first candle is a tall black candle.
# The second candle is a smaller black candle that is entirely contained within the body of the first candle.
# This pattern is considered a bearish continuation pattern, although it's sometimes interpreted as a bullish reversal.


def do_detect_homing_pigeon(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Homing Pigeon candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes
    df['body_size'] = abs(df['close'] - df['open'])

    # Identify potential Homing Pigeon patterns
    is_homing_pigeon = (
        (df['close'] < df['open']) &  # Both candles are black
        (df['close'].shift(-1) < df['open'].shift(-1)) &
        (df['open'].shift(-1) > df['close']) &  # Second candle inside the first
        (df['close'].shift(-1) < df['open']) &
        (df['body_size'].shift(-1) < df['body_size']) # Second candle smaller
    )


    # Shift back to align with the first candle of the pattern
    is_homing_pigeon = is_homing_pigeon.shift(1).fillna(False)

    return is_homing_pigeon


# Ref: https://thepatternsite.com/Identical3Crows.html
# This function detects the Identical Three Crows candlestick pattern.
# It requires a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The function returns a pandas Series of booleans indicating the presence of the pattern.

def do_detect_identical_three_crows(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Identical Three Crows candlestick pattern.

    Args:
        df: DataFrame with OHLCV data and a 'date' column.

    Returns:
        pandas.Series: Boolean Series indicating Identical Three Crows patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    #Check for three consecutive black candles.
    is_black = df['close'] < df['open']
    three_black_candles = is_black.rolling(3).all()

    #Check if the last two candles open near the previous candle's close.
    opens_near_close = abs(df['open'].shift(-1) - df['close']) < abs(0.1 * (df['high'] - df['low']).shift())
    opens_near_close2 = abs(df['open'].shift(-2) - df['close'].shift(-1)) < abs(0.1 * (df['high'] - df['low']).shift(-1))

    # Combine conditions to detect the pattern.
    identical_three_crows = three_black_candles & opens_near_close & opens_near_close2

    return identical_three_crows.shift(2)




# Ref: https://thepatternsite.com/InNeck.html
# The In Neck pattern consists of two candles.
# The first candle is a tall black candle.
# The second candle is a white candle that opens below the first candle's low,
# but closes near the first candle's close, ideally inside the body of the first candle.

def do_detect_in_neck(df: pd.DataFrame) -> pd.Series:
    """
    Detects the In Neck candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating In Neck patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_black'] = df['close'] < df['open']
    df['is_white'] = df['close'] > df['open']


    # Check for the In Neck pattern
    in_neck = (
        df['is_black'].shift(1) &  # Previous candle is black
        df['is_white'] &  # Current candle is white
        df['open'].shift(1) > df['open'] & # Current open is below previous open
        df['close'].shift(1) > df['close']  # Current close is below previous close
        # Add condition to check if current close is within the body of previous black candle
    )

    return in_neck

# Ref: https://thepatternsite.com/HammerInv.html
# The inverted hammer is a two-line candlestick pattern.
# The first candle is tall and black (close near the day's low).
# The second candle is short, with a tall upper shadow and little or no lower shadow.
# The second candle's open must be below the first candle's close.
# The second candle cannot be a doji.

def do_detect_inverted_hammer_2_line(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Inverted Hammer (2-line) candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Inverted Hammer patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body and shadow lengths
    df['body'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.loc[df['close'] > df['open'], 'close']
    df['upper_shadow'] = df['upper_shadow'].fillna(df['high'] - df.loc[df['close'] <= df['open'], 'open'])
    df['lower_shadow'] = df.loc[df['close'] > df['open'], 'open'] - df['low']
    df['lower_shadow'] = df['lower_shadow'].fillna(df.loc[df['close'] <= df['open'], 'close'] - df['low'])

    #Identify the inverted hammer pattern
    inverted_hammer = (
        (df['body'].shift(1) > df['body']) &  #First candle is taller
        (df['close'].shift(1) < df['open'].shift(1)) & # First candle is black
        (df['close'].shift(1) < df['low'].shift(1) + df['body'].shift(1) *0.2) &  #Close near the low
        (df['open'] < df['close'].shift(1)) & #Second candle opens below the first candle's close
        (df['upper_shadow'] > df['body']) & #Tall upper shadow
        (df['lower_shadow'] < df['body']*0.2) # Short or no lower shadow

    )


    return inverted_hammer

# Ref: https://thepatternsite.com/KickingBear.html
# The bearish kicking candlestick pattern consists of two marubozu candles: a white candle followed by a black candle, separated by a gap.
# The function checks for this pattern in a given DataFrame.
# It returns a boolean Series indicating the presence of the pattern for each row.

def do_detect_bearish_kicking(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Bearish Kicking candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['color'] = (df['close'] > df['open']).astype(int) # 1 for white (bullish), 0 for black (bearish)

    # Check for the pattern: white marubozu, gap, then black marubozu
    is_bearish_kicking = (df['color'].shift(1) == 1) & (df['open'] > df['close'].shift(1)) & (df['color'] == 0) & (df['body_size'].shift(1) > 0) & (df['body_size'] > 0)
    
    return is_bearish_kicking

# Ref: https://thepatternsite.com/KickingBull.html
# This function detects the bullish kicking candlestick pattern.
# A bullish kicking pattern consists of two marubozu candles:
# a black one followed by a white one with an upward gap between them.
# The function takes a Pandas DataFrame as input and returns a Pandas Series
# of booleans indicating the presence of the pattern.

def do_detect_bullish_kicking(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Kicking candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns: "open", "high", "low", "close", "volume", and "date".

    Returns:
        pandas.Series: Boolean Series indicating Bullish Kicking patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadows.
    df['body_size'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.apply(lambda row: max(row['open'], row['close']), axis=1)
    df['lower_shadow'] = df.apply(lambda row: min(row['open'], row['close']), axis=1) - df['low']

    # Identify marubozu candles (no upper or lower shadows).
    df['is_marubozu'] = (df['upper_shadow'] == 0) & (df['lower_shadow'] == 0)

    # Identify black and white marubozu candles.
    df['is_black_marubozu'] = df['is_marubozu'] & (df['close'] < df['open'])
    df['is_white_marubozu'] = df['is_marubozu'] & (df['close'] > df['open'])


    # Detect the pattern: black marubozu, upward gap, white marubozu.
    bullish_kicking = (df['is_black_marubozu'].shift(1) &
                       (df['open'] > df['close'].shift(1)) &
                       df['is_white_marubozu'])

    return bullish_kicking


# Ref: https://thepatternsite.com/LadderBottom.html
# The Ladder Bottom pattern consists of five candles.
# The first three are tall black candles with progressively lower lows.
# The fourth candle is a black candle with an upper shadow.
# The fifth candle is a white candle that gaps open above the previous day's body.

def do_detect_ladder_bottom(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Ladder Bottom candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Ladder Bottom patterns (True for each row where the pattern is detected).
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    #Helper function to check if a candle is black
    is_black = lambda row: row['close'] < row['open']

    #Helper function to check if candle has upper shadow
    has_upper_shadow = lambda row: row['high'] > max(row['open'], row['close'])

    #Check for pattern
    pattern_detected = (
        df['close'].shift(4) < df['open'].shift(4) &  #First candle is black
        df['close'].shift(3) < df['open'].shift(3) & #Second candle is black
        df['close'].shift(3) < df['close'].shift(4) & #Second candle lower low than first
        df['close'].shift(2) < df['open'].shift(2) & #Third candle is black
        df['close'].shift(2) < df['close'].shift(3) & #Third candle lower low than second
        df['close'].shift(1) < df['open'].shift(1) & #Fourth candle is black
        has_upper_shadow(df.iloc[:,[0,1,2,3]].iloc[3]) & #Fourth candle has an upper shadow
        df['open'].shift(0) > df['close'].shift(1) & #Fifth candle gaps above previous close
        df['close'].shift(0) > df['open'].shift(0) #Fifth candle is white
        )
    return pattern_detected


# Ref: https://thepatternsite.com/LastEngulfBottom.html
# The Last Engulfing Bottom pattern consists of two candles: a white candle followed by a black candle.
# The black candle's body must completely engulf the white candle's body.  Shadows are ignored.
# This function detects this pattern.

def do_detect_last_engulfing_bottom(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Last Engulfing Bottom candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).

    Returns:
        A pandas Series (boolean mask) indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate body size for each candle
    df["body_size"] = abs(df["close"] - df["open"])

    # Shift data for comparison with previous candle
    df["prev_open"] = df["open"].shift(1)
    df["prev_close"] = df["close"].shift(1)
    df["prev_body_size"] = df["body_size"].shift(1)

    # Check for engulfing condition
    mask = (
        (df["close"] < df["prev_open"])  # Black candle closes below previous open
        & (df["open"] > df["prev_close"])  # Black candle opens above previous close
    )

    return mask

# Ref: https://thepatternsite.com/LastEngulfTop.html
# The Last Engulfing Top is a two-candle pattern where a black candle is followed by a taller white candle that engulfs the black candle's body.
# The white candle's body is entirely above the black candle's high and entirely below the black candle's low.
# This pattern is typically considered a bearish reversal pattern, but studies show it acts as a bullish continuation more often.


def do_detect_last_engulfing_top(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Last Engulfing Top candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series with True where the pattern is detected, False otherwise.  Index is preserved.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    def is_last_engulfing_top(row):
        # Check for two candles
        if row.name == 0:
            return False

        # Check if first candle is black (close < open)
        prev_close = df.loc[row.name - 1, 'close']
        prev_open = df.loc[row.name - 1, 'open']
        if not (prev_close < prev_open):
            return False

        # Check if second candle is white (close > open)
        current_close = row['close']
        current_open = row['open']
        if not (current_close > current_open):
            return False

        # Check engulfing condition
        return (current_open < prev_close) and (current_close > prev_open)


    return df.apply(is_last_engulfing_top, axis=1)


# Ref: https://thepatternsite.com/LongBlack.html
# This function detects the "Long Black Day" candlestick pattern.
# A long black day is defined as a tall black candle with shadows shorter than the body.
# The body is considered "tall" if it is three times the average body height of recent candles.

def do_detect_long_black_day(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Long Black Day candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    df['body'] = abs(df['close'] - df['open'])
    avg_body = df['body'].rolling(window=10).mean()  # Using 10 days as an example, this can be customized.
    df['is_long_black_day'] = (df['close'] < df['open']) & (df['body'] >= 3 * avg_body) & (df['high'] - df['close'] < df['body']) & (df['open'] - df['low'] < df['body'])
    return df['is_long_black_day']


# Ref: https://thepatternsite.com/LongWhiteDay.html
# The following function implements the detection of the "Long White Day" candlestick pattern.
# It identifies a tall white candle with shadows shorter than the body, and a body at least three times the average body height over the last 2-3 weeks.


def do_detect_long_white_day(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Long White Day candlestick pattern.

    Args:
        df: DataFrame with OHLC data, volume, and date.

    Returns:
        Boolean Series indicating Long White Day patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate average body height over the last 2-3 weeks (10-15 trading days).
    avg_body_height = df["close"] - df["open"]
    avg_body_height = avg_body_height.rolling(window=10, min_periods=1).mean()  # Using 10 days as a compromise for 2-3 weeks


    #Identify long white candles with shorter shadows.
    is_long_white = (df["close"] > df["open"]) & (df["high"] - df["close"] < df["close"] - df["open"]) & (df["open"] - df["low"] < df["close"] - df["open"]) & (df["close"] - df["open"] >= 3 * avg_body_height)


    return is_long_white


# Ref: https://thepatternsite.com/LongLegDoji.html
# This function detects the "Long Legged Doji" candlestick pattern.
# A long legged doji is characterized by an opening and closing price that are very close to each other,
# and long upper and lower shadows.  The length of the shadows is subjective and needs to be defined via
# thresholds.

def do_detect_long_legged_doji(df: pd.DataFrame, wick_body_ratio: float = 2.0) -> pd.Series:
    """
    Detects Long Legged Doji pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns: "open", "high", "low", "close".
        wick_body_ratio: Minimum ratio of the sum of upper and lower wicks to the body size.

    Returns:
        pandas.Series: Boolean Series indicating Long Legged Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size
    df["body_size"] = abs(df["close"] - df["open"])

    # Calculate upper and lower wick sizes
    df["upper_wick"] = df["high"] - df.apply(lambda row: max(row["open"], row["close"]), axis=1)
    df["lower_wick"] = df.apply(lambda row: min(row["open"], row["close"]), axis=1) - df["low"]

    # Calculate total wick size
    df["total_wick"] = df["upper_wick"] + df["lower_wick"]

    # Identify Long Legged Dojis based on the wick_body_ratio threshold
    is_long_legged_doji = (df["body_size"] <= 0.001) & (df["total_wick"] / df["body_size"] >= wick_body_ratio)

    return is_long_legged_doji


# Ref: https://thepatternsite.com/BlackMarubozu.html
# This function detects the 'Marubozu, black' candlestick pattern.
# A black marubozu is a tall black candle with no shadows.

def do_detect_black_marubozu(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close' columns).

    Returns:
        pandas.Series: Boolean Series indicating Black Marubozu patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Check for a tall black candle with no shadows.
    is_black_marubozu = (df['close'] < df['open']) & (df['high'] == df['open']) & (df['low'] == df['close'])
    return is_black_marubozu


# Ref: https://thepatternsite.com/CloseBlkMarubozu.html
# The closing black marubozu is a tall black candlestick with an upper shadow but no lower one.
# It's identified by a single black candle with an upper shadow but no lower shadow.
# The function checks for this condition and returns a boolean Series indicating the presence of the pattern.

def do_detect_closing_black_marubozu(df: pd.DataFrame) -> pd.Series:
    """
    Detects the 'Closing Black Marubozu' candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        Boolean Series indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Check for closing black marubozu condition: upper shadow, no lower shadow, close < open.
    is_closing_black_marubozu = (df['high'] > df['close']) & (df['close'] == df['low']) & (df['close'] < df['open'])

    return is_closing_black_marubozu

# Ref: https://thepatternsite.com/ClosingWhiteMarubozu.html
# The closing white marubozu is characterized by a tall white candle with no upper shadow and a lower shadow.
# It's considered a continuation pattern.  This function detects this pattern.

def do_detect_closing_white_marubozu(df: pd.DataFrame) -> pd.Series:
    """
    Detects the 'Closing White Marubozu' candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Check for tall white candle (close > open) and no upper shadow (high == close).
    # A lower shadow is present (low < open).
    is_marubozu = (df['close'] > df['open']) & (df['high'] == df['close']) & (df['low'] < df['open'])
    return is_marubozu


# Ref: https://thepatternsite.com/OpenBlkMaru.html
# This function detects the "Opening Black Marubozu" candlestick pattern.
# A black marubozu is characterized by a tall black candle with no upper shadow and a lower shadow.
# The function checks for these characteristics in a given DataFrame.  No prior trend is considered.

def do_detect_opening_black_marubozu(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Opening Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    no_upper_shadow = df['high'] == df['open']
    has_lower_shadow = df['low'] < df['close']

    return (is_black & no_upper_shadow & has_lower_shadow)


# Ref: https://thepatternsite.com/OpenWhiteMarubozu.html
# This function detects the "Opening White Marubozu" candlestick pattern.
# The pattern is characterized by a tall white candle with an upper shadow but no lower shadow.
# The height is relative and should be compared with other recent candles.
# The function takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# It returns a pandas Series of booleans, where True indicates the presence of the pattern and False indicates its absence.

def do_detect_opening_white_marubozu(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Opening White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.

    Returns:
        pandas.Series: Boolean Series indicating Opening White Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size and upper wick length for each candle
    df['body'] = df['close'] - df['open']
    df['upper_wick'] = df['high'] - df['close']
    df['lower_wick'] = df['open'] - df['low']


    # Identify Opening White Marubozu candles
    is_opening_white_marubozu = (df['body'] > 0) & (df['lower_wick'] == 0) & (df['upper_wick'] > 0)

    return is_opening_white_marubozu


# Ref: https://thepatternsite.com/WhiteMarubozu.html
# This function detects the 'Marubozu, white' candlestick pattern.
# A white marubozu is a tall white candle with no shadows (upper or lower).
# It's considered a continuation pattern, but its predictive power is not exceptionally high.


def do_detect_white_marubozu(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close' columns).

    Returns:
        pandas.Series: Boolean Series indicating White Marubozu patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    is_white_marubozu = (df['open'] < df['close']) & (df['high'] == df['close']) & (df['low'] == df['open'])
    return is_white_marubozu


# Ref: https://thepatternsite.com/MatchingLow.html
# The Matching Low pattern consists of two consecutive candles.
# The first candle is a black candle with a tall body.
# The second candle is a black candle whose closing price matches the closing price of the first candle.
# The low of the second candle does not need to match the low of the first candle.


def do_calculate_matching_low(df: pandas.core.frame.DataFrame, low_diff_threshold: float = 0.001) -> pandas.Series:
    """
    Detects Matching Low pattern.

    Args:
        df: DataFrame with OHLC data.
        low_diff_threshold: Maximum difference (as fraction) between current and previous lows for a match.

    Returns:
        pandas.Series: Boolean Series indicating Matching Low patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and closing prices
    df['body_size'] = abs(df['close'] - df['open'])
    df['close_price'] = df['close']

    # Shift data to compare with the previous candle
    shifted_close_price = df['close_price'].shift(1)
    
    # Create a boolean mask indicating Matching Low pattern
    mask = (df['close'] < df['open']) & (shifted_close_price < shifted_close_price.shift(1)) & (abs(df['close'] - shifted_close_price) < low_diff_threshold)

    return mask


# Ref: https://thepatternsite.com/MatHold.html
# This function detects the Mat Hold candlestick pattern.
# The pattern consists of five candles:
# 1. A tall white candle.
# 2. A small black candle with a higher close than the previous day's low.
# 3. A small candle (any color).
# 4. A small black candle, with all three candles (days 2-4) showing a downward trend but remaining above the low of day 1.
# 5. A tall white candle closing above the high of the previous four candles.

def do_detect_mat_hold(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Mat Hold candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date as index.

    Returns:
        pandas.Series: Boolean Series indicating Mat Hold patterns.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors.
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_white'] = df['close'] > df['open']

    # Function to check if a candle is small relative to the first candle's body
    def is_small_candle(row, first_candle_body):
      return row['body_size'] < first_candle_body

    # Detect Mat Hold patterns.
    mat_hold = pandas.Series(data=False, index=df.index)
    for i in range(len(df) - 4):
        # Check for first tall white candle
        if df['is_white'].iloc[i] and df['body_size'].iloc[i] > df['body_size'].iloc[i+1] and df['body_size'].iloc[i] > df['body_size'].iloc[i+2] and df['body_size'].iloc[i] > df['body_size'].iloc[i+3] :
            # Check for small black candles
            if (not df['is_white'].iloc[i+1] ) and (not df['is_white'].iloc[i+3]) and df['close'].iloc[i+1] > df['low'].iloc[i] and df['close'].iloc[i+2] > df['low'].iloc[i] and df['close'].iloc[i+3] > df['low'].iloc[i] and df['close'].iloc[i+4] > df['high'].iloc[i+3]:
                mat_hold.iloc[i] = True

    return mat_hold


# Ref: https://thepatternsite.com/MeetingLinesBear.html
# This function detects the 'Meeting Lines, Bearish' candlestick pattern.
# The pattern consists of two candles: a tall white candle followed by a tall black candle.
# The closing prices of the two candles should be close to each other.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_meeting_lines_bearish(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Meeting Lines candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body size and shadow lengths.
    df['body'] = abs(df['close'] - df['open'])
    df['upper_shadow'] = df['high'] - df.loc[:,['open', 'close']].max(axis=1)
    df['lower_shadow'] = df.loc[:,['open', 'close']].min(axis=1) - df['low']

    # Identify tall candles (body significantly larger than shadows).
    is_tall_candle = df['body'] > df['upper_shadow'] and df['body'] > df['lower_shadow']

    # Check for the pattern: tall white candle followed by a tall black candle with close prices near each other.
    is_pattern = (is_tall_candle & (df['close'] > df['open'])).shift(1) & (is_tall_candle & (df['close'] < df['open'])) & (abs(df['close'].shift(1) - df['close']) < df['body'])

    # Return boolean Series indicating pattern presence.
    return is_pattern

# Ref: https://thepatternsite.com/MeetingLinesBull.html
# This function detects the "Bullish Meeting Lines" candlestick pattern.
# The pattern consists of two candles: a tall black candle followed by a tall white candle.
# The closes of the two candles should be near one another.
# The function operates on a Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# It returns a pandas Series of booleans indicating the presence of the pattern.

def do_detect_bullish_meeting_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Meeting Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Meeting Lines patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body size and color
    df['body_size'] = abs(df['close'] - df['open'])
    df['candle_color'] = (df['close'] > df['open']).astype(int)


    # Identify potential patterns
    potential_patterns = (df['candle_color'].shift(1) == 0) & (df['candle_color'] == 1) & (abs(df['close'] - df['close'].shift(1)) < df['body_size'].shift(1))

    # Ensure the previous candle is "tall" (subjective)
    potential_patterns = potential_patterns & (df['body_size'].shift(1) > df['body_size'].rolling(window=10).mean().shift(1))

    #Ensure the current candle is "tall"
    potential_patterns = potential_patterns & (df['body_size'] > df['body_size'].rolling(window=10).mean())

    return potential_patterns

# Ref: https://thepatternsite.com/MorningDojiStar.html
# This function detects the Morning Doji Star candlestick pattern.
# It checks for a tall black candle followed by a doji, and then a tall white candle.
# The doji's body must gap below the previous candle's body, and the final candle's body must gap above the doji's.

def do_detect_morning_doji_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Morning Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume and date.

    Returns:
        A pandas Series (boolean mask) indicating Morning Doji Star patterns.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and color.
    df['body_size'] = abs(df['close'] - df['open'])
    df['color'] = (df['close'] > df['open']).astype(int)  # 1 for white (bullish), 0 for black (bearish)


    #Identify the pattern
    morning_doji_star = (df['color'].shift(2) == 0) & \
                        (df['body_size'].shift(2) > df['body_size'].shift(1)) & \
                        (df['low'].shift(1) > df['low'].shift(2)) & \
                        (abs(df['close'].shift(1) - df['open'].shift(1)) < df['body_size'].shift(2) * 0.1) & \
                        (df['color'].shift(1) == 0.5) & \
                        (df['open'].shift(1) > df['close'].shift(2)) & \
                        (df['close'] > df['open'].shift(1)) & \
                        (df['open'] > df['close'].shift(1)) & \
                        (df['color'] == 1) & \
                        (df['body_size'] > df['body_size'].shift(1))


    return morning_doji_star


# Ref: https://thepatternsite.com/MorningStar.html
# This function detects the Morning Star candlestick pattern.
# The pattern consists of three candles:
# 1. A long black candle (downward trend).
# 2. A small-bodied candle (the "star") that gaps below the previous candle's body.
# 3. A long white candle that gaps above the second candle's body and closes at least midway into the first candle's body.

def do_detect_morning_star(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Morning Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).

    Returns:
        pandas.Series: Boolean Series indicating Morning Star patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_white'] = df['close'] > df['open']

    # Apply pattern detection logic
    def detect_pattern(row):
        try:
            prev_row = df.iloc[row.name - 1]
            prev_prev_row = df.iloc[row.name - 2]

            # Check for long black candle
            is_long_black = not prev_prev_row['is_white'] and prev_prev_row['body_size'] > df['body_size'].rolling(window=3).mean().iloc[row.name-2] * 1.5

            # Check for a small-bodied "star"
            is_star = abs(prev_row['close'] - prev_row['open']) < df['body_size'].rolling(window=3).mean().iloc[row.name-1] * 0.5 and prev_row['low'] < prev_prev_row['low']

            # Check for long white candle
            is_long_white = row['is_white'] and row['body_size'] > df['body_size'].rolling(window=3).mean().iloc[row.name] * 1.5 and row['open'] > prev_row['high']

            return is_long_black and is_star and is_long_white
        except IndexError:
            return False

    morning_star_series = df.iloc[2:].apply(detect_pattern, axis=1)
    return morning_star_series

# Ref: https://thepatternsite.com/NorthernDoji.html
# This function detects the Northern Doji candlestick pattern.
# A Northern Doji is a doji candlestick (open and close prices are nearly equal) that appears in an upward trend.
# The function takes a Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# It returns a Pandas Series of booleans, where True indicates the presence of a Northern Doji pattern.


def do_detect_northern_doji(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Northern Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.

    Returns:
        pandas.Series: Boolean Series indicating Northern Doji patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the difference between open and close prices.
    df['open_close_diff'] = abs(df['open'] - df['close'])

    # Define a threshold for considering the open and close prices as nearly equal.
    open_close_diff_threshold = 0.01  # Example threshold, adjust as needed

    # Identify doji candles based on the threshold.
    is_doji = df['open_close_diff'] <= open_close_diff_threshold

    # Check if the doji candles are within an upward trend.
    is_upward_trend = df['close'] > df['open'].shift(1)

    # Combine the conditions to detect Northern Doji.
    is_northern_doji = is_doji & is_upward_trend

    return is_northern_doji


# Ref: https://thepatternsite.com/OnNeck.html
# The On Neck candlestick pattern consists of two candles:
# 1. A tall black candle in a downward trend.
# 2. A white candle whose close is at or near the low of the previous candle.

def do_detect_on_neck(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the On Neck candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        Boolean Series indicating On Neck patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body size and color
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_black'] = df['close'] < df['open']

    # Identify potential On Neck patterns
    on_neck_candidates = df['is_black'] & df.shift(1)['is_black'].apply(lambda x: not x)

    # Check closing price condition
    on_neck_patterns = on_neck_candidates & (abs(df['close'] - df.shift(1)['low']) < 0.001 * df.shift(1)['high'])

    return on_neck_patterns

# Ref: https://thepatternsite.com/OpenBlkMaru.html
# The following function detects the "Opening Black Marubozu" candlestick pattern.
# An opening black marubozu is a tall, black candle with no upper shadow but a lower shadow.
# No prior price trend is required for its identification.  It's considered a continuation pattern approximately half the time.

def do_detect_opening_black_marubozu(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Opening Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Check for tall black candle with no upper shadow and a lower shadow
    is_black_marubozu = (df['close'] < df['open']) & (df['high'] == df['open']) & (df['low'] < df['close'])
    return is_black_marubozu

# Ref: https://thepatternsite.com/OpenWhiteMarubozu.html
# This function detects the Opening White Marubozu candlestick pattern.
# An Opening White Marubozu is a tall white candle with an upper shadow but no lower shadow.
# The function checks for the presence of this pattern in a given DataFrame.
# It returns a boolean Series indicating the presence or absence of the pattern for each row.

def do_detect_opening_white_marubozu(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Opening White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume, indexed by date.

    Returns:
        Boolean Series indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Check for tall white candle with upper shadow but no lower shadow.
    is_opening_white_marubozu = (df["close"] > df["open"]) & (df["high"] > df["close"]) & (df["open"] == df["low"])
    return is_opening_white_marubozu


# Ref: https://thepatternsite.com/Piercing.html
# The piercing pattern is a two-candle bullish reversal pattern.
# It consists of a black candle followed by a white candle.
# The white candle opens below the black candle's low and closes between the midpoint of the black candle's body and its opening price.

def do_detect_piercing_pattern(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Piercing Pattern candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Piercing patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate the midpoint of the black candle's body
    midpoint = (df["open"].shift(1) + df["close"].shift(1)) / 2

    # Check the conditions for the piercing pattern
    mask = (df["close"].shift(1) < df["open"].shift(1)) & \
           (df["open"] < df["close"].shift(1)) & \
           (df["close"] > midpoint)

    return mask

# Ref: https://thepatternsite.com/RickshawMan.html
# The Rickshaw Man pattern is characterized by a tall candle with a doji body (open and close prices nearly equal),
# the body near the middle of the candle, and unusually tall upper and lower shadows.  This function
# identifies candles that meet these criteria.  The exact thresholds for "tall" shadows and "near" the middle
# are parameters.


def do_detect_rickshaw_man(df: pd.DataFrame, body_middle_threshold: float = 0.2, shadow_length_threshold: float = 0.7) -> pd.Series:
    """
    Detects the Rickshaw Man candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close') and optionally volume and date columns.
        body_middle_threshold: Maximum distance of the body midpoint from the candle midpoint (as fraction of total range).
        shadow_length_threshold: Minimum ratio of the sum of upper and lower shadows to the total candle range (high - low).

    Returns:
        Boolean Series indicating Rickshaw Man patterns.  Returns an empty Series if df is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size
    df['body_size'] = abs(df['close'] - df['open'])

    # Calculate total candle range
    df['total_range'] = df['high'] - df['low']

    # Calculate midpoint of body and candle
    df['body_midpoint'] = (df['open'] + df['close']) / 2
    df['candle_midpoint'] = (df['high'] + df['low']) / 2

    # Calculate upper and lower shadow lengths
    df['upper_shadow'] = df['high'] - max(df['open'], df['close'])
    df['lower_shadow'] = min(df['open'], df['close']) - df['low']

    # Calculate the distance of the body midpoint from the candle midpoint as fraction of range
    df['body_midpoint_distance'] = abs(df['body_midpoint'] - df['candle_midpoint']) / df['total_range']


    # Apply criteria
    is_rickshaw_man = (df['body_size'] / df['total_range'] <= body_middle_threshold) & \
                      ((df['upper_shadow'] + df['lower_shadow']) / df['total_range'] >= shadow_length_threshold) & \
                      (df['body_midpoint_distance'] <= body_middle_threshold)

    return is_rickshaw_man

# Ref: https://thepatternsite.com/Rising3Methods.html
# The Rising Three Methods pattern consists of five candles.
# 1. A long white candle in an uptrend.
# 2. Three smaller candles that trend lower, but close within the high-low range of the first candle. Candles 2 and 4 are black, but candle 3 can be any color.
# 3. A long white candle that closes above the close of the first candle.

def do_detect_rising_three_methods(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Rising Three Methods candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.  Index must be preserved.

    Returns:
        A pandas Series indicating Rising Three Methods patterns (True/False).
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Helper function to check candle body size and color
    def check_candle(row):
        body_size = abs(row['close'] - row['open'])
        return body_size

    # Calculate body sizes and color for all candles
    df['body_size'] = df.apply(check_candle, axis=1)

    # Detect pattern
    pattern_detected = False
    rising_three_methods = pd.Series(index=df.index, data=False)  # Initialize with False

    for i in range(len(df) - 4):
        # Check for tall white candle (1)
        if df['close'].iloc[i] > df['open'].iloc[i] and df['body_size'].iloc[i] > df['body_size'].iloc[i+1] and df['body_size'].iloc[i] > df['body_size'].iloc[i+2] and df['body_size'].iloc[i] > df['body_size'].iloc[i+3]:
            # Check for three smaller candles trending lower (2) and final tall white candle (3)
            if (df['high'].iloc[i] >= df['high'].iloc[i+1] >= df['high'].iloc[i+2] >= df['high'].iloc[i+3]) and \
               (df['low'].iloc[i] <= df['low'].iloc[i+1] <= df['low'].iloc[i+2] <= df['low'].iloc[i+3]) and \
               (df['close'].iloc[i+1] <= df['close'].iloc[i]) and \
               (df['close'].iloc[i+3] > df['close'].iloc[i]) and \
               (df['open'].iloc[i+3] < df['close'].iloc[i+3]): #condition for long white candle
               rising_three_methods.iloc[i+3] = True
                

    return rising_three_methods


# Ref: https://thepatternsite.com/RisingWindow.html
# The Rising Window pattern is characterized by a gap in an upward trend where yesterday's high is below today's low.
# This function identifies this pattern in a DataFrame containing OHLCV data.


def do_detect_rising_window(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Rising Window candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of a Rising Window pattern.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the difference between today's low and yesterday's high
    gap = df['low'] - df['high'].shift(1)

    # Identify Rising Windows: Gap is positive and previous day's close is less than the previous day's high.
    rising_window = (gap > 0) & (df['close'].shift(1) < df['high'].shift(1))

    return rising_window


# Ref: https://thepatternsite.com/SeparateLinesBear.html
# This function detects the "Bearish Separating Lines" candlestick pattern.
# The pattern consists of two candles: a tall white candle followed by a tall black candle,
# with similar opening prices.  It's a bearish continuation pattern, best performing after upward breakouts.

def do_detect_bearish_separating_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Separating Lines candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['candle_color'] = np.where(df['close'] > df['open'], 'white', 'black')


    # Identify potential patterns (two consecutive candles)
    pattern_mask = (df['candle_color'].shift(1) == 'white') & (df['candle_color'] == 'black')

    # Additional criteria: similar opening prices and tall candles
    pattern_mask &= (abs(df['open'] - df['open'].shift(1)) / df['open'].shift(1) < 0.05)
    pattern_mask &= (df['body_size'].shift(1) > df['body_size'].shift(1).rolling(window=20).mean()) # tall white candle
    pattern_mask &= (df['body_size'] > df['body_size'].rolling(window=20).mean()) # tall black candle


    return pattern_mask

# Ref: https://thepatternsite.com/SeparateLinesBull.html
# The bullish separating lines candlestick pattern consists of two candles.
# The first is a long black candle, and the second is a long white candle.
# The two candles share a common opening price.
# The pattern is a bullish continuation pattern.
# The pattern is best identified in an upward trending market.

def do_detect_bullish_separating_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bullish Separating Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Separating Lines patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body and wick lengths
    df["body"] = abs(df["close"] - df["open"])
    df["upper_wick"] = df["high"] - df.apply(lambda x: max(x["open"], x["close"]), axis=1)
    df["lower_wick"] = df.apply(lambda x: min(x["open"], x["close"]), axis=1) - df["low"]

    # Check for the pattern: tall black candle followed by tall white candle with common opening price.
    # Using shift to look back at the previous row
    bullish_separating_lines = (
        (df["close"] < df["open"]) # First candle is black
        & (df["body"].shift(-1) > df["body"]) # Second candle body is taller
        & (df["close"].shift(-1) > df["open"].shift(-1)) # Second candle is white
        & (abs(df["open"] - df["open"].shift(-1)) < 0.01 * df["high"]) #Opening prices are nearly equal

    )

    return bullish_separating_lines

# Ref: https://thepatternsite.com/ShootingStar.html
# This function detects the "Shooting Star (1 line)" candlestick pattern.
# A shooting star is characterized by a small body with a long upper shadow and a short or no lower shadow.
# The function identifies this pattern based on the relationship between the open, high, close, and low prices of a candle.
# It returns a boolean Series indicating the presence of the pattern for each row (candle) in the input DataFrame.


def do_detect_shooting_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Shooting Star (1 line) candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Shooting Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size and upper shadow length
    body_size = abs(df["close"] - df["open"])
    upper_shadow = df["high"] - max(df["open"], df["close"])

    # Define criteria for shooting star
    is_shooting_star = (upper_shadow >= 2 * body_size) & (min(df["open"], df["close"]) - df["low"] <= body_size)

    return is_shooting_star

# Ref: https://thepatternsite.com/ShootingStar2.html
# The two-line shooting star consists of a white candle followed by a small-bodied candle with a large upper shadow (at least three times the height of the body), and a gap between the two candles.
# The large upper shadow suggests that the bulls could not hold onto their gains, indicating fading momentum.
# However, this pattern often acts as a bullish continuation rather than a bearish reversal.


def do_detect_two_line_shooting_star(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the two-line shooting star candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and upper shadow lengths.
    body_size = abs(df["close"] - df["open"])
    upper_shadow = df["high"] - df.apply(lambda row: max(row["open"], row["close"]), axis=1)

    # Check conditions for two-line shooting star.
    pattern = (
        (df["close"] > df["open"])  # First candle is white
        & (upper_shadow.shift(-1) >= 3 * body_size.shift(-1))  # Second candle has large upper shadow
        & (df["low"].shift(-1) >= df["close"].shift(-1)) # second candle has small or no lower shadow
        & (df["open"].shift(-1) > df["close"]) # there is a gap between the bodies of the two candles.
    )
    return pattern

# Ref: https://thepatternsite.com/BlkCandleShort.html
# This function detects the "Short Black Candle" pattern based on the criteria described in the provided documentation.
# A short black candle is characterized by a small body and short shadows (upper and lower).
# The function identifies these candles by comparing the body size to the upper and lower shadow lengths.


def do_detect_short_black_candle(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Short Black Candle candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).

    Returns:
        Boolean Series indicating Short Black Candle patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size and shadow lengths.
    body_size = df["close"] - df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]

    # Identify short black candles: close < open, short body, short shadows.
    # The conditions below are based on the description.  The parameters can be adjusted for sensitivity.

    is_short_black_candle = (body_size < 0) & (abs(body_size) < upper_shadow) & (abs(body_size) < lower_shadow) & (upper_shadow < abs(body_size) * 2) & (lower_shadow < abs(body_size) * 2)


    return is_short_black_candle


# Ref: https://thepatternsite.com/ShortWhiteCandle.html
# This function detects the "Short White Candle" candlestick pattern.
# A short white candle is characterized by a small body with shadows shorter than the body.
# The function identifies this pattern by comparing the body size to the upper and lower shadows.


def do_detect_short_white_candle(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Short White Candle candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close' columns).

    Returns:
        pandas.Series: Boolean Series indicating Short White Candle patterns (True for pattern, False otherwise).  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    body_size = df["close"] - df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]

    is_short_white_candle = (body_size > 0) & (body_size < upper_shadow) & (body_size < lower_shadow)

    return is_short_white_candle


# Ref: https://thepatternsite.com/SidebySideWhiteLinesBear.html
# This function detects the 'Bearish Side by Side White Lines' candlestick pattern.
# The pattern consists of three candles: a black candle followed by two white candles.
# The bodies of the two white candles should be roughly the same size, have similar opening prices,
# and their closing prices must remain below the body of the black candle.

def do_detect_bearish_side_by_side_white_lines(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Bearish Side by Side White Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date, index is retained.

    Returns:
        Boolean Series indicating the pattern (True) or not (False).  Returns an empty Series if df is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors.
    df['body_size'] = abs(df['close'] - df['open'])
    df['is_black'] = df['close'] < df['open']
    df['is_white'] = df['close'] > df['open']

    # Apply pattern detection logic.
    pattern_detected = (
        df['is_black'].shift(2) &
        df['is_white'].shift(1) &
        df['is_white'] &
        (df['open'].shift(1) > df['open'].shift(2)) &
        (df['open'].shift(1) < df['open']*1.05) &
        (df['close'].shift(1) < df['open'].shift(2)) &
        (df['close'] < df['open'].shift(2)) &
        (abs(df['body_size'].shift(1) - df['body_size']) < df['body_size'].shift(1)*0.25)
    )
    return pattern_detected

# Ref: https://thepatternsite.com/SidebySideWhiteLinesBull.html
# This function detects the "Bullish Side by Side White Lines" candlestick pattern.
# The pattern consists of three consecutive white candlesticks where the last two candles
# have bodies of similar size, opening near the same price, and above the high of the first candle.
# A gap is expected between the first and the last two candles.

def do_detect_bullish_side_by_side_white_lines(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Bullish Side by Side White Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Bullish Side by Side White Lines patterns.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Check for three consecutive white candles.
    is_white = df['close'] > df['open']
    three_white_candles = is_white.rolling(3).sum() == 3
    
    # Check body size similarity and open price proximity for the last two candles.
    # This uses a threshold (0.1) which should be tuned or made a parameter.
    body_size_diff_threshold = 0.1
    similar_bodies = abs( (df['close'] - df['open']).shift(1) - (df['close'] - df['open']).shift(2) ) / ((df['close'] - df['open']).shift(2)) < body_size_diff_threshold

    #check open price similarity
    open_price_diff = abs(df['open'].shift(1) - df['open'].shift(2))

    #Check gap between first and last two candles
    gap_exists = df['open'].shift(1) > df['high'].shift(2)


    # Combine conditions.
    bullish_side_by_side_white_lines = three_white_candles & similar_bodies & gap_exists

    return bullish_side_by_side_white_lines

# Ref: https://thepatternsite.com/SouthernDoji.html
# This function detects the Southern Doji candlestick pattern.
# A Southern Doji is a doji candlestick that appears in a downward price trend.
# The function identifies a doji by comparing the open and close prices.
# A doji is considered to have occurred if the absolute difference between the open and close is within a specified tolerance.
# The function then checks if the doji appears in a downward trend by comparing it to the previous candle's close.

def do_detect_southern_doji(df: pd.DataFrame, doji_tolerance: float = 0.001) -> pd.Series:
    """
    Detects the Southern Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date index.  Must contain columns "open", "high", "low", "close", "volume", "date".
        doji_tolerance: Maximum absolute difference between open and close for a doji.

    Returns:
        A pandas Series of booleans indicating Southern Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the absolute difference between open and close prices.
    abs_diff = abs(df["open"] - df["close"])

    # Identify doji candles.
    is_doji = abs_diff <= doji_tolerance * (df["high"] - df["low"])

    # Check for downward trend before the doji.
    is_downward_trend = df["close"] < df["close"].shift(1)

    # Combine conditions to identify Southern Doji patterns.
    is_southern_doji = is_doji & is_downward_trend

    return is_southern_doji

# Ref: https://thepatternsite.com/SpinTopBlack.html
# This function detects the "Spinning Top, Black" candlestick pattern.
# A black spinning top is characterized by a small black real body with upper and lower shadows that are longer than the real body.
# The function takes a Pandas DataFrame as input with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# It returns a Pandas Series of booleans, indicating whether a black spinning top pattern is present for each row.


def do_detect_black_spinning_top(df: pd.DataFrame) -> pd.Series:
    """
    Detects the black spinning top candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.

    Returns:
        Pandas Series with boolean values indicating the presence of the pattern.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the real body size
    df['real_body'] = abs(df['close'] - df['open'])
    
    # Calculate the upper and lower shadow sizes
    df['upper_shadow'] = df['high'] - df.apply(lambda row: max(row['open'], row['close']), axis=1)
    df['lower_shadow'] = df.apply(lambda row: min(row['open'], row['close']), axis=1) - df['low']

    # Check for black spinning top conditions
    is_black_spinning_top = (df['close'] < df['open']) & (df['real_body'] < df['upper_shadow']) & (df['real_body'] < df['lower_shadow'])

    return is_black_spinning_top

# Ref: https://thepatternsite.com/SpinTopWhite.html
# This function detects the "Spinning Top, White" candlestick pattern.
# A white spinning top is characterized by a small white body and tall shadows.
# The breakout can be with or against the prevailing price trend.
# The function identifies this pattern by checking the relationship between the body size and the shadow lengths.

def do_detect_white_spinning_top(df: pd.DataFrame) -> pd.Series:
    """
    Detects the white spinning top candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = df['close'] - df['open']
    upper_shadow = df['high'] - df['close']
    lower_shadow = df['open'] - df['low']

    is_white_spinning_top = (body_size > 0) & (body_size < upper_shadow) & (body_size < lower_shadow)

    return is_white_spinning_top


# Ref: https://thepatternsite.com/StickSandwich.html
# The Stick Sandwich pattern consists of three candles:
# 1. A black candle in a downward trend.
# 2. A white candle that closes above the previous day's close.
# 3. A black candle that closes at or near the close of the first day.
# This function detects this pattern.  It assumes the input DataFrame has the required columns.

def do_detect_stick_sandwich(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Stick Sandwich candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = df['close'] - df['open']
    df['is_black'] = df['body_size'] < 0
    df['is_white'] = df['body_size'] > 0

    # Detect the pattern
    pattern = (df['is_black'] &
               df['body_size'].shift(-1).apply(lambda x: x > 0) &
               df['is_black'].shift(-2) &
               df['close'].shift(-2).apply(lambda x: abs(x - df['close']) < 0.001 * df['close']))

    return pattern

# Ref: https://thepatternsite.com/TakuriLine.html
# The Takuri line is a single candlestick pattern characterized by a small body, a long lower shadow at least three times the height of the body, and little or no upper shadow.  It's considered a bullish reversal pattern.
# This function detects the Takuri line pattern in a given DataFrame.

def do_calculate_takuri_line(df: pd.DataFrame, lower_shadow_threshold: float = 3.0) -> pd.Series:
    """
    Detects Takuri line candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close).  Must also contain a 'date' column.
        lower_shadow_threshold: Minimum ratio of lower shadow length to body size.

    Returns:
        pandas.Series: Boolean Series indicating Takuri line patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = abs(df['close'] - df['open'])
    lower_shadow = df['open'] - df['low']
    upper_shadow = df['high'] - df['close']

    is_takuri = (lower_shadow / body_size >= lower_shadow_threshold) & (upper_shadow <= body_size)

    return is_takuri


# Ref: https://thepatternsite.com/ThreeBlackCrows.html
# This function detects the Three Black Crows candlestick pattern.
# The pattern consists of three consecutive long black candlesticks appearing after an upward trend.
# Each candlestick should open within the body of the previous candlestick and close near its low, creating lower lows.

def do_detect_three_black_crows(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Three Black Crows candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.

    Returns:
        pandas.Series: Boolean Series indicating Three Black Crows patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes
    df['body'] = df['close'] - df['open']

    # Identify black candles (close < open)
    df['is_black'] = df['body'] < 0

    # Check for three consecutive black candles
    three_black_crows = (df['is_black'] & df['is_black'].shift(1) & df['is_black'].shift(2))

    # Check if the candles open within the body of the previous candle
    df['open_within_body'] = (df['open'] >= df['low'].shift(1)) & (df['open'] <= df['high'].shift(1))
    three_black_crows &= (df['open_within_body'] & df['open_within_body'].shift(1))
    

    # Check for lower lows
    three_black_crows &= (df['low'] > df['low'].shift(1)) & (df['low'].shift(1) > df['low'].shift(2))


    # Return the boolean Series indicating the pattern
    return three_black_crows

# Ref: https://thepatternsite.com/ThreeInsideDown.html
# The three inside down candlestick pattern consists of three candles.
# An upward price trend leads to a tall white candle.
# Following that, a smaller black candle appears that fits inside the body of the white candle.
# The last day has a lower close, but the candle can be any color.
# The lower close confirms the downward move.


def do_detect_three_inside_down(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Three Inside Down candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    def is_three_inside_down(row):
        # Check if it is the third candle in the series
        if row.name < 2:
            return False

        # Check for tall white candle
        is_tall_white = df['close'].iloc[row.name - 2] > df['open'].iloc[row.name - 2]

        #Check for small black candle inside tall white candle
        is_small_black_inside = (df['open'].iloc[row.name-1] < df['close'].iloc[row.name-2] and
                                 df['close'].iloc[row.name-1] < df['open'].iloc[row.name-1] and
                                 df['open'].iloc[row.name-1] > df['open'].iloc[row.name-2] and
                                 df['close'].iloc[row.name-1] < df['close'].iloc[row.name-2])

        #check for final candle closing lower
        is_lower_close = df['close'].iloc[row.name] < df['close'].iloc[row.name -1]


        return is_tall_white and is_small_black_inside and is_lower_close


    return df.apply(is_three_inside_down, axis=1)


# Ref: https://thepatternsite.com/ThreeInsideUp.html
# This function detects the "Three Inside Up" candlestick pattern.
# The pattern consists of three candles:
# 1. A tall black candle in a downward trend.
# 2. A small white candle whose body is entirely contained within the body of the previous candle.
# 3. A white candle that closes above the previous close.

def do_detect_three_inside_up(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Three Inside Up candlestick pattern.

    Args:
        df: DataFrame with OHLC data and datetime index.  Must contain columns "open", "high", "low", "close", "volume", and "date".

    Returns:
        Boolean Series indicating Three Inside Up patterns.  Index is aligned with input DataFrame.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['color'] = (df['close'] > df['open']).astype(int) # 1 for white, 0 for black

    # Check for the pattern
    pattern_detected = (
        (df['color'].shift(2) == 0) &  # Previous 2nd candle is black
        (df['body_size'].shift(1) < df['body_size'].shift(2)) &  # Previous candle body smaller than previous 2nd candle
        (df['open'].shift(1) > df['open'].shift(2)) &  #Previous candle's open higher than 2nd candle's open
        (df['close'].shift(1) < df['close'].shift(2)) & #Previous candle's close lower than 2nd candle's close
        (df['low'].shift(1) > df['low'].shift(2)) & #Previous candle's low higher than 2nd candle's low
        (df['high'].shift(1) < df['high'].shift(2)) & #Previous candle's high lower than 2nd candle's high
        (df['color'].shift(1) == 1) & #Previous candle is white
        (df['close'] > df['close'].shift(1)) & # Current candle is white and closes above previous
        (df['color'] == 1)
    )

    return pattern_detected


# Ref: https://thepatternsite.com/ThreeLineStrikeBear.html
# The function detects the bearish three-line strike candlestick pattern.
# It checks for three consecutive black candles with decreasing lows, followed by a tall white candle 
# that opens below the previous close and closes above the first black candle's open.

def do_detect_bearish_three_line_strike(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Bearish Three-Line Strike candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.

    Returns:
        pandas.Series: Boolean Series indicating Bearish Three-Line Strike patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body'] = abs(df['close'] - df['open'])
    df['color'] = (df['close'] > df['open']).astype(int) # 1 for white, 0 for black

    # Detect the pattern
    pattern = (
        (df['color'].shift(3) == 0) &  # First candle black
        (df['color'].shift(2) == 0) &  # Second candle black
        (df['color'].shift(1) == 0) &  # Third candle black
        (df['low'].shift(3) > df['low'].shift(2)) &  # Decreasing lows
        (df['low'].shift(2) > df['low'].shift(1)) &
        (df['color'] == 1) &  # Fourth candle white
        (df['open'] < df['close'].shift(1)) &  # Opens below previous close
        (df['close'] > df['open'].shift(3))  # Closes above first candle's open
    )

    return pattern

# Ref: https://thepatternsite.com/ThreeLineStrikeBull.html
# This function detects the bullish three-line strike candlestick pattern.
# The pattern consists of three consecutive white candles with increasing closes,
# followed by a long black candle that closes below the open of the first candle.

def do_detect_bullish_three_line_strike(df: pd.DataFrame) -> pd.Series:
    """
    Detects the bullish three-line strike candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.

    Returns:
        Boolean Series indicating bullish three-line strike patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body'] = df['close'] - df['open']
    df['is_white'] = df['body'] > 0

    # Detect the pattern
    pattern = (
        df['is_white'].shift(3) &
        df['is_white'].shift(2) &
        df['is_white'].shift(1) &
        ~df['is_white'] &
        (df['close'] < df['open'].shift(3)) &
        (df['close'].shift(1) > df['close'].shift(2)) &
        (df['close'].shift(2) > df['close'].shift(3))
    )

    return pattern

# Ref: https://thepatternsite.com/ThreeOutsideDown.html
# The three outside down candlestick pattern consists of three candles.
# It starts with a white (bullish) candle in an upward trend.
# Then, a black (bearish) candle appears, opening higher and closing lower than the previous candle's body.
# Finally, another black candle has a lower close than the second candle.
# This pattern is considered a bearish reversal pattern.
# The function below will detect this pattern.

def do_detect_three_outside_down(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Three Outside Down candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating whether a Three Outside Down pattern exists for each row.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes
    df['body'] = abs(df['close'] - df['open'])

    # Check for pattern
    pattern = (
        (df['close'] > df['open']) &  # First candle is bullish
        (df['close'].shift(-1) < df['open'].shift(-1)) &  # Second candle opens higher than previous close, closes lower than previous open
        (df['open'].shift(-1) > df['close']) &
        (df['close'].shift(-2) < df['close'].shift(-1))  # Third candle has a lower close than the second candle.
    )

    return pattern
# Ref: https://thepatternsite.com/ThreeOutsideUp.html
# This function detects the "Three Outside Up" candlestick pattern.
# The pattern consists of three candles:
# 1. A black candle in a downward trend.
# 2. A white candle that opens below the previous candle's body and closes above it.
# 3. A white candle with a higher close than the second candle.

def do_detect_three_outside_up(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Three Outside Up candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body size and color
    df['body_size'] = df['close'] - df['open']
    df['is_white'] = df['body_size'] > 0

    # Detect the pattern
    pattern_detected = (
        (df['is_white'].shift(2) == False) &  # Previous 2nd candle is black
        (df['is_white'].shift(1) == True) &  # Previous 1st candle is white
        (df['is_white'] == True) &  # Current candle is white
        (df['open'].shift(1) < df['open'].shift(2)) & # 1st candle opens below 2nd candle
        (df['close'].shift(1) > df['close'].shift(2)) & # 1st candle closes above 2nd candle
        (df['close'] > df['close'].shift(1)) # Current candle closes higher than 1st candle
    )

    return pattern_detected


# Ref: https://thepatternsite.com/ThreeStarsSouth.html
# The Three Stars in the South pattern is a bullish reversal pattern consisting of three candles.
# The first candle is a long black candle with a long lower shadow.
# The second candle is similar to the first, but smaller and with a higher low.
# The third candle is a black marubozu (a candle with no upper or lower shadow) that closes within the high-low range of the second candle.
# A downward breakout from this pattern is considered a continuation pattern.


def do_calculate_three_stars_in_the_south(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Three Stars in the South candlestick pattern.

    Args:
        df: DataFrame with OHLC data.

    Returns:
        pandas.Series: Boolean Series indicating Three Stars in the South patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    def is_three_stars_in_the_south(data):
        c1, c2, c3 = data
        #First candle is tall, black, long lower shadow
        if not (c1['close'] < c1['open'] and c1['low'] < c1['open'] *0.75 and c1['close'] < c1['open'] * 0.9): return False
        #Second candle is smaller and higher low
        if not (c2['close'] < c2['open'] and c2['low'] > c1['low'] and c2['high'] < c1['high'] and c2['close'] > c1['close']): return False
        #Third candle is black marubozu inside c2
        if not (c3['close'] < c3['open'] and c3['low'] >= c2['low'] and c3['high'] <= c2['high']): return False        
        return True

    three_stars_in_the_south = df[['open', 'high', 'low', 'close']].rolling(3).apply(is_three_stars_in_the_south, raw=True)
    return three_stars_in_the_south.fillna(False).astype(bool)


# Ref: https://thepatternsite.com/ThreeWhiteSoldiers.html
# This function detects the Three White Soldiers candlestick pattern.
# The pattern consists of three consecutive tall white candles, each with a close near the high,
# higher closes, and bodies that overlap (opening price within the prior candle's body).
# The function takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# It returns a pandas Series of booleans indicating the presence of the pattern at each index.


def do_detect_three_white_soldiers(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the Three White Soldiers candlestick pattern.

    Args:
        df: DataFrame with OHLCV data and a 'date' column.

    Returns:
        pandas.Series: Boolean Series indicating Three White Soldiers patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and upper shadows
    df['body'] = df['close'] - df['open']
    df['upper_shadow'] = df['high'] - df['close']

    # Check for three consecutive white candles
    is_white = df['body'] > 0
    three_white_soldiers = (is_white) & (is_white.shift(1)) & (is_white.shift(2))

    # Check for overlapping bodies and high closes
    three_white_soldiers &= (df['open'].shift(1) >= df['open'].shift(2)) & (df['open'].shift(1) <= df['close'].shift(2))
    three_white_soldiers &= (df['open'] >= df['open'].shift(1)) & (df['open'] <= df['close'].shift(1))
    three_white_soldiers &= (df['close'] > df['close'].shift(1)) & (df['close'] > df['close'].shift(2))
    three_white_soldiers &= (df['upper_shadow'] / (df['high'] - df['low']) < 0.3).shift(2)
    three_white_soldiers &= (df['upper_shadow'].shift(1) / (df['high'].shift(1) - df['low'].shift(1)) < 0.3)
    three_white_soldiers &= (df['upper_shadow'].shift(1) / (df['high'].shift(1) - df['low'].shift(1)) < 0.3)

    return three_white_soldiers

# Ref: https://thepatternsite.com/Thrusting.html
# The Thrusting pattern consists of two candles:
# 1. A black candle in a downward trend.
# 2. A white candle that opens below the previous low and closes near, but below, the midpoint of the black candle's body.


def do_detect_thrusting(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Thrusting candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating Thrusting patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate midpoint of previous candle's body
    midpoint = (df["open"].shift(1) + df["close"].shift(1)) / 2

    # Conditions for Thrusting pattern
    is_thrusting = (
        (df["close"].shift(1) < df["open"].shift(1))  # Previous candle is black
        & (df["open"] < df["low"].shift(1))  # Current candle opens below previous low
        & (df["close"] < midpoint)  # Current candle closes below midpoint of previous candle
        & (df["close"] > df["open"])  # Current candle is white
    )

    return is_thrusting

# Ref: https://thepatternsite.com/TriStarBear.html
# This function detects the bearish tri-star candlestick pattern.
# The pattern consists of three doji candles, with the middle one having a body above the other two.
# The function takes a Pandas DataFrame as input, containing OHLCV data, and returns a pandas Series of booleans.
# True indicates the presence of the pattern, False otherwise.

def do_detect_bearish_tri_star(df: pd.DataFrame) -> pd.Series:
    """
    Detects the bearish tri-star candlestick pattern.

    Args:
        df: DataFrame with OHLCV data.

    Returns:
        pandas.Series: Boolean Series indicating bearish tri-star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size of each candle.
    body_size = abs(df["close"] - df["open"])

    # Identify potential doji candles (small body size).
    doji_mask = body_size < 0.005 * (df["high"] - df["low"])

    # Check for three consecutive doji candles.
    three_doji_mask = doji_mask & doji_mask.shift(1) & doji_mask.shift(2)

    # Check if the middle doji has a body above the other two.
    middle_high = df["high"].shift(1)
    middle_low = df["low"].shift(1)
    valid_tri_star = (df["open"].shift(1) > df["open"].shift(2)) & (df["close"].shift(1) > df["close"].shift(2)) & (middle_high > df["high"].shift(2)) & (middle_low < df["low"].shift(2))


    # Combine conditions to detect the pattern.
    bearish_tri_star_mask = three_doji_mask & valid_tri_star

    return bearish_tri_star_mask

# Ref: https://thepatternsite.com/TriStarBull.html
# This function detects the bullish tri-star candlestick pattern.
# It identifies three doji candles following a downward trend,
# where the middle doji's body is below the other two.
# The function returns a boolean Series indicating the presence
# of the pattern for each row in the input DataFrame.

def do_detect_bullish_tri_star(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the bullish tri-star candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.

    Returns:
        pandas.Series: Boolean Series indicating bullish tri-star patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate body size and wick lengths
    df['body'] = abs(df['close'] - df['open'])
    df['upper_wick'] = df['high'] - df.apply(lambda row: max(row['open'], row['close']), axis=1)
    df['lower_wick'] = df.apply(lambda row: min(row['open'], row['close']), axis=1) - df['low']
    
    #Identify Doji Candles
    df['is_doji'] = df['body'] < 0.05 * (df['high']-df['low']) #5% threshold for simplicity

    # Detect Bullish Tri-Star
    bullish_tri_star = pandas.Series(False, index=df.index)
    for i in range(2, len(df)):
        if (df['is_doji'].iloc[i-2] and df['is_doji'].iloc[i-1] and df['is_doji'].iloc[i] and
            df['low'].iloc[i-1] < df['low'].iloc[i-2] and df['low'].iloc[i-1] < df['low'].iloc[i] and
            df['close'].iloc[i] > df['close'].iloc[i-2] and df['close'].iloc[i] > df['close'].iloc[i-1]
           ): # Check for downward trend before tri-star and upward breakout after tri-star
            bullish_tri_star.iloc[i] = True

    return bullish_tri_star


# Ref: https://thepatternsite.com/TweezersBottom.html
# The tweezers bottom candlestick pattern consists of two candles sharing the same low price.
# It's characterized by a downward price trend leading to the pattern.
# While theoretically a bullish reversal, testing shows it's a bearish continuation pattern a significant portion of the time.
# The function identifies the pattern based on the low price of consecutive candles.

def do_detect_tweezers_bottom(df: pandas.DataFrame) -> pandas.Series:
    """
    Detects the Tweezers Bottom candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series of booleans indicating the presence of the Tweezers Bottom pattern.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Check if the low price of consecutive candles are equal
    is_tweezers_bottom = df['low'].rolling(2).apply(lambda x: x[0] == x[1])

    # Shift the result to align with the second candle of the pattern
    is_tweezers_bottom = is_tweezers_bottom.shift(-1).fillna(False)

    return is_tweezers_bottom.astype(bool)


# Ref: https://thepatternsite.com/TweezersTop.html
# This function detects the Tweezers Top candlestick pattern.
# A Tweezers Top is characterized by two consecutive candlesticks with the same high price, appearing within an upward trend.
# The function takes a Pandas DataFrame as input, containing 'open', 'high', 'low', 'close', 'volume', and 'date' columns.  
# It returns a Pandas Series of booleans, indicating the presence (True) or absence (False) of the pattern for each row in the DataFrame.

def do_detect_tweezers_top(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Tweezers Top candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        Pandas Series with booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Check if the high of the current candle is equal to the high of the previous candle.
    same_high = df['high'] == df['high'].shift(1)

    # Check if the pattern appears in an upward trend.  This is a simplification; a more robust check would be needed.
    upward_trend = df['close'] > df['open']

    # Combine the conditions to identify the Tweezers Top pattern.
    is_tweezers_top = same_high & upward_trend

    # Shift the result to align with the second candle (the pattern is identified by the second candle).
    return is_tweezers_top.shift(-1).fillna(False)


# Ref: https://thepatternsite.com/TwoBlackGapping.html
# This function detects the "Two Black Gapping" candlestick pattern.
# The pattern consists of two consecutive black candles with a gap between them,
# where the second candle's high is lower than the first candle's high.
# The function takes a Pandas DataFrame as input, containing OHLCV (Open, High, Low, Close, Volume) data and a date column.
# It returns a Pandas Series of boolean values, indicating the presence (True) or absence (False) of the pattern for each row.


def do_detect_two_black_gapping(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Two Black Gapping candlestick pattern.

    Args:
        df: DataFrame with OHLCV data and a 'date' column.

    Returns:
        Pandas Series with boolean values indicating the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Check for two consecutive black candles with a gap and decreasing highs.
    two_black_gapping = (df['close'] < df['open']) & \
                       (df['open'].shift(-1) > df['close']) & \
                       (df['high'].shift(-1) < df['high']) & \
                       (df['open'].shift(-1) > df['close'].shift(-1))


    return two_black_gapping

# Ref: https://thepatternsite.com/TwoCrows.html
# The function identifies the "Two Crows" candlestick pattern in OHLCV data.
# It checks for a tall white candle followed by two black candles meeting specific criteria.
# The function returns a boolean Series indicating the presence of the pattern.


def do_detect_two_crows(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects the Two Crows candlestick pattern.

    Args:
        df: DataFrame with OHLCV data ('open', 'high', 'low', 'close', 'volume', 'date').

    Returns:
        pandas.Series: Boolean Series indicating Two Crows patterns (True for each row where a pattern is detected)
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = abs(df['close'] - df['open'])
    df['color'] = (df['close'] > df['open']).astype(int)  # 1 for white (up), 0 for black (down)

    # Identify potential Two Crows patterns
    two_crows = (df['color'].shift(2) == 1) & \
                (df['color'].shift(1) == 0) & \
                (df['color'] == 0) & \
                (df['open'].shift(1) > df['open'].shift(2)) & \
                (df['open'] < df['close'].shift(1)) & \
                (df['close'] < df['open'].shift(2))


    return two_crows

# Ref: https://thepatternsite.com/Unique3RiverBottom.html
# This function detects the "Unique Three-River Bottom" candlestick pattern.
# The pattern consists of three candles:
# 1. A tall black candle.
# 2. A smaller black candle whose body is inside the first candle's body, but its lower shadow extends below the first candle's low.
# 3. A short white candle whose body is below the second candle's body.
# The function returns a pandas Series of booleans, indicating the presence of the pattern.

def do_detect_unique_three_river_bottom(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Unique Three-River Bottom candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Unique Three-River Bottom patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadow lengths
    df['body'] = abs(df['close'] - df['open'])
    df['lower_shadow'] = df[['low', 'open', 'close']].min(axis=1) - df['low']
    df['upper_shadow'] = df['high'] - df[['high', 'open', 'close']].max(axis=1)

    # Check for the pattern using a rolling window of size 3
    def is_unique_three_river_bottom(window):
        c1, c2, c3 = window
        #First candle is tall black candle
        is_black1 = c1['close'] < c1['open']
        is_tall1 = c1['body'] > 0
        #Second candle is smaller black candle whose lower shadow extends below the prior candle
        is_black2 = c2['close'] < c2['open']
        is_smaller2 = c2['body'] < c1['body']
        is_lower_shadow2 = c2['low'] < c1['low']
        #Third candle is a small white candle below the second candle
        is_white3 = c3['close'] > c3['open']
        is_small3 = c3['body'] < c2['body']
        is_below3 = c3['close'] < c2['close']

        return (is_black1 and is_tall1 and is_black2 and is_smaller2 and is_lower_shadow2 and is_white3 and is_small3 and is_below3)


    pattern_mask = df[['open', 'high', 'low', 'close']].rolling(3).apply(is_unique_three_river_bottom, raw=True)

    return pattern_mask.iloc[2:]


# Ref: https://thepatternsite.com/UpGap3Methods.html
# This function detects the Upside Gap Three Methods candlestick pattern.
# The pattern consists of three candles: two tall white candles with a gap between them,
# followed by a black candle that fills the gap.  The black candle opens above the low
# of the second white candle and closes below the high of the first white candle.

def do_detect_upside_gap_three_methods(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Upside Gap Three Methods candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date.

    Returns:
        pandas.Series: Boolean Series indicating Upside Gap Three Methods patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    #Calculate daily ranges
    df['daily_range'] = df['high'] - df['low']

    #Check for three-candle pattern and gap conditions
    pattern = (
        (df['close'] > df['open']) &  #First candle is white
        (df['close'].shift(-1) > df['open'].shift(-1)) & #Second candle is white
        (df['open'].shift(-1) > df['close']) & #Gap between first and second candles
        (df['close'].shift(-2) < df['open'].shift(-1)) & #Third candle is black
        (df['open'].shift(-2) > df['low'].shift(-1)) & #Third candle opens above second candle's low
        (df['close'].shift(-2) < df['high']) #Third candle closes below first candle's high
    )

    return pattern

# Ref: https://thepatternsite.com/UpGapTwoCrows.html
# This function detects the "Upside Gap Two Crows" candlestick pattern.
# The pattern consists of three candles:
# 1. A tall white candle in an upward trend.
# 2. A black candle gapping above the previous candle's body.
# 3. Another black candle engulfing the body of the second candle,
#    with a close above the first candle's close.

def do_detect_upside_gap_two_crows(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Upside Gap Two Crows candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and colors
    df['body_size'] = df['close'] - df['open']
    df['is_white'] = df['body_size'] > 0

    # Detect the pattern using a rolling window
    pattern_detected = (
        df['is_white'].shift(2) &  # First candle is white
        (df['open'].shift(1) > df['close'].shift(2)) &  # Second candle gaps above the first
        (df['body_size'].shift(1) < 0) &  # Second candle is black
        (df['body_size'] < 0) &  # Third candle is black
        (df['open'] > df['close'].shift(1)) & #Third candle engulfs second
        (df['close'] < df['open'].shift(1)) & #Third candle engulfs second
        (df['close'] > df['close'].shift(2))  # Third candle close is above the first candle's close
    )
    return pattern_detected

# Ref: https://thepatternsite.com/UpsideTasukiGap.html
# This function detects the Upside Tasuki Gap candlestick pattern.
# The pattern consists of three candles:
# 1. A white candle in an upward trend.
# 2. Another white candle that gaps higher, with non-overlapping shadows.
# 3. A black candle that opens within the body of the second candle and closes within the gap.

def do_detect_upside_tasuki_gap(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Upside Tasuki Gap candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns: "open", "high", "low", "close", "volume", and "date".

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes and shadow lengths
    df['body'] = df['close'] - df['open']
    df['upper_shadow'] = df['high'] - df.apply(lambda x: max(x['open'], x['close']), axis=1)
    df['lower_shadow'] = df.apply(lambda x: min(x['open'], x['close']), axis=1) - df['low']

    # Detect the pattern
    pattern_detected = (
        (df['body'].shift(2) > 0) &  # First candle is white
        (df['close'].shift(1) > df['close'].shift(2)) &  # Second candle gaps higher
        (df['open'].shift(1) > df['close'].shift(2)) & #Second candle opens higher
        (df['close'].shift(1) > df['open'].shift(1)) & #Second candle is white
        ((df['upper_shadow'].shift(1) + df['low'].shift(1)) < (df['open'].shift(2))) & #Gap between shadows of first and second candle
        (df['body'].shift(1) > 0) & # Second candle is white
        (df['open'].shift(0) < df['close'].shift(1)) & #Third candle opens in the body of the second candle
        (df['body'].shift(0) < 0) & # Third candle is black
        (df['close'].shift(0) > df['open'].shift(2)) & #Third candle closes above the open of the first candle
        (df['close'].shift(0) < df['open'].shift(1))  #Third candle closes inside the gap
        
    )
    return pattern_detected

# Ref: https://thepatternsite.com/volpatterns.html
# This function detects volume patterns based on the provided OHLCV data.  
# The specific criteria for volume patterns are not explicitly defined in the provided HTML, 
# so this implementation provides a basic example that can be customized.  
# More sophisticated volume pattern detection would require a more detailed specification.
# For instance, the detection of "Volume" patterns is not specified, so I have made some assumptions.
# The HTML does not describe algorithms for detecting volume patterns. Therefore, a placeholder algorithm is provided.

def do_detect_volume_patterns(df: pandas.core.frame.DataFrame) -> pandas.core.series.Series:
    """
    Detects volume patterns in OHLCV data.  FIXME: Requires a more precise definition of "volume patterns".

    Args:
        df: DataFrame with OHLCV data ('open', 'high', 'low', 'close', 'volume', 'date').

    Returns:
        pandas.Series: Boolean Series indicating the presence of volume patterns (True) or absence (False).
    """
    if df.empty:
        return pandas.Series([], dtype=bool)

    # Placeholder:  A simple example - detect days with above-average volume.
    average_volume = df['volume'].mean()
    volume_patterns = df['volume'] > average_volume

    return volume_patterns

# Ref: https://thepatternsite.com/WhiteCandle.html
# This function detects the White Candle pattern based on the description provided in the URL.
# A white candle is defined as a candle with a white body (close > open) and shadows shorter than the body.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans indicating the presence of the pattern.

def do_detect_white_candle(df: pd.DataFrame) -> pd.Series:
    """
    Detects the White Candle pattern in a DataFrame.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.

    Returns:
        Pandas Series with booleans indicating White Candle patterns.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size
    body_size = df["close"] - df["open"]

    # Calculate upper and lower shadow sizes
    upper_shadow = df["high"] - df.apply(lambda row: max(row["open"], row["close"]), axis=1)
    lower_shadow = df.apply(lambda row: min(row["open"], row["close"]), axis=1) - df["low"]

    # Identify white candles (close > open)
    is_white = body_size > 0

    # Check if shadows are shorter than body
    is_short_shadow = (upper_shadow < body_size) & (lower_shadow < body_size)

    # Combine conditions to detect white candle pattern
    white_candle_pattern = is_white & is_short_shadow

    return white_candle_pattern

# Ref: https://thepatternsite.com/LongWhiteDay.html
# This function detects the "Long White Day" candlestick pattern.
# A long white day is characterized by a tall white candle with shadows shorter than the body,
# and a body at least three times taller than the average body height over the last 2 or 3 weeks.


def do_detect_long_white_day(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Long White Day candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        pandas.Series: Boolean Series indicating Long White Day patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate average body height over the last 3 weeks
    avg_body_height = df['close'].sub(df['open']).abs().rolling(window=21).mean()

    #Identify long white candles
    is_long_white = (df['close'] > df['open']) & (df['high'] - df['close'] < df['close'] - df['open']) & (df['close'] - df['low'] < df['close'] - df['open']) & (df['close'].sub(df['open']).abs() >= 3 * avg_body_height)
    
    return is_long_white


# Ref: https://thepatternsite.com/WhiteMarubozu.html
# The white marubozu candlestick is a tall white candle with no shadows.
# It suggests a continuation of the existing price trend but only 56% of the time.
# The function checks for a tall white candle with no upper or lower shadows.

def do_detect_white_marubozu(df: pandas.core.frame.DataFrame) -> pandas.Series:
    """
    Detects the White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.

    Returns:
        pandas.Series: Boolean Series indicating White Marubozu patterns.
    """
    if df.empty:
        return pandas.Series([], dtype=bool)
    
    return (df['close'] > df['open']) & (df['high'] == df['close']) & (df['low'] == df['open'])
# Ref: https://thepatternsite.com/SpinTopWhite.html
# The white spinning top is characterized by a small white body and tall shadows.
# The breakout direction is unpredictable.
# This function identifies white spinning top patterns based on the criteria described in the documentation.
# It calculates the body size and shadow lengths to determine if the pattern is present.


def do_detect_white_spinning_top(df: pd.DataFrame) -> pd.Series:
    """
    Detects the White Spinning Top candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.

    Returns:
        pandas.Series: Boolean Series indicating White Spinning Top patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = df['close'] - df['open']
    upper_shadow = df['high'] - df.apply(lambda row: max(row['open'], row['close']), axis=1)
    lower_shadow = df.apply(lambda row: min(row['open'], row['close']), axis=1) - df['low']

    #Define thresholds.  These should be tuned.
    body_threshold = 0.1 #Ratio of body size to total candle range (high-low)
    shadow_threshold = 2.0 #Ratio of minimum shadow length to the body size

    is_white_spinning_top = (body_size > 0) & (body_size / (df['high'] - df['low']) < body_threshold) & ((upper_shadow > shadow_threshold * abs(body_size)) | (lower_shadow > shadow_threshold * abs(body_size)))

    return is_white_spinning_top


# Ref: https://thepatternsite.com/FallingWindow.html
# This function detects the "Falling Window" candlestick pattern.
# A falling window is characterized by a gap in a downward trend where yesterday's low is above today's high.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans,
# indicating the presence of the pattern for each row.


def do_detect_falling_window(df: pd.DataFrame) -> pd.Series:
    """
    Detects the Falling Window candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

    Returns:
        Pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Shift the 'low' column to align with the next day's 'high'
    shifted_low = df['low'].shift(1)

    # Check for the condition: yesterday's low > today's high
    falling_window = shifted_low > df['high']

    return falling_window


# Ref: https://thepatternsite.com/RisingWindow.html
# This function detects the "Rising Window" candlestick pattern.
# A rising window is characterized by a gap in an upward trend where yesterday's high is below today's low.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans,
# indicating the presence (True) or absence (False) of the pattern for each row.

def do_detect_rising_window(df: pd.DataFrame) -> pd.Series:
    """
    Detects Rising Window candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).

    Returns:
        Pandas Series of booleans indicating the presence of Rising Window pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Shift yesterday's high to compare with today's low
    previous_high = df["high"].shift(1)

    # Detect rising window: yesterday's high < today's low
    rising_window = df["low"] > previous_high

    return rising_window

'\nImportant Notes:\n\nSimplifications: These functions are simplified representations of the patterns. Real-world identification often needs more complex logic and potentially additional conditions. Fine-tuning parameters will be critical for accurate pattern detection in your specific data.\nParameter Tuning: Experiment with different values for parameters (gap_threshold, body_threshold, wick_body_ratio, etc.) to optimize performance for your data.\nVisual Verification: Always visually confirm the patterns identified by the functions. Automated detection can produce false positives or negatives.\nContext Matters: Candlestick patterns are most useful in conjunction with other technical indicators and analysis. Do not base trading decisions solely on automated pattern recognition.\nRemember to thoroughly test these functions with your own data and refine them as needed. These functions are provided as a starting point; you will likely need to further adapt them for optimal performance 

In [16]:
class OptimizerDefinition:
    def __init__(self, name: str, parameters: typing.Dict[str, "ParameterType"], optimization_function, factory=None): 
        if not isinstance(name, str):
            raise TypeError("name must be a string")

        if not isinstance(parameters, dict):
            raise TypeError("parameters must be a dictionary")

        if not all(isinstance(param, ParameterType) for param in parameters.values()):
            raise TypeError("All values in parameters must be ParameterType objects")

        if len(set(parameters.keys())) != len(parameters.keys()): # Check for duplicate keys
            raise ValueError("Parameter names must be unique.")

        if not callable(calculation_function):
            raise TypeError("calculation_function must be callable")

        self.name = name
        self.parameters = parameters
        self.calculation_function = calculation_function
        self.factory = factory

    def create_optimizer(self, **kwargs: typing.Any):
        params = {}
        for name, param_def in self.parameters.items():
            value = kwargs.get(name)

            if value is None:
                value = param_def.get_default()

            if param_def.data_type == "integer" and not isinstance(value, int):
                raise TypeError(f"Value for parameter '{name}' must be an integer")
            elif param_def.data_type == "real" and not isinstance(value, (int, float)):
                raise TypeError(f"Value for parameter '{name}' must be a number")
            elif param_def.data_type == "boolean" and not isinstance(value, bool):
                raise TypeError(f"Value for parameter '{name}' must be a boolean")
            elif param_def.data_type == "string" and not isinstance(value, str):
                raise TypeError(f"Value for parameter '{name}' must be a string")
            elif param_def.data_type in ("integer", "real"):
                if param_def.min_val is not None and value < param_def.min_val:  # Check min_val
                    raise ValueError(f"Value for parameter '{name}' must be greater than or equal to {param_def.min_val}")
                if param_def.max_val is not None and value > param_def.max_val:  # Check max_val
                    raise ValueError(f"Value for parameter '{name}' must be less than or equal to {param_def.max_val}")

            if param_def.data_type == "string" and param_def.allowed_strings is not None and value not in param_def.allowed_strings:
                raise ValueError(f"Value {value} is not in allowed strings for parameter {name}")

            params[name] = value

        return OptimizerInstance(self.name, params, self)

    def calculate(self, data: pd.DataFrame, params: typing.Dict[str, typing.Any]) -> pd.DataFrame:
        """
        Calculates the optimization using the provided data and parameters.
        """
        kwargs = params.copy() 
        return self.optimization_function(data, **kwargs)

    def __repr__(self):
        return f"OptimizerDefinition(name='{self.name}', parameters={self.parameters}, calculation_function={self.calculation_function.__name__ if hasattr(self.calculation_function, '__name__') else str(self.calculation_function)}, factory={self.factory})"